<a href="https://colab.research.google.com/github/bluesoju/SC_Vuln_Detect_Use_LSTM/blob/main/SC_detect_VULN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/NCKH

/content/drive/MyDrive/NCKH


In [ ]:
#Import Lib for LSTM
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
import nltk
nltk.download("punkt")
from tensorflow.keras.preprocessing.text import Tokenizer
import numpy as np

#Load data from File csv
data = pd.read_csv("contract_labled_opcode.csv")

#Filter values NA and replace 0
data = data.fillna(0)

#shuffe Data
data = shuffle(data)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


## Test NLTK

In [ ]:
#import lib for Natural Language Processing
import nltk

In [ ]:
#Append data and label to X_nltk and y_nltk
X_nltk = data['Opcode'].values
Y_nltk = pd.get_dummies(data[['arithmetic', 'clean','reentrancy', 'time_manipulation', 'TOD']]).values

In [ ]:
#Check Type and shape of X, y
print("X shape: ", X_nltk.shape)
print("Y shape: ", Y_nltk.shape)
print("type X: ", type(X_nltk[1]))
print("type Y: ", type(Y_nltk))

X shape:  (704,)
Y shape:  (704, 5)
type X:  <class 'str'>
type Y:  <class 'numpy.ndarray'>


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
X_nltk_vec = vectorizer.fit_transform(X_nltk.ravel())

In [ ]:
X_nltk_vec

<704x60 sparse matrix of type '<class 'numpy.int64'>'
	with 26593 stored elements in Compressed Sparse Row format>

In [ ]:
#feature Name is each opcode from input
vectorizer.get_feature_names()

/usr/local/lib/python3.8/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


['arithmetic_op',
 'balance',
 'byte',
 'call',
 'callcode',
 'calldatacopy',
 'calldataload',
 'calldatasize',
 'callvalue',
 'chainid',
 'codecopy',
 'codesize',
 'comparasion',
 'constant1',
 'constant2',
 'create',
 'create2',
 'delegatecall',
 'dup',
 'eq',
 'extcodecopy',
 'extcodehash',
 'extcodesize',
 'gas',
 'gasprice',
 'getpc',
 'invalid',
 'iszero',
 'jump',
 'jumpdest',
 'jumpi',
 'log',
 'log0',
 'logic_op',
 'mload',
 'msize',
 'mstore',
 'mstore8',
 'pop',
 'push',
 'push1',
 'push2',
 'push3',
 'push4',
 'return',
 'returndatacopy',
 'returndatasize',
 'revert',
 'sar',
 'selfbalance',
 'selfdestruct',
 'sha3',
 'shl',
 'shr',
 'signextend',
 'sload',
 'sstore',
 'staticcall',
 'stop',
 'swap']

In [ ]:
X_nltk_vec = X_nltk_vec.toarray()

In [ ]:
from sklearn.model_selection import train_test_split
X_train_nltk, X_test_nltk, y_train_nltk, y_test_nltk = train_test_split(X_nltk_vec, Y, test_size = 0.2)

In [ ]:
X_train_nltk = np.array(X_train_nltk).astype(np.float32)
X_test_nltk = np.array(X_test_nltk).astype(np.float32)


In [ ]:
# The maximum number of words to be used. (most frequent)
MAX_NB_WORDS = 5000
# Max number of words in each complaint.
MAX_SEQUENCE_LENGTH = 300
# This is fixed.
EMBEDDING_DIM = 100

In [ ]:
#It's my model design from my data

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, SpatialDropout1D, Dropout
from tensorflow.keras.layers import LSTM
from keras.layers import Embedding
from keras.callbacks import ModelCheckpoint, EarlyStopping

model_lstm = Sequential()
model_lstm.add(Embedding(MAX_NB_WORDS, EMBEDDING_DIM, input_length=X_nltk_vec.shape[1]))
model_lstm.add(SpatialDropout1D(0.2))
model_lstm.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2,  activation='tanh'))
model_lstm.add(Dense(64, activation='relu'))
model_lstm.add(Dense(32,activation='tanh'))
model_lstm.add(Dropout(0.2))
model_lstm.add(Dense(5, activation='sigmoid'))
model_lstm.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


epochs = 20 # An epoch is calculated when we feed all the data in the training set to the neural network once
batch_size = 64 #Batch size is the number of data samples in one training session

model_lstm.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_5 (Embedding)     (None, 60, 100)           5000000   
                                                                 
 spatial_dropout1d_5 (Spatia  (None, 60, 100)          0         
 lDropout1D)                                                     
                                                                 
 lstm_5 (LSTM)               (None, 128)               117248    
                                                                 
 dense_15 (Dense)            (None, 64)                8256      
                                                                 
 dense_16 (Dense)            (None, 32)                2080      
                                                                 
 dropout_5 (Dropout)         (None, 32)                0         
                                                      

In [ ]:
history = model_lstm.fit(X_train_nltk, y_train_nltk, epochs=epochs, batch_size=batch_size )


Epoch 1/20
9/9 [==============================] - 13s 538ms/step - loss: 1.5674 - accuracy: 0.3890
Epoch 2/20
9/9 [==============================] - 4s 439ms/step - loss: 1.4318 - accuracy: 0.4423
Epoch 3/20
9/9 [==============================] - 4s 464ms/step - loss: 1.3806 - accuracy: 0.4423
Epoch 4/20
9/9 [==============================] - 5s 531ms/step - loss: 1.3357 - accuracy: 0.4423
Epoch 5/20
9/9 [==============================] - 5s 585ms/step - loss: 1.4438 - accuracy: 0.4529
Epoch 6/20
9/9 [==============================] - 4s 437ms/step - loss: 1.3661 - accuracy: 0.4973
Epoch 7/20
9/9 [==============================] - 4s 451ms/step - loss: 1.2351 - accuracy: 0.4991
Epoch 8/20
9/9 [==============================] - 4s 491ms/step - loss: 1.1368 - accuracy: 0.5382
Epoch 9/20
9/9 [==============================] - 4s 397ms/step - loss: 1.0432 - accuracy: 0.6323
Epoch 10/20
9/9 [==============================] - 3s 348ms/step - loss: 0.9586 - accuracy: 0.6252
Epoch 11/20
9/9 [=

In [ ]:
from keras.utils.data_utils import pad_sequences

# I will move the natural language processing segments and do the prediction 
def Get_vuln(opcode_scan):
  # opcode_scan = np.array(opcode_scan)
  test = vectorizer.fit_transform(opcode_scan)
  test1 = test.toarray()
  print(test1.shape)
  padded = pad_sequences(test1, maxlen=60)
  padded = [padded]
  pred = model_lstm.predict(padded)
  labels = ['arithmetic', 'clean','reentrancy', 'time_manipulation', 'TOD']
  print(pred, labels[np.argmax(pred)])

In [ ]:
opcode_clean = ["PUSH1 PUSH1 MSTORE PUSH1 CALLDATASIZE COMPARASION PUSH2 JUMPI PUSH1 CALLDATALOAD PUSH SWAP ARITHMETIC_OP PUSH4 LOGIC_OP DUP PUSH4 EQ PUSH2 JUMPI DUP PUSH4 EQ PUSH2 JUMPI DUP PUSH4 EQ PUSH2 JUMPI DUP PUSH4 EQ PUSH2 JUMPI DUP PUSH4 EQ PUSH2 JUMPI DUP PUSH4 EQ PUSH2 JUMPI DUP PUSH4 EQ PUSH2 JUMPI DUP PUSH4 EQ PUSH2 JUMPI DUP PUSH4 EQ PUSH2 JUMPI DUP PUSH4 EQ PUSH2 JUMPI DUP PUSH4 EQ PUSH2 JUMPI DUP PUSH4 EQ PUSH2 JUMPI DUP PUSH4 EQ PUSH2 JUMPI DUP PUSH4 EQ PUSH2 JUMPI DUP PUSH4 EQ PUSH2 JUMPI DUP PUSH4 EQ PUSH2 JUMPI DUP PUSH4 EQ PUSH2 JUMPI DUP PUSH4 EQ PUSH2 JUMPI DUP PUSH4 EQ PUSH2 JUMPI DUP PUSH4 EQ PUSH2 JUMPI DUP PUSH4 EQ PUSH2 JUMPI DUP PUSH4 EQ PUSH2 JUMPI DUP PUSH4 EQ PUSH2 JUMPI DUP PUSH4 EQ PUSH2 JUMPI JUMPDEST PUSH1 DUP REVERT JUMPDEST CALLVALUE DUP ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST POP PUSH2 PUSH2 JUMP JUMPDEST PUSH1 MLOAD DUP DUP PUSH LOGIC_OP PUSH LOGIC_OP DUP MSTORE PUSH1 ARITHMETIC_OP SWAP POP POP PUSH1 MLOAD DUP SWAP ARITHMETIC_OP SWAP RETURN JUMPDEST CALLVALUE DUP ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST POP PUSH2 PUSH2 JUMP JUMPDEST STOP JUMPDEST CALLVALUE DUP ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST POP PUSH2 PUSH2 JUMP JUMPDEST PUSH1 MLOAD DUP DUP PUSH LOGIC_OP PUSH LOGIC_OP DUP MSTORE PUSH1 ARITHMETIC_OP SWAP POP POP PUSH1 MLOAD DUP SWAP ARITHMETIC_OP SWAP RETURN JUMPDEST CALLVALUE DUP ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST POP PUSH2 PUSH2 JUMP JUMPDEST PUSH1 MLOAD DUP DUP PUSH LOGIC_OP PUSH LOGIC_OP DUP MSTORE PUSH1 ARITHMETIC_OP SWAP POP POP PUSH1 MLOAD DUP SWAP ARITHMETIC_OP SWAP RETURN JUMPDEST CALLVALUE DUP ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST POP PUSH2 PUSH2 JUMP JUMPDEST PUSH1 MLOAD DUP DUP PUSH LOGIC_OP PUSH LOGIC_OP DUP MSTORE PUSH1 ARITHMETIC_OP SWAP POP POP PUSH1 MLOAD DUP SWAP ARITHMETIC_OP SWAP RETURN JUMPDEST CALLVALUE DUP ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST POP PUSH2 PUSH2 JUMP JUMPDEST PUSH1 MLOAD DUP DUP ISZERO ISZERO ISZERO ISZERO DUP MSTORE PUSH1 ARITHMETIC_OP SWAP POP POP PUSH1 MLOAD DUP SWAP ARITHMETIC_OP SWAP RETURN JUMPDEST CALLVALUE DUP ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST POP PUSH2 PUSH2 JUMP JUMPDEST STOP JUMPDEST CALLVALUE DUP ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST POP PUSH2 PUSH2 JUMP JUMPDEST STOP JUMPDEST CALLVALUE DUP ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST POP PUSH2 PUSH2 JUMP JUMPDEST PUSH1 MLOAD DUP DUP PUSH LOGIC_OP PUSH LOGIC_OP DUP MSTORE PUSH1 ARITHMETIC_OP DUP DUP MSTORE PUSH1 ARITHMETIC_OP DUP PUSH1 ARITHMETIC_OP DUP ISZERO ISZERO ISZERO ISZERO DUP MSTORE PUSH1 ARITHMETIC_OP DUP DUP ARITHMETIC_OP DUP MSTORE DUP DUP DUP MLOAD DUP MSTORE PUSH1 ARITHMETIC_OP SWAP POP DUP MLOAD SWAP PUSH1 ARITHMETIC_OP SWAP PUSH1 ARITHMETIC_OP DUP DUP DUP PUSH1 JUMPDEST DUP DUP COMPARASION ISZERO PUSH2 JUMPI DUP DUP ARITHMETIC_OP MLOAD DUP DUP ARITHMETIC_OP MSTORE PUSH1 DUP ARITHMETIC_OP SWAP POP PUSH2 JUMP JUMPDEST POP POP POP POP SWAP POP ARITHMETIC_OP SWAP POP POP POP POP POP POP PUSH1 MLOAD DUP SWAP ARITHMETIC_OP SWAP RETURN JUMPDEST CALLVALUE DUP ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST POP PUSH2 PUSH1 DUP CALLDATASIZE ARITHMETIC_OP DUP ARITHMETIC_OP SWAP DUP DUP CALLDATALOAD PUSH LOGIC_OP SWAP PUSH1 ARITHMETIC_OP SWAP SWAP SWAP SWAP POP POP POP PUSH2 JUMP JUMPDEST PUSH1 MLOAD DUP DUP ISZERO ISZERO ISZERO ISZERO DUP MSTORE PUSH1 ARITHMETIC_OP SWAP POP POP PUSH1 MLOAD DUP SWAP ARITHMETIC_OP SWAP RETURN JUMPDEST CALLVALUE DUP ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST POP PUSH2 PUSH1 DUP CALLDATASIZE ARITHMETIC_OP DUP ARITHMETIC_OP SWAP DUP DUP CALLDATALOAD PUSH LOGIC_OP SWAP PUSH1 ARITHMETIC_OP SWAP SWAP SWAP SWAP POP POP POP PUSH2 JUMP JUMPDEST STOP JUMPDEST CALLVALUE DUP ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST POP PUSH2 PUSH2 JUMP JUMPDEST PUSH1 MLOAD DUP DUP PUSH LOGIC_OP PUSH LOGIC_OP DUP MSTORE PUSH1 ARITHMETIC_OP SWAP POP POP PUSH1 MLOAD DUP SWAP ARITHMETIC_OP SWAP RETURN JUMPDEST CALLVALUE DUP ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST POP PUSH2 PUSH2 JUMP JUMPDEST PUSH1 MLOAD DUP DUP PUSH LOGIC_OP PUSH LOGIC_OP DUP MSTORE PUSH1 ARITHMETIC_OP SWAP POP POP PUSH1 MLOAD DUP SWAP ARITHMETIC_OP SWAP RETURN JUMPDEST CALLVALUE DUP ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST POP PUSH2 PUSH2 JUMP JUMPDEST STOP JUMPDEST CALLVALUE DUP ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST POP PUSH2 PUSH2 JUMP JUMPDEST PUSH1 MLOAD DUP DUP ISZERO ISZERO ISZERO ISZERO DUP MSTORE PUSH1 ARITHMETIC_OP SWAP POP POP PUSH1 MLOAD DUP SWAP ARITHMETIC_OP SWAP RETURN JUMPDEST CALLVALUE DUP ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST POP PUSH2 PUSH2 JUMP JUMPDEST PUSH1 MLOAD DUP DUP ISZERO ISZERO ISZERO ISZERO DUP MSTORE PUSH1 ARITHMETIC_OP SWAP POP POP PUSH1 MLOAD DUP SWAP ARITHMETIC_OP SWAP RETURN JUMPDEST CALLVALUE DUP ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST POP PUSH2 PUSH1 DUP CALLDATASIZE ARITHMETIC_OP DUP ARITHMETIC_OP SWAP DUP DUP CALLDATALOAD PUSH LOGIC_OP SWAP PUSH1 ARITHMETIC_OP SWAP SWAP SWAP SWAP DUP CALLDATALOAD PUSH LOGIC_OP SWAP PUSH1 ARITHMETIC_OP SWAP SWAP SWAP SWAP POP POP POP PUSH2 JUMP JUMPDEST STOP JUMPDEST CALLVALUE DUP ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST POP PUSH2 PUSH2 JUMP JUMPDEST PUSH1 MLOAD DUP DUP PUSH LOGIC_OP PUSH LOGIC_OP DUP MSTORE PUSH1 ARITHMETIC_OP SWAP POP POP PUSH1 MLOAD DUP SWAP ARITHMETIC_OP SWAP RETURN JUMPDEST CALLVALUE DUP ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST POP PUSH2 PUSH2 JUMP JUMPDEST PUSH1 MLOAD DUP DUP PUSH LOGIC_OP PUSH LOGIC_OP DUP MSTORE PUSH1 ARITHMETIC_OP SWAP POP POP PUSH1 MLOAD DUP SWAP ARITHMETIC_OP SWAP RETURN JUMPDEST CALLVALUE DUP ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST POP PUSH2 PUSH2 JUMP JUMPDEST PUSH1 MLOAD DUP DUP PUSH LOGIC_OP PUSH LOGIC_OP DUP MSTORE PUSH1 ARITHMETIC_OP DUP DUP MSTORE PUSH1 ARITHMETIC_OP DUP ISZERO ISZERO ISZERO ISZERO DUP MSTORE PUSH1 ARITHMETIC_OP SWAP POP POP POP POP PUSH1 MLOAD DUP SWAP ARITHMETIC_OP SWAP RETURN JUMPDEST CALLVALUE DUP ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST POP PUSH2 PUSH2 JUMP JUMPDEST PUSH1 MLOAD DUP DUP ISZERO ISZERO ISZERO ISZERO DUP MSTORE PUSH1 ARITHMETIC_OP SWAP POP POP PUSH1 MLOAD DUP SWAP ARITHMETIC_OP SWAP RETURN JUMPDEST CALLVALUE DUP ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST POP PUSH2 PUSH2 JUMP JUMPDEST PUSH1 MLOAD DUP DUP ISZERO ISZERO ISZERO ISZERO DUP MSTORE PUSH1 ARITHMETIC_OP SWAP POP POP PUSH1 MLOAD DUP SWAP ARITHMETIC_OP SWAP RETURN JUMPDEST CALLVALUE DUP ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST POP PUSH2 PUSH2 JUMP JUMPDEST PUSH1 MLOAD DUP DUP PUSH LOGIC_OP PUSH LOGIC_OP DUP MSTORE PUSH1 ARITHMETIC_OP SWAP POP POP PUSH1 MLOAD DUP SWAP ARITHMETIC_OP SWAP RETURN JUMPDEST CALLVALUE DUP ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST POP PUSH2 PUSH2 JUMP JUMPDEST PUSH1 MLOAD DUP DUP PUSH LOGIC_OP PUSH LOGIC_OP DUP MSTORE PUSH1 ARITHMETIC_OP SWAP POP POP PUSH1 MLOAD DUP SWAP ARITHMETIC_OP SWAP RETURN JUMPDEST PUSH1 PUSH1 SWAP SLOAD SWAP PUSH2 ARITHMETIC_OP SWAP ARITHMETIC_OP PUSH LOGIC_OP DUP JUMP JUMPDEST PUSH1 DUP PUSH1 PUSH2 PUSH2 JUMP JUMPDEST ISZERO ISZERO PUSH2 JUMPI PUSH1 MLOAD PUSH DUP MSTORE PUSH1 ARITHMETIC_OP DUP DUP PUSH1 ARITHMETIC_OP DUP DUP ARITHMETIC_OP DUP MSTORE PUSH1 DUP MSTORE PUSH1 ARITHMETIC_OP DUP PUSH DUP MSTORE POP PUSH1 ARITHMETIC_OP SWAP POP POP PUSH1 MLOAD DUP SWAP ARITHMETIC_OP SWAP REVERT JUMPDEST PUSH2 PUSH2 JUMP JUMPDEST ISZERO ISZERO ISZERO PUSH2 JUMPI PUSH1 MLOAD PUSH DUP MSTORE PUSH1 ARITHMETIC_OP DUP DUP PUSH1 ARITHMETIC_OP DUP DUP ARITHMETIC_OP DUP MSTORE PUSH1 DUP MSTORE PUSH1 ARITHMETIC_OP DUP PUSH DUP MSTORE PUSH1 ARITHMETIC_OP PUSH DUP MSTORE POP PUSH1 ARITHMETIC_OP SWAP POP POP PUSH1 MLOAD DUP SWAP ARITHMETIC_OP SWAP REVERT JUMPDEST PUSH2 PUSH2 JUMP JUMPDEST SWAP POP PUSH1 PUSH1 SWAP SLOAD SWAP PUSH2 ARITHMETIC_OP SWAP ARITHMETIC_OP PUSH LOGIC_OP PUSH LOGIC_OP PUSH4 CONSTANT2 PUSH1 MLOAD DUP PUSH4 LOGIC_OP PUSH ARITHMETIC_OP DUP MSTORE PUSH1 ARITHMETIC_OP DUP DUP PUSH LOGIC_OP PUSH LOGIC_OP DUP MSTORE PUSH1 ARITHMETIC_OP SWAP POP POP PUSH1 DUP MLOAD DUP DUP ARITHMETIC_OP DUP PUSH1 DUP DUP EXTCODESIZE ISZERO DUP ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST POP GAS CALL ISZERO DUP ISZERO PUSH2 JUMPI RETURNDATASIZE PUSH1 DUP RETURNDATACOPY RETURNDATASIZE PUSH1 REVERT JUMPDEST POP POP POP POP PUSH1 MLOAD RETURNDATASIZE PUSH1 DUP COMPARASION ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST DUP ARITHMETIC_OP SWAP DUP DUP MLOAD SWAP PUSH1 ARITHMETIC_OP SWAP SWAP SWAP SWAP DUP MLOAD SWAP PUSH1 ARITHMETIC_OP SWAP SWAP SWAP SWAP POP POP POP DUP SWAP POP DUP SWAP POP POP POP DUP PUSH LOGIC_OP PUSH4 PUSH1 PUSH1 SWAP SLOAD SWAP PUSH2 ARITHMETIC_OP SWAP ARITHMETIC_OP PUSH LOGIC_OP CONSTANT2 DUP PUSH1 MLOAD DUP PUSH4 LOGIC_OP PUSH ARITHMETIC_OP DUP MSTORE PUSH1 ARITHMETIC_OP DUP DUP PUSH LOGIC_OP PUSH LOGIC_OP DUP MSTORE PUSH1 ARITHMETIC_OP DUP PUSH LOGIC_OP PUSH LOGIC_OP DUP MSTORE PUSH1 ARITHMETIC_OP DUP PUSH LOGIC_OP DUP MSTORE PUSH1 ARITHMETIC_OP SWAP POP POP POP POP PUSH1 PUSH1 MLOAD DUP DUP ARITHMETIC_OP DUP PUSH1 DUP DUP EXTCODESIZE ISZERO DUP ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST POP GAS CALL ISZERO DUP ISZERO PUSH2 JUMPI RETURNDATASIZE PUSH1 DUP RETURNDATACOPY RETURNDATASIZE PUSH1 REVERT JUMPDEST POP POP POP POP PUSH1 MLOAD RETURNDATASIZE PUSH1 DUP COMPARASION ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST DUP ARITHMETIC_OP SWAP DUP DUP MLOAD SWAP PUSH1 ARITHMETIC_OP SWAP SWAP SWAP SWAP POP POP POP ISZERO ISZERO PUSH2 JUMPI INVALID JUMPDEST DUP PUSH LOGIC_OP PUSH4 CONSTANT2 DUP PUSH1 MLOAD DUP PUSH4 LOGIC_OP PUSH ARITHMETIC_OP DUP MSTORE PUSH1 ARITHMETIC_OP DUP DUP PUSH LOGIC_OP PUSH LOGIC_OP DUP MSTORE PUSH1 ARITHMETIC_OP DUP PUSH LOGIC_OP DUP MSTORE PUSH1 ARITHMETIC_OP SWAP POP POP POP PUSH1 PUSH1 MLOAD DUP DUP ARITHMETIC_OP DUP PUSH1 DUP DUP EXTCODESIZE ISZERO DUP ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST POP GAS CALL ISZERO DUP ISZERO PUSH2 JUMPI RETURNDATASIZE PUSH1 DUP RETURNDATACOPY RETURNDATASIZE PUSH1 REVERT JUMPDEST POP POP POP POP PUSH1 MLOAD RETURNDATASIZE PUSH1 DUP COMPARASION ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST DUP ARITHMETIC_OP SWAP DUP DUP MLOAD SWAP PUSH1 ARITHMETIC_OP SWAP SWAP SWAP SWAP POP POP POP ISZERO ISZERO PUSH2 JUMPI INVALID JUMPDEST PUSH2 PUSH2 JUMP JUMPDEST PUSH CONSTANT2 DUP PUSH1 MLOAD DUP DUP PUSH LOGIC_OP PUSH LOGIC_OP DUP MSTORE PUSH1 ARITHMETIC_OP DUP PUSH LOGIC_OP PUSH LOGIC_OP DUP MSTORE PUSH1 ARITHMETIC_OP SWAP POP POP POP PUSH1 MLOAD DUP SWAP ARITHMETIC_OP SWAP LOG POP POP POP JUMP JUMPDEST PUSH1 PUSH1 SWAP SLOAD SWAP PUSH2 ARITHMETIC_OP SWAP ARITHMETIC_OP PUSH LOGIC_OP DUP JUMP JUMPDEST PUSH1 PUSH1 SWAP SLOAD SWAP PUSH2 ARITHMETIC_OP SWAP ARITHMETIC_OP PUSH LOGIC_OP DUP JUMP JUMPDEST PUSH1 PUSH2 PUSH2 JUMP JUMPDEST ISZERO ISZERO PUSH2 JUMPI PUSH1 MLOAD PUSH DUP MSTORE PUSH1 ARITHMETIC_OP DUP DUP PUSH1 ARITHMETIC_OP DUP DUP ARITHMETIC_OP DUP MSTORE PUSH1 DUP MSTORE PUSH1 ARITHMETIC_OP DUP PUSH DUP MSTORE POP PUSH1 ARITHMETIC_OP SWAP POP POP PUSH1 MLOAD DUP SWAP ARITHMETIC_OP SWAP REVERT JUMPDEST PUSH1 PUSH1 SWAP SLOAD SWAP PUSH2 ARITHMETIC_OP SWAP ARITHMETIC_OP PUSH LOGIC_OP SWAP POP SWAP JUMP JUMPDEST PUSH1 DUP PUSH1 SWAP SLOAD SWAP PUSH2 ARITHMETIC_OP SWAP ARITHMETIC_OP PUSH1 LOGIC_OP SWAP POP SWAP JUMP JUMPDEST PUSH1 DUP PUSH1 DUP PUSH1 DUP PUSH2 PUSH2 JUMP JUMPDEST ISZERO ISZERO PUSH2 JUMPI PUSH2 PUSH2 JUMP JUMPDEST PUSH2 PUSH2 JUMP JUMPDEST ISZERO ISZERO PUSH2 JUMPI INVALID JUMPDEST JUMPDEST PUSH1 PUSH1 SWAP SLOAD SWAP PUSH2 ARITHMETIC_OP SWAP ARITHMETIC_OP PUSH1 LOGIC_OP ISZERO ISZERO ISZERO PUSH2 JUMPI PUSH1 MLOAD PUSH DUP MSTORE PUSH1 ARITHMETIC_OP DUP DUP PUSH1 ARITHMETIC_OP DUP DUP ARITHMETIC_OP DUP MSTORE PUSH1 DUP MSTORE PUSH1 ARITHMETIC_OP DUP PUSH DUP MSTORE POP PUSH1 ARITHMETIC_OP SWAP POP POP PUSH1 MLOAD DUP SWAP ARITHMETIC_OP SWAP REVERT JUMPDEST PUSH1 PUSH1 PUSH1 PUSH2 ARITHMETIC_OP DUP SLOAD DUP PUSH1 ARITHMETIC_OP LOGIC_OP LOGIC_OP SWAP DUP ISZERO ISZERO ARITHMETIC_OP LOGIC_OP SWAP SSTORE POP PUSH1 PUSH1 SWAP SLOAD SWAP PUSH2 ARITHMETIC_OP SWAP ARITHMETIC_OP PUSH LOGIC_OP PUSH LOGIC_OP PUSH4 PUSH1 MLOAD DUP PUSH4 LOGIC_OP PUSH ARITHMETIC_OP DUP MSTORE PUSH1 ARITHMETIC_OP PUSH1 PUSH1 MLOAD DUP DUP ARITHMETIC_OP DUP PUSH1 DUP DUP EXTCODESIZE ISZERO DUP ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST POP GAS CALL ISZERO DUP ISZERO PUSH2 JUMPI RETURNDATASIZE PUSH1 DUP RETURNDATACOPY RETURNDATASIZE PUSH1 REVERT JUMPDEST POP POP POP POP PUSH1 MLOAD RETURNDATASIZE PUSH1 DUP COMPARASION ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST DUP ARITHMETIC_OP SWAP DUP DUP MLOAD SWAP PUSH1 ARITHMETIC_OP SWAP SWAP SWAP SWAP POP POP POP SWAP POP PUSH1 DUP PUSH LOGIC_OP DUP ISZERO ISZERO PUSH2 JUMPI INVALID JUMPDEST ARITHMETIC_OP SWAP POP DUP DUP ARITHMETIC_OP SWAP POP DUP PUSH LOGIC_OP DUP DUP ARITHMETIC_OP PUSH LOGIC_OP EQ ISZERO ISZERO PUSH2 JUMPI INVALID JUMPDEST PUSH1 PUSH1 SWAP SLOAD SWAP PUSH2 ARITHMETIC_OP SWAP ARITHMETIC_OP PUSH LOGIC_OP PUSH LOGIC_OP PUSH4 PUSH1 MLOAD DUP PUSH4 LOGIC_OP PUSH ARITHMETIC_OP DUP MSTORE PUSH1 ARITHMETIC_OP PUSH1 PUSH1 MLOAD DUP DUP ARITHMETIC_OP DUP PUSH1 DUP DUP EXTCODESIZE ISZERO DUP ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST POP GAS CALL ISZERO DUP ISZERO PUSH2 JUMPI RETURNDATASIZE PUSH1 DUP RETURNDATACOPY RETURNDATASIZE PUSH1 REVERT JUMPDEST POP POP POP POP PUSH1 MLOAD RETURNDATASIZE PUSH1 DUP COMPARASION ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST DUP ARITHMETIC_OP SWAP DUP DUP MLOAD SWAP PUSH1 ARITHMETIC_OP SWAP SWAP SWAP SWAP POP POP POP SWAP POP PUSH1 PUSH1 SWAP SLOAD SWAP PUSH2 ARITHMETIC_OP SWAP ARITHMETIC_OP PUSH LOGIC_OP PUSH LOGIC_OP PUSH4 PUSH1 MLOAD DUP PUSH4 LOGIC_OP PUSH ARITHMETIC_OP DUP MSTORE PUSH1 ARITHMETIC_OP PUSH1 PUSH1 MLOAD DUP DUP ARITHMETIC_OP DUP PUSH1 DUP DUP EXTCODESIZE ISZERO DUP ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST POP GAS CALL ISZERO DUP ISZERO PUSH2 JUMPI RETURNDATASIZE PUSH1 DUP RETURNDATACOPY RETURNDATASIZE PUSH1 REVERT JUMPDEST POP POP POP POP PUSH1 MLOAD RETURNDATASIZE PUSH1 DUP COMPARASION ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST DUP ARITHMETIC_OP SWAP DUP DUP MLOAD SWAP PUSH1 ARITHMETIC_OP SWAP SWAP SWAP SWAP POP POP POP SWAP POP PUSH2 PUSH2 JUMP JUMPDEST SWAP POP DUP PUSH LOGIC_OP PUSH4 DUP DUP PUSH1 MLOAD DUP PUSH4 LOGIC_OP PUSH ARITHMETIC_OP DUP MSTORE PUSH1 ARITHMETIC_OP DUP DUP PUSH LOGIC_OP PUSH LOGIC_OP DUP MSTORE PUSH1 ARITHMETIC_OP DUP PUSH LOGIC_OP DUP MSTORE PUSH1 ARITHMETIC_OP SWAP POP POP POP PUSH1 PUSH1 MLOAD DUP DUP ARITHMETIC_OP DUP PUSH1 DUP DUP EXTCODESIZE ISZERO DUP ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST POP GAS CALL ISZERO DUP ISZERO PUSH2 JUMPI RETURNDATASIZE PUSH1 DUP RETURNDATACOPY RETURNDATASIZE PUSH1 REVERT JUMPDEST POP POP POP POP PUSH1 MLOAD RETURNDATASIZE PUSH1 DUP COMPARASION ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST DUP ARITHMETIC_OP SWAP DUP DUP MLOAD SWAP PUSH1 ARITHMETIC_OP SWAP SWAP SWAP SWAP POP POP POP ISZERO ISZERO PUSH2 JUMPI INVALID JUMPDEST DUP PUSH LOGIC_OP PUSH4 DUP DUP PUSH1 MLOAD DUP PUSH4 LOGIC_OP PUSH ARITHMETIC_OP DUP MSTORE PUSH1 ARITHMETIC_OP DUP DUP PUSH LOGIC_OP PUSH LOGIC_OP DUP MSTORE PUSH1 ARITHMETIC_OP DUP PUSH LOGIC_OP DUP MSTORE PUSH1 ARITHMETIC_OP SWAP POP POP POP PUSH1 PUSH1 MLOAD DUP DUP ARITHMETIC_OP DUP PUSH1 DUP DUP EXTCODESIZE ISZERO DUP ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST POP GAS CALL ISZERO DUP ISZERO PUSH2 JUMPI RETURNDATASIZE PUSH1 DUP RETURNDATACOPY RETURNDATASIZE PUSH1 REVERT JUMPDEST POP POP POP POP PUSH1 MLOAD RETURNDATASIZE PUSH1 DUP COMPARASION ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST DUP ARITHMETIC_OP SWAP DUP DUP MLOAD SWAP PUSH1 ARITHMETIC_OP SWAP SWAP SWAP SWAP POP POP POP ISZERO ISZERO PUSH2 JUMPI INVALID JUMPDEST PUSH DUP DUP DUP DUP PUSH1 MLOAD DUP DUP PUSH LOGIC_OP PUSH LOGIC_OP DUP MSTORE PUSH1 ARITHMETIC_OP DUP PUSH LOGIC_OP PUSH LOGIC_OP DUP MSTORE PUSH1 ARITHMETIC_OP DUP PUSH LOGIC_OP PUSH LOGIC_OP DUP MSTORE PUSH1 ARITHMETIC_OP DUP PUSH LOGIC_OP PUSH LOGIC_OP DUP MSTORE PUSH1 ARITHMETIC_OP SWAP POP POP POP POP POP PUSH1 MLOAD DUP SWAP ARITHMETIC_OP SWAP LOG POP POP POP POP POP POP JUMP JUMPDEST PUSH1 DUP PUSH1 DUP PUSH2 PUSH2 JUMP JUMPDEST ISZERO ISZERO PUSH2 JUMPI PUSH1 MLOAD PUSH DUP MSTORE PUSH1 ARITHMETIC_OP DUP DUP PUSH1 ARITHMETIC_OP DUP DUP ARITHMETIC_OP DUP MSTORE PUSH1 DUP MSTORE PUSH1 ARITHMETIC_OP DUP PUSH DUP MSTORE POP PUSH1 ARITHMETIC_OP SWAP POP POP PUSH1 MLOAD DUP SWAP ARITHMETIC_OP SWAP REVERT JUMPDEST PUSH2 PUSH2 JUMP JUMPDEST ISZERO ISZERO PUSH2 JUMPI PUSH1 MLOAD PUSH DUP MSTORE PUSH1 ARITHMETIC_OP DUP DUP PUSH1 ARITHMETIC_OP DUP DUP ARITHMETIC_OP DUP MSTORE PUSH1 DUP MSTORE PUSH1 ARITHMETIC_OP DUP PUSH DUP MSTORE POP PUSH1 ARITHMETIC_OP SWAP POP POP PUSH1 MLOAD DUP SWAP ARITHMETIC_OP SWAP REVERT JUMPDEST PUSH2 PUSH2 JUMP JUMPDEST ISZERO ISZERO ISZERO PUSH2 JUMPI PUSH1 MLOAD PUSH DUP MSTORE PUSH1 ARITHMETIC_OP DUP DUP PUSH1 ARITHMETIC_OP DUP DUP ARITHMETIC_OP DUP MSTORE PUSH1 DUP MSTORE PUSH1 ARITHMETIC_OP DUP PUSH DUP MSTORE POP PUSH1 ARITHMETIC_OP SWAP POP POP PUSH1 MLOAD DUP SWAP ARITHMETIC_OP SWAP REVERT JUMPDEST PUSH2 PUSH2 JUMP JUMPDEST SWAP POP PUSH2 PUSH2 JUMP JUMPDEST SWAP POP PUSH1 PUSH1 SWAP SLOAD SWAP PUSH2 ARITHMETIC_OP SWAP ARITHMETIC_OP PUSH LOGIC_OP PUSH LOGIC_OP PUSH4 PUSH1 MLOAD DUP PUSH4 LOGIC_OP PUSH ARITHMETIC_OP DUP MSTORE PUSH1 ARITHMETIC_OP PUSH1 PUSH1 MLOAD DUP DUP ARITHMETIC_OP DUP PUSH1 DUP DUP EXTCODESIZE ISZERO DUP ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST POP GAS CALL ISZERO DUP ISZERO PUSH2 JUMPI RETURNDATASIZE PUSH1 DUP RETURNDATACOPY RETURNDATASIZE PUSH1 REVERT JUMPDEST POP POP POP POP DUP PUSH LOGIC_OP PUSH4 PUSH1 PUSH1 SWAP SLOAD SWAP PUSH2 ARITHMETIC_OP SWAP ARITHMETIC_OP PUSH LOGIC_OP CONSTANT2 DUP PUSH LOGIC_OP PUSH4 PUSH1 PUSH1 SWAP SLOAD SWAP PUSH2 ARITHMETIC_OP SWAP ARITHMETIC_OP PUSH LOGIC_OP PUSH1 MLOAD DUP PUSH4 LOGIC_OP PUSH ARITHMETIC_OP DUP MSTORE PUSH1 ARITHMETIC_OP DUP DUP PUSH LOGIC_OP PUSH LOGIC_OP DUP MSTORE PUSH1 ARITHMETIC_OP SWAP POP POP PUSH1 PUSH1 MLOAD DUP DUP ARITHMETIC_OP DUP PUSH1 DUP DUP EXTCODESIZE ISZERO DUP ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST POP GAS CALL ISZERO DUP ISZERO PUSH2 JUMPI RETURNDATASIZE PUSH1 DUP RETURNDATACOPY RETURNDATASIZE PUSH1 REVERT JUMPDEST POP POP POP POP PUSH1 MLOAD RETURNDATASIZE PUSH1 DUP COMPARASION ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST DUP ARITHMETIC_OP SWAP DUP DUP MLOAD SWAP PUSH1 ARITHMETIC_OP SWAP SWAP SWAP SWAP POP POP POP PUSH1 MLOAD DUP PUSH4 LOGIC_OP PUSH ARITHMETIC_OP DUP MSTORE PUSH1 ARITHMETIC_OP DUP DUP PUSH LOGIC_OP PUSH LOGIC_OP DUP MSTORE PUSH1 ARITHMETIC_OP DUP PUSH LOGIC_OP PUSH LOGIC_OP DUP MSTORE PUSH1 ARITHMETIC_OP DUP DUP MSTORE PUSH1 ARITHMETIC_OP SWAP POP POP POP POP PUSH1 PUSH1 MLOAD DUP DUP ARITHMETIC_OP DUP PUSH1 DUP DUP EXTCODESIZE ISZERO DUP ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST POP GAS CALL ISZERO DUP ISZERO PUSH2 JUMPI RETURNDATASIZE PUSH1 DUP RETURNDATACOPY RETURNDATASIZE PUSH1 REVERT JUMPDEST POP POP POP POP PUSH1 MLOAD RETURNDATASIZE PUSH1 DUP COMPARASION ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST DUP ARITHMETIC_OP SWAP DUP DUP MLOAD SWAP PUSH1 ARITHMETIC_OP SWAP SWAP SWAP SWAP POP POP POP ISZERO ISZERO PUSH2 JUMPI INVALID JUMPDEST DUP PUSH LOGIC_OP PUSH4 PUSH1 PUSH1 SWAP SLOAD SWAP PUSH2 ARITHMETIC_OP SWAP ARITHMETIC_OP PUSH LOGIC_OP CONSTANT2 DUP PUSH LOGIC_OP PUSH4 PUSH1 PUSH1 SWAP SLOAD SWAP PUSH2 ARITHMETIC_OP SWAP ARITHMETIC_OP PUSH LOGIC_OP PUSH1 MLOAD DUP PUSH4 LOGIC_OP PUSH ARITHMETIC_OP DUP MSTORE PUSH1 ARITHMETIC_OP DUP DUP PUSH LOGIC_OP PUSH LOGIC_OP DUP MSTORE PUSH1 ARITHMETIC_OP SWAP POP POP PUSH1 PUSH1 MLOAD DUP DUP ARITHMETIC_OP DUP PUSH1 DUP DUP EXTCODESIZE ISZERO DUP ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST POP GAS CALL ISZERO DUP ISZERO PUSH2 JUMPI RETURNDATASIZE PUSH1 DUP RETURNDATACOPY RETURNDATASIZE PUSH1 REVERT JUMPDEST POP POP POP POP PUSH1 MLOAD RETURNDATASIZE PUSH1 DUP COMPARASION ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST DUP ARITHMETIC_OP SWAP DUP DUP MLOAD SWAP PUSH1 ARITHMETIC_OP SWAP SWAP SWAP SWAP POP POP POP PUSH1 MLOAD DUP PUSH4 LOGIC_OP PUSH ARITHMETIC_OP DUP MSTORE PUSH1 ARITHMETIC_OP DUP DUP PUSH LOGIC_OP PUSH LOGIC_OP DUP MSTORE PUSH1 ARITHMETIC_OP DUP PUSH LOGIC_OP PUSH LOGIC_OP DUP MSTORE PUSH1 ARITHMETIC_OP DUP DUP MSTORE PUSH1 ARITHMETIC_OP SWAP POP POP POP POP PUSH1 PUSH1 MLOAD DUP DUP ARITHMETIC_OP DUP PUSH1 DUP DUP EXTCODESIZE ISZERO DUP ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST POP GAS CALL ISZERO DUP ISZERO PUSH2 JUMPI RETURNDATASIZE PUSH1 DUP RETURNDATACOPY RETURNDATASIZE PUSH1 REVERT JUMPDEST POP POP POP POP PUSH1 MLOAD RETURNDATASIZE PUSH1 DUP COMPARASION ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST DUP ARITHMETIC_OP SWAP DUP DUP MLOAD SWAP PUSH1 ARITHMETIC_OP SWAP SWAP SWAP SWAP POP POP POP ISZERO ISZERO PUSH2 JUMPI INVALID JUMPDEST DUP PUSH LOGIC_OP PUSH4 CONSTANT2 PUSH1 MLOAD DUP PUSH4 LOGIC_OP PUSH ARITHMETIC_OP DUP MSTORE PUSH1 ARITHMETIC_OP DUP DUP PUSH LOGIC_OP PUSH LOGIC_OP DUP MSTORE PUSH1 ARITHMETIC_OP SWAP POP POP PUSH1 PUSH1 MLOAD DUP DUP ARITHMETIC_OP DUP PUSH1 DUP DUP EXTCODESIZE ISZERO DUP ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST POP GAS CALL ISZERO DUP ISZERO PUSH2 JUMPI RETURNDATASIZE PUSH1 DUP RETURNDATACOPY RETURNDATASIZE PUSH1 REVERT JUMPDEST POP POP POP POP PUSH1 MLOAD RETURNDATASIZE PUSH1 DUP COMPARASION ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST DUP ARITHMETIC_OP SWAP DUP DUP MLOAD SWAP PUSH1 ARITHMETIC_OP SWAP SWAP SWAP SWAP POP POP POP SWAP POP DUP SWAP POP DUP DUP PUSH LOGIC_OP EQ ISZERO ISZERO PUSH2 JUMPI INVALID JUMPDEST PUSH1 PUSH1 SWAP SLOAD SWAP PUSH2 ARITHMETIC_OP SWAP ARITHMETIC_OP PUSH LOGIC_OP PUSH LOGIC_OP PUSH4 DUP PUSH1 MLOAD DUP PUSH4 LOGIC_OP PUSH ARITHMETIC_OP DUP MSTORE PUSH1 ARITHMETIC_OP DUP DUP PUSH LOGIC_OP PUSH LOGIC_OP DUP MSTORE PUSH1 ARITHMETIC_OP SWAP POP POP PUSH1 PUSH1 MLOAD DUP DUP ARITHMETIC_OP DUP PUSH1 DUP DUP EXTCODESIZE ISZERO DUP ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST POP GAS CALL ISZERO DUP ISZERO PUSH2 JUMPI RETURNDATASIZE PUSH1 DUP RETURNDATACOPY RETURNDATASIZE PUSH1 REVERT JUMPDEST POP POP POP POP PUSH2 PUSH2 JUMP JUMPDEST ISZERO ISZERO PUSH2 JUMPI INVALID JUMPDEST PUSH DUP PUSH1 MLOAD DUP DUP PUSH LOGIC_OP PUSH LOGIC_OP DUP MSTORE PUSH1 ARITHMETIC_OP SWAP POP POP PUSH1 MLOAD DUP SWAP ARITHMETIC_OP SWAP LOG POP POP POP POP JUMP JUMPDEST PUSH1 DUP PUSH1 PUSH1 PUSH1 PUSH1 ARITHMETIC_OP PUSH1 SWAP SLOAD SWAP PUSH2 ARITHMETIC_OP SWAP ARITHMETIC_OP PUSH LOGIC_OP PUSH1 DUP ARITHMETIC_OP SLOAD PUSH1 PUSH1 ARITHMETIC_OP PUSH1 PUSH1 ARITHMETIC_OP PUSH1 SWAP SLOAD SWAP PUSH2 ARITHMETIC_OP SWAP ARITHMETIC_OP PUSH1 LOGIC_OP DUP DUP SLOAD DUP PUSH1 ARITHMETIC_OP PUSH1 ARITHMETIC_OP PUSH1 MLOAD SWAP DUP ARITHMETIC_OP PUSH1 MSTORE DUP SWAP SWAP SWAP DUP DUP MSTORE PUSH1 ARITHMETIC_OP DUP DUP SLOAD DUP ISZERO PUSH2 JUMPI PUSH1 ARITHMETIC_OP DUP ARITHMETIC_OP SWAP SWAP PUSH1 MSTORE PUSH1 PUSH1 SHA3 SWAP JUMPDEST DUP SLOAD DUP MSTORE PUSH1 ARITHMETIC_OP SWAP PUSH1 ARITHMETIC_OP SWAP DUP DUP COMPARASION PUSH2 JUMPI JUMPDEST POP POP POP POP POP SWAP POP SWAP POP SWAP POP SWAP POP SWAP POP SWAP SWAP SWAP SWAP JUMP JUMPDEST PUSH1 PUSH1 MSTORE DUP PUSH1 MSTORE PUSH1 PUSH1 SHA3 PUSH1 SWAP POP SLOAD SWAP PUSH2 ARITHMETIC_OP SWAP ARITHMETIC_OP PUSH1 LOGIC_OP DUP JUMP JUMPDEST PUSH1 DUP PUSH1 DUP PUSH2 PUSH2 JUMP JUMPDEST ISZERO ISZERO PUSH2 JUMPI PUSH2 PUSH2 JUMP JUMPDEST PUSH2 PUSH2 JUMP JUMPDEST ISZERO ISZERO PUSH2 JUMPI INVALID JUMPDEST JUMPDEST PUSH1 PUSH1 DUP PUSH LOGIC_OP PUSH LOGIC_OP DUP MSTORE PUSH1 ARITHMETIC_OP SWAP DUP MSTORE PUSH1 ARITHMETIC_OP PUSH1 SHA3 PUSH1 SWAP SLOAD SWAP PUSH2 ARITHMETIC_OP SWAP ARITHMETIC_OP PUSH1 LOGIC_OP ISZERO ISZERO ISZERO PUSH2 JUMPI PUSH1 MLOAD PUSH DUP MSTORE PUSH1 ARITHMETIC_OP DUP DUP PUSH1 ARITHMETIC_OP DUP DUP ARITHMETIC_OP DUP MSTORE PUSH1 DUP MSTORE PUSH1 ARITHMETIC_OP DUP PUSH DUP MSTORE POP PUSH1 ARITHMETIC_OP SWAP POP POP PUSH1 MLOAD DUP SWAP ARITHMETIC_OP SWAP REVERT JUMPDEST PUSH1 PUSH1 PUSH1 DUP PUSH LOGIC_OP PUSH LOGIC_OP DUP MSTORE PUSH1 ARITHMETIC_OP SWAP DUP MSTORE PUSH1 ARITHMETIC_OP PUSH1 SHA3 PUSH1 PUSH2 ARITHMETIC_OP DUP SLOAD DUP PUSH1 ARITHMETIC_OP LOGIC_OP LOGIC_OP SWAP DUP ISZERO ISZERO ARITHMETIC_OP LOGIC_OP SWAP SSTORE POP PUSH1 PUSH1 SWAP SLOAD SWAP PUSH2 ARITHMETIC_OP SWAP ARITHMETIC_OP PUSH LOGIC_OP PUSH LOGIC_OP PUSH4 DUP PUSH1 MLOAD DUP PUSH4 LOGIC_OP PUSH ARITHMETIC_OP DUP MSTORE PUSH1 ARITHMETIC_OP DUP DUP PUSH LOGIC_OP PUSH LOGIC_OP DUP MSTORE PUSH1 ARITHMETIC_OP SWAP POP POP PUSH1 DUP MLOAD DUP DUP ARITHMETIC_OP DUP PUSH1 DUP DUP EXTCODESIZE ISZERO DUP ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST POP GAS CALL ISZERO DUP ISZERO PUSH2 JUMPI RETURNDATASIZE PUSH1 DUP RETURNDATACOPY RETURNDATASIZE PUSH1 REVERT JUMPDEST POP POP POP POP PUSH1 MLOAD RETURNDATASIZE PUSH1 DUP COMPARASION ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST DUP ARITHMETIC_OP SWAP DUP DUP MLOAD SWAP PUSH1 ARITHMETIC_OP SWAP SWAP SWAP SWAP DUP MLOAD SWAP PUSH1 ARITHMETIC_OP SWAP SWAP SWAP SWAP POP POP POP DUP SWAP POP DUP SWAP POP POP POP PUSH2 PUSH2 JUMP JUMPDEST SWAP POP PUSH2 PUSH2 JUMP JUMPDEST SWAP POP DUP PUSH LOGIC_OP PUSH4 DUP DUP PUSH1 MLOAD DUP PUSH4 LOGIC_OP PUSH ARITHMETIC_OP DUP MSTORE PUSH1 ARITHMETIC_OP DUP DUP PUSH LOGIC_OP PUSH LOGIC_OP DUP MSTORE PUSH1 ARITHMETIC_OP DUP PUSH LOGIC_OP DUP MSTORE PUSH1 ARITHMETIC_OP SWAP POP POP POP PUSH1 PUSH1 MLOAD DUP DUP ARITHMETIC_OP DUP PUSH1 DUP DUP EXTCODESIZE ISZERO DUP ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST POP GAS CALL ISZERO DUP ISZERO PUSH2 JUMPI RETURNDATASIZE PUSH1 DUP RETURNDATACOPY RETURNDATASIZE PUSH1 REVERT JUMPDEST POP POP POP POP PUSH1 MLOAD RETURNDATASIZE PUSH1 DUP COMPARASION ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST DUP ARITHMETIC_OP SWAP DUP DUP MLOAD SWAP PUSH1 ARITHMETIC_OP SWAP SWAP SWAP SWAP POP POP POP ISZERO ISZERO PUSH2 JUMPI INVALID JUMPDEST DUP PUSH LOGIC_OP PUSH4 DUP DUP PUSH1 MLOAD DUP PUSH4 LOGIC_OP PUSH ARITHMETIC_OP DUP MSTORE PUSH1 ARITHMETIC_OP DUP DUP PUSH LOGIC_OP PUSH LOGIC_OP DUP MSTORE PUSH1 ARITHMETIC_OP DUP PUSH LOGIC_OP DUP MSTORE PUSH1 ARITHMETIC_OP SWAP POP POP POP PUSH1 PUSH1 MLOAD DUP DUP ARITHMETIC_OP DUP PUSH1 DUP DUP EXTCODESIZE ISZERO DUP ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST POP GAS CALL ISZERO DUP ISZERO PUSH2 JUMPI RETURNDATASIZE PUSH1 DUP RETURNDATACOPY RETURNDATASIZE PUSH1 REVERT JUMPDEST POP POP POP POP PUSH1 MLOAD RETURNDATASIZE PUSH1 DUP COMPARASION ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST DUP ARITHMETIC_OP SWAP DUP DUP MLOAD SWAP PUSH1 ARITHMETIC_OP SWAP SWAP SWAP SWAP POP POP POP ISZERO ISZERO PUSH2 JUMPI INVALID JUMPDEST PUSH DUP DUP DUP PUSH1 MLOAD DUP DUP PUSH LOGIC_OP PUSH LOGIC_OP DUP MSTORE PUSH1 ARITHMETIC_OP DUP PUSH LOGIC_OP PUSH LOGIC_OP DUP MSTORE PUSH1 ARITHMETIC_OP DUP PUSH LOGIC_OP PUSH LOGIC_OP DUP MSTORE PUSH1 ARITHMETIC_OP SWAP POP POP POP POP PUSH1 MLOAD DUP SWAP ARITHMETIC_OP SWAP LOG POP POP POP POP POP JUMP JUMPDEST PUSH1 PUSH2 PUSH2 JUMP JUMPDEST ISZERO ISZERO PUSH2 JUMPI PUSH1 MLOAD PUSH DUP MSTORE PUSH1 ARITHMETIC_OP DUP DUP PUSH1 ARITHMETIC_OP DUP DUP ARITHMETIC_OP DUP MSTORE PUSH1 DUP MSTORE PUSH1 ARITHMETIC_OP DUP PUSH DUP MSTORE POP PUSH1 ARITHMETIC_OP SWAP POP POP PUSH1 MLOAD DUP SWAP ARITHMETIC_OP SWAP REVERT JUMPDEST PUSH1 PUSH1 SWAP SLOAD SWAP PUSH2 ARITHMETIC_OP SWAP ARITHMETIC_OP PUSH LOGIC_OP PUSH LOGIC_OP PUSH4 PUSH1 MLOAD DUP PUSH4 LOGIC_OP PUSH ARITHMETIC_OP DUP MSTORE PUSH1 ARITHMETIC_OP PUSH1 PUSH1 MLOAD DUP DUP ARITHMETIC_OP DUP PUSH1 DUP DUP EXTCODESIZE ISZERO DUP ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST POP GAS CALL ISZERO DUP ISZERO PUSH2 JUMPI RETURNDATASIZE PUSH1 DUP RETURNDATACOPY RETURNDATASIZE PUSH1 REVERT JUMPDEST POP POP POP POP PUSH1 MLOAD RETURNDATASIZE PUSH1 DUP COMPARASION ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST DUP ARITHMETIC_OP SWAP DUP DUP MLOAD SWAP PUSH1 ARITHMETIC_OP SWAP SWAP SWAP SWAP POP POP POP SWAP POP SWAP JUMP JUMPDEST PUSH1 PUSH1 PUSH1 SWAP SLOAD SWAP PUSH2 ARITHMETIC_OP SWAP ARITHMETIC_OP PUSH LOGIC_OP SWAP POP SWAP JUMP JUMPDEST PUSH1 DUP SWAP SLOAD SWAP PUSH2 ARITHMETIC_OP SWAP ARITHMETIC_OP PUSH1 LOGIC_OP ISZERO ISZERO ISZERO PUSH2 JUMPI PUSH1 MLOAD PUSH DUP MSTORE PUSH1 ARITHMETIC_OP DUP DUP PUSH1 ARITHMETIC_OP DUP DUP ARITHMETIC_OP DUP MSTORE PUSH1 DUP MSTORE PUSH1 ARITHMETIC_OP DUP PUSH DUP MSTORE POP PUSH1 ARITHMETIC_OP SWAP POP POP PUSH1 MLOAD DUP SWAP ARITHMETIC_OP SWAP REVERT JUMPDEST PUSH1 PUSH1 DUP PUSH2 ARITHMETIC_OP DUP SLOAD DUP PUSH1 ARITHMETIC_OP LOGIC_OP LOGIC_OP SWAP DUP ISZERO ISZERO ARITHMETIC_OP LOGIC_OP SWAP SSTORE POP PUSH1 PUSH1 SWAP SLOAD SWAP PUSH2 ARITHMETIC_OP SWAP ARITHMETIC_OP PUSH LOGIC_OP PUSH LOGIC_OP PUSH4 CONSTANT2 PUSH1 MLOAD DUP PUSH4 LOGIC_OP PUSH ARITHMETIC_OP DUP MSTORE PUSH1 ARITHMETIC_OP DUP DUP PUSH LOGIC_OP PUSH LOGIC_OP DUP MSTORE PUSH1 ARITHMETIC_OP SWAP POP POP PUSH1 PUSH1 MLOAD DUP DUP ARITHMETIC_OP DUP PUSH1 DUP DUP EXTCODESIZE ISZERO DUP ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST POP GAS CALL ISZERO DUP ISZERO PUSH2 JUMPI RETURNDATASIZE PUSH1 DUP RETURNDATACOPY RETURNDATASIZE PUSH1 REVERT JUMPDEST POP POP POP POP PUSH1 MLOAD RETURNDATASIZE PUSH1 DUP COMPARASION ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST DUP ARITHMETIC_OP SWAP DUP DUP MLOAD SWAP PUSH1 ARITHMETIC_OP SWAP SWAP SWAP SWAP POP POP POP PUSH1 PUSH1 PUSH2 ARITHMETIC_OP DUP SLOAD DUP PUSH ARITHMETIC_OP LOGIC_OP LOGIC_OP SWAP DUP PUSH LOGIC_OP ARITHMETIC_OP LOGIC_OP SWAP SSTORE POP PUSH1 PUSH1 SWAP SLOAD SWAP PUSH2 ARITHMETIC_OP SWAP ARITHMETIC_OP PUSH LOGIC_OP PUSH LOGIC_OP PUSH4 CONSTANT2 PUSH1 PUSH1 ARITHMETIC_OP PUSH1 SWAP SLOAD SWAP PUSH2 ARITHMETIC_OP SWAP ARITHMETIC_OP PUSH LOGIC_OP PUSH1 DUP ARITHMETIC_OP SLOAD PUSH1 PUSH1 ARITHMETIC_OP PUSH1 PUSH1 ARITHMETIC_OP PUSH1 SWAP SLOAD SWAP PUSH2 ARITHMETIC_OP SWAP ARITHMETIC_OP PUSH1 LOGIC_OP PUSH1 MLOAD DUP PUSH4 LOGIC_OP PUSH ARITHMETIC_OP DUP MSTORE PUSH1 ARITHMETIC_OP DUP DUP PUSH LOGIC_OP PUSH LOGIC_OP DUP MSTORE PUSH1 ARITHMETIC_OP DUP PUSH LOGIC_OP PUSH LOGIC_OP DUP MSTORE PUSH1 ARITHMETIC_OP DUP DUP MSTORE PUSH1 ARITHMETIC_OP DUP PUSH1 ARITHMETIC_OP DUP ISZERO ISZERO ISZERO ISZERO DUP MSTORE PUSH1 ARITHMETIC_OP DUP DUP ARITHMETIC_OP DUP MSTORE DUP DUP DUP SLOAD DUP MSTORE PUSH1 ARITHMETIC_OP SWAP POP DUP SLOAD DUP ISZERO PUSH2 JUMPI PUSH1 ARITHMETIC_OP DUP ARITHMETIC_OP SWAP SWAP PUSH1 MSTORE PUSH1 PUSH1 SHA3 SWAP JUMPDEST DUP SLOAD DUP MSTORE PUSH1 ARITHMETIC_OP SWAP PUSH1 ARITHMETIC_OP SWAP DUP DUP COMPARASION PUSH2 JUMPI JUMPDEST POP POP SWAP POP POP POP POP POP POP POP PUSH1 PUSH1 MLOAD DUP DUP ARITHMETIC_OP DUP PUSH1 DUP DUP EXTCODESIZE ISZERO DUP ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST POP GAS CALL ISZERO DUP ISZERO PUSH2 JUMPI RETURNDATASIZE PUSH1 DUP RETURNDATACOPY RETURNDATASIZE PUSH1 REVERT JUMPDEST POP POP POP POP PUSH1 MLOAD RETURNDATASIZE PUSH1 DUP COMPARASION ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST DUP ARITHMETIC_OP SWAP DUP DUP MLOAD SWAP PUSH1 ARITHMETIC_OP SWAP SWAP SWAP SWAP POP POP POP PUSH1 PUSH1 PUSH2 ARITHMETIC_OP DUP SLOAD DUP PUSH ARITHMETIC_OP LOGIC_OP LOGIC_OP SWAP DUP PUSH LOGIC_OP ARITHMETIC_OP LOGIC_OP SWAP SSTORE POP PUSH PUSH1 MLOAD PUSH1 MLOAD DUP SWAP ARITHMETIC_OP SWAP LOG JUMP JUMPDEST PUSH1 PUSH2 PUSH2 JUMP JUMPDEST ISZERO ISZERO PUSH2 JUMPI PUSH1 MLOAD PUSH DUP MSTORE PUSH1 ARITHMETIC_OP DUP DUP PUSH1 ARITHMETIC_OP DUP DUP ARITHMETIC_OP DUP MSTORE PUSH1 DUP MSTORE PUSH1 ARITHMETIC_OP DUP PUSH DUP MSTORE POP PUSH1 ARITHMETIC_OP SWAP POP POP PUSH1 MLOAD DUP SWAP ARITHMETIC_OP SWAP REVERT JUMPDEST PUSH1 PUSH1 SWAP SLOAD SWAP PUSH2 ARITHMETIC_OP SWAP ARITHMETIC_OP PUSH LOGIC_OP PUSH LOGIC_OP PUSH4 PUSH1 MLOAD DUP PUSH4 LOGIC_OP PUSH ARITHMETIC_OP DUP MSTORE PUSH1 ARITHMETIC_OP PUSH1 PUSH1 MLOAD DUP DUP ARITHMETIC_OP DUP PUSH1 DUP DUP EXTCODESIZE ISZERO DUP ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST POP GAS CALL ISZERO DUP ISZERO PUSH2 JUMPI RETURNDATASIZE PUSH1 DUP RETURNDATACOPY RETURNDATASIZE PUSH1 REVERT JUMPDEST POP POP POP POP PUSH1 MLOAD RETURNDATASIZE PUSH1 DUP COMPARASION ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST DUP ARITHMETIC_OP SWAP DUP DUP MLOAD SWAP PUSH1 ARITHMETIC_OP SWAP SWAP SWAP SWAP POP POP POP SWAP POP SWAP JUMP JUMPDEST PUSH1 DUP SWAP SLOAD SWAP PUSH2 ARITHMETIC_OP SWAP ARITHMETIC_OP PUSH1 LOGIC_OP DUP JUMP JUMPDEST PUSH1 DUP PUSH1 DUP PUSH2 PUSH2 JUMP JUMPDEST ISZERO ISZERO PUSH2 JUMPI PUSH1 MLOAD PUSH DUP MSTORE PUSH1 ARITHMETIC_OP DUP DUP PUSH1 ARITHMETIC_OP DUP DUP ARITHMETIC_OP DUP MSTORE PUSH1 DUP MSTORE PUSH1 ARITHMETIC_OP DUP PUSH DUP MSTORE POP PUSH1 ARITHMETIC_OP SWAP POP POP PUSH1 MLOAD DUP SWAP ARITHMETIC_OP SWAP REVERT JUMPDEST PUSH2 PUSH2 JUMP JUMPDEST ISZERO ISZERO ISZERO PUSH2 JUMPI PUSH1 MLOAD PUSH DUP MSTORE PUSH1 ARITHMETIC_OP DUP DUP PUSH1 ARITHMETIC_OP DUP DUP ARITHMETIC_OP DUP MSTORE PUSH1 DUP MSTORE PUSH1 ARITHMETIC_OP DUP PUSH DUP MSTORE PUSH1 ARITHMETIC_OP PUSH DUP MSTORE POP PUSH1 ARITHMETIC_OP SWAP POP POP PUSH1 MLOAD DUP SWAP ARITHMETIC_OP SWAP REVERT JUMPDEST PUSH1 PUSH1 SWAP SLOAD SWAP PUSH2 ARITHMETIC_OP SWAP ARITHMETIC_OP PUSH LOGIC_OP PUSH LOGIC_OP PUSH4 DUP DUP PUSH1 MLOAD DUP PUSH4 LOGIC_OP PUSH ARITHMETIC_OP DUP MSTORE PUSH1 ARITHMETIC_OP DUP DUP PUSH LOGIC_OP PUSH LOGIC_OP DUP MSTORE PUSH1 ARITHMET"]

In [ ]:
Get_vuln(opcode_clean)

(1, 35)
1/1 [==============================] - 0s 36ms/step
[[0.41557682 0.99621356 0.04458271 0.08147041 0.28655946]] clean


In [ ]:
opcode_TOD =["PUSH1 PUSH1 MSTORE PUSH1 CALLDATASIZE COMPARASION PUSH2 JUMPI PUSH1 CALLDATALOAD PUSH SWAP ARITHMETIC_OP DUP PUSH3 EQ PUSH2 JUMPI DUP PUSH4 EQ PUSH2 JUMPI DUP PUSH4 EQ PUSH2 JUMPI DUP PUSH4 EQ PUSH2 JUMPI DUP PUSH4 EQ PUSH2 JUMPI DUP PUSH4 EQ PUSH2 JUMPI DUP PUSH4 EQ PUSH2 JUMPI DUP PUSH4 EQ PUSH2 JUMPI DUP PUSH4 EQ PUSH2 JUMPI DUP PUSH4 EQ PUSH2 JUMPI DUP PUSH4 EQ PUSH2 JUMPI DUP PUSH4 EQ PUSH2 JUMPI DUP PUSH4 EQ PUSH2 JUMPI DUP PUSH4 EQ PUSH2 JUMPI DUP PUSH4 EQ PUSH2 JUMPI DUP PUSH4 EQ PUSH2 JUMPI DUP PUSH4 EQ PUSH2 JUMPI DUP PUSH4 EQ PUSH2 JUMPI DUP PUSH4 EQ PUSH2 JUMPI DUP PUSH4 EQ PUSH2 JUMPI DUP PUSH4 EQ PUSH2 JUMPI DUP PUSH4 EQ PUSH2 JUMPI DUP PUSH4 EQ PUSH2 JUMPI DUP PUSH4 EQ PUSH2 JUMPI DUP PUSH4 EQ PUSH2 JUMPI DUP PUSH4 EQ PUSH2 JUMPI DUP PUSH4 EQ PUSH2 JUMPI DUP PUSH4 EQ PUSH2 JUMPI DUP PUSH4 EQ PUSH2 JUMPI DUP PUSH4 EQ PUSH2 JUMPI DUP PUSH4 EQ PUSH2 JUMPI DUP PUSH4 EQ PUSH2 JUMPI DUP PUSH4 EQ PUSH2 JUMPI DUP PUSH4 EQ PUSH2 JUMPI DUP PUSH4 EQ PUSH2 JUMPI DUP PUSH4 EQ PUSH2 JUMPI DUP PUSH4 EQ PUSH2 JUMPI DUP PUSH4 EQ PUSH2 JUMPI DUP PUSH4 EQ PUSH2 JUMPI DUP PUSH4 EQ PUSH2 JUMPI DUP PUSH4 EQ PUSH2 JUMPI DUP PUSH4 EQ PUSH2 JUMPI DUP PUSH4 EQ PUSH2 JUMPI DUP PUSH4 EQ PUSH2 JUMPI DUP PUSH4 EQ PUSH2 JUMPI DUP PUSH4 EQ PUSH2 JUMPI DUP PUSH4 EQ PUSH2 JUMPI DUP PUSH4 EQ PUSH2 JUMPI DUP PUSH4 EQ PUSH2 JUMPI DUP PUSH4 EQ PUSH2 JUMPI DUP PUSH4 EQ PUSH2 JUMPI DUP PUSH4 EQ PUSH2 JUMPI DUP PUSH4 EQ PUSH2 JUMPI DUP PUSH4 EQ PUSH2 JUMPI DUP PUSH4 EQ PUSH2 JUMPI DUP PUSH4 EQ PUSH2 JUMPI DUP PUSH4 EQ PUSH2 JUMPI DUP PUSH4 EQ PUSH2 JUMPI DUP PUSH4 EQ PUSH2 JUMPI DUP PUSH4 EQ PUSH2 JUMPI DUP PUSH4 EQ PUSH2 JUMPI DUP PUSH4 EQ PUSH2 JUMPI DUP PUSH4 EQ PUSH2 JUMPI DUP PUSH4 EQ PUSH2 JUMPI DUP PUSH4 EQ PUSH2 JUMPI DUP PUSH4 EQ PUSH2 JUMPI DUP PUSH4 EQ PUSH2 JUMPI DUP PUSH4 EQ PUSH2 JUMPI DUP PUSH4 EQ PUSH2 JUMPI DUP PUSH4 EQ PUSH2 JUMPI DUP PUSH4 EQ PUSH2 JUMPI DUP PUSH4 EQ PUSH2 JUMPI DUP PUSH4 EQ PUSH2 JUMPI DUP PUSH4 EQ PUSH2 JUMPI DUP PUSH4 EQ PUSH2 JUMPI JUMPDEST PUSH CONSTANT2 CALLVALUE PUSH1 MLOAD DUP DUP PUSH LOGIC_OP PUSH LOGIC_OP DUP MSTORE PUSH1 ARITHMETIC_OP DUP DUP MSTORE PUSH1 ARITHMETIC_OP SWAP POP POP POP PUSH1 MLOAD DUP SWAP ARITHMETIC_OP SWAP LOG STOP JUMPDEST CALLVALUE DUP ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST POP PUSH2 PUSH2 JUMP JUMPDEST PUSH1 MLOAD DUP DUP DUP MSTORE PUSH1 ARITHMETIC_OP SWAP POP POP PUSH1 MLOAD DUP SWAP ARITHMETIC_OP SWAP RETURN JUMPDEST CALLVALUE DUP ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST POP PUSH2 PUSH1 DUP CALLDATASIZE ARITHMETIC_OP PUSH1 DUP COMPARASION ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST DUP ARITHMETIC_OP SWAP DUP DUP CALLDATALOAD SWAP PUSH1 ARITHMETIC_OP SWAP SWAP SWAP SWAP POP POP POP PUSH2 JUMP JUMPDEST PUSH1 MLOAD DUP DUP PUSH LOGIC_OP PUSH LOGIC_OP DUP MSTORE PUSH1 ARITHMETIC_OP DUP DUP MSTORE PUSH1 ARITHMETIC_OP DUP PUSH1 ARITHMETIC_OP DUP ISZERO ISZERO ISZERO ISZERO DUP MSTORE PUSH1 ARITHMETIC_OP DUP DUP MSTORE PUSH1 ARITHMETIC_OP DUP ISZERO ISZERO ISZERO ISZERO DUP MSTORE PUSH1 ARITHMETIC_OP DUP ISZERO ISZERO ISZERO ISZERO DUP MSTORE PUSH1 ARITHMETIC_OP DUP DUP MSTORE PUSH1 ARITHMETIC_OP DUP DUP MSTORE PUSH1 ARITHMETIC_OP DUP DUP ARITHMETIC_OP DUP MSTORE DUP DUP DUP MLOAD DUP MSTORE PUSH1 ARITHMETIC_OP SWAP POP DUP MLOAD SWAP PUSH1 ARITHMETIC_OP SWAP DUP DUP DUP PUSH1 JUMPDEST DUP DUP COMPARASION ISZERO PUSH2 JUMPI DUP DUP ARITHMETIC_OP MLOAD DUP DUP ARITHMETIC_OP MSTORE PUSH1 DUP ARITHMETIC_OP SWAP POP PUSH2 JUMP JUMPDEST POP POP POP POP SWAP POP SWAP DUP ARITHMETIC_OP SWAP PUSH1 LOGIC_OP DUP ISZERO PUSH2 JUMPI DUP DUP ARITHMETIC_OP DUP MLOAD PUSH1 DUP PUSH1 ARITHMETIC_OP PUSH2 ARITHMETIC_OP ARITHMETIC_OP LOGIC_OP LOGIC_OP DUP MSTORE PUSH1 ARITHMETIC_OP SWAP POP JUMPDEST POP SWAP POP POP POP POP POP POP POP POP POP POP POP PUSH1 MLOAD DUP SWAP ARITHMETIC_OP SWAP RETURN JUMPDEST CALLVALUE DUP ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST POP PUSH2 PUSH1 DUP CALLDATASIZE ARITHMETIC_OP PUSH1 DUP COMPARASION ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST DUP ARITHMETIC_OP SWAP DUP DUP CALLDATALOAD SWAP PUSH1 ARITHMETIC_OP SWAP SWAP SWAP SWAP POP POP POP PUSH2 JUMP JUMPDEST STOP JUMPDEST CALLVALUE DUP ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST POP PUSH2 PUSH1 DUP CALLDATASIZE ARITHMETIC_OP PUSH1 DUP COMPARASION ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST DUP ARITHMETIC_OP SWAP DUP DUP CALLDATALOAD SWAP PUSH1 ARITHMETIC_OP SWAP SWAP SWAP SWAP POP POP POP PUSH2 JUMP JUMPDEST STOP JUMPDEST PUSH2 PUSH2 JUMP JUMPDEST STOP JUMPDEST CALLVALUE DUP ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST POP PUSH2 PUSH1 DUP CALLDATASIZE ARITHMETIC_OP PUSH1 DUP COMPARASION ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST DUP ARITHMETIC_OP SWAP DUP DUP CALLDATALOAD SWAP PUSH1 ARITHMETIC_OP SWAP SWAP SWAP SWAP POP POP POP PUSH2 JUMP JUMPDEST STOP JUMPDEST CALLVALUE DUP ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST POP PUSH2 PUSH1 DUP CALLDATASIZE ARITHMETIC_OP PUSH1 DUP COMPARASION ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST DUP ARITHMETIC_OP SWAP DUP DUP CALLDATALOAD SWAP PUSH1 ARITHMETIC_OP SWAP SWAP SWAP SWAP POP POP POP PUSH2 JUMP JUMPDEST STOP JUMPDEST PUSH2 PUSH2 JUMP JUMPDEST STOP JUMPDEST CALLVALUE DUP ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST POP PUSH2 PUSH2 JUMP JUMPDEST PUSH1 MLOAD DUP DUP ISZERO ISZERO ISZERO ISZERO DUP MSTORE PUSH1 ARITHMETIC_OP SWAP POP POP PUSH1 MLOAD DUP SWAP ARITHMETIC_OP SWAP RETURN JUMPDEST CALLVALUE DUP ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST POP PUSH2 PUSH1 DUP CALLDATASIZE ARITHMETIC_OP PUSH1 DUP COMPARASION ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST DUP ARITHMETIC_OP SWAP DUP DUP CALLDATALOAD SWAP PUSH1 ARITHMETIC_OP SWAP SWAP SWAP SWAP POP POP POP PUSH2 JUMP JUMPDEST STOP JUMPDEST CALLVALUE DUP ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST POP PUSH2 PUSH1 DUP CALLDATASIZE ARITHMETIC_OP PUSH1 DUP COMPARASION ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST DUP ARITHMETIC_OP SWAP DUP DUP CALLDATALOAD SWAP PUSH1 ARITHMETIC_OP SWAP SWAP SWAP SWAP POP POP POP PUSH2 JUMP JUMPDEST STOP JUMPDEST CALLVALUE DUP ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST POP PUSH2 PUSH1 DUP CALLDATASIZE ARITHMETIC_OP PUSH1 DUP COMPARASION ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST DUP ARITHMETIC_OP SWAP DUP DUP CALLDATALOAD SWAP PUSH1 ARITHMETIC_OP SWAP SWAP SWAP SWAP POP POP POP PUSH2 JUMP JUMPDEST STOP JUMPDEST CALLVALUE DUP ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST POP PUSH2 PUSH1 DUP CALLDATASIZE ARITHMETIC_OP PUSH1 DUP COMPARASION ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST DUP ARITHMETIC_OP SWAP DUP DUP CALLDATALOAD SWAP PUSH1 ARITHMETIC_OP SWAP SWAP SWAP SWAP POP POP POP PUSH2 JUMP JUMPDEST STOP JUMPDEST PUSH2 PUSH2 JUMP JUMPDEST STOP JUMPDEST PUSH2 PUSH2 JUMP JUMPDEST STOP JUMPDEST CALLVALUE DUP ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST POP PUSH2 PUSH1 DUP CALLDATASIZE ARITHMETIC_OP PUSH1 DUP COMPARASION ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST DUP ARITHMETIC_OP SWAP DUP DUP CALLDATALOAD SWAP PUSH1 ARITHMETIC_OP SWAP SWAP SWAP SWAP DUP CALLDATALOAD SWAP PUSH1 ARITHMETIC_OP SWAP PUSH DUP COMPARASION ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST DUP ARITHMETIC_OP DUP PUSH1 DUP ARITHMETIC_OP COMPARASION ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST DUP CALLDATALOAD SWAP PUSH1 ARITHMETIC_OP SWAP DUP PUSH1 DUP ARITHMETIC_OP DUP ARITHMETIC_OP COMPARASION PUSH DUP COMPARASION LOGIC_OP ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST SWAP SWAP DUP DUP PUSH1 ARITHMETIC_OP PUSH1 DUP SWAP ARITHMETIC_OP ARITHMETIC_OP PUSH1 ARITHMETIC_OP PUSH1 MLOAD SWAP DUP ARITHMETIC_OP PUSH1 MSTORE DUP SWAP SWAP SWAP SWAP DUP DUP MSTORE PUSH1 ARITHMETIC_OP DUP DUP DUP DUP DUP CALLDATACOPY PUSH1 DUP DUP ARITHMETIC_OP MSTORE PUSH1 LOGIC_OP PUSH1 DUP ARITHMETIC_OP LOGIC_OP SWAP POP DUP DUP ARITHMETIC_OP SWAP POP POP POP POP POP POP POP SWAP SWAP SWAP SWAP SWAP POP POP POP PUSH2 JUMP JUMPDEST STOP JUMPDEST PUSH2 PUSH2 JUMP JUMPDEST STOP JUMPDEST PUSH2 PUSH2 JUMP JUMPDEST STOP JUMPDEST CALLVALUE DUP ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST POP PUSH2 PUSH1 DUP CALLDATASIZE ARITHMETIC_OP PUSH1 DUP COMPARASION ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST DUP ARITHMETIC_OP SWAP DUP DUP CALLDATALOAD SWAP PUSH1 ARITHMETIC_OP SWAP SWAP SWAP SWAP POP POP POP PUSH2 JUMP JUMPDEST STOP JUMPDEST CALLVALUE DUP ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST POP PUSH2 PUSH1 DUP CALLDATASIZE ARITHMETIC_OP PUSH1 DUP COMPARASION ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST DUP ARITHMETIC_OP SWAP DUP DUP CALLDATALOAD SWAP PUSH1 ARITHMETIC_OP SWAP SWAP SWAP SWAP POP POP POP PUSH2 JUMP JUMPDEST STOP JUMPDEST PUSH2 PUSH2 JUMP JUMPDEST STOP JUMPDEST CALLVALUE DUP ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST POP PUSH2 PUSH2 JUMP JUMPDEST PUSH1 MLOAD DUP DUP DUP MSTORE PUSH1 ARITHMETIC_OP SWAP POP POP PUSH1 MLOAD DUP SWAP ARITHMETIC_OP SWAP RETURN JUMPDEST CALLVALUE DUP ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST POP PUSH2 PUSH1 DUP CALLDATASIZE ARITHMETIC_OP PUSH1 DUP COMPARASION ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST DUP ARITHMETIC_OP SWAP DUP DUP CALLDATALOAD SWAP PUSH1 ARITHMETIC_OP SWAP SWAP SWAP SWAP POP POP POP PUSH2 JUMP JUMPDEST STOP JUMPDEST CALLVALUE DUP ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST POP PUSH2 PUSH1 DUP CALLDATASIZE ARITHMETIC_OP PUSH1 DUP COMPARASION ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST DUP ARITHMETIC_OP SWAP DUP DUP CALLDATALOAD SWAP PUSH1 ARITHMETIC_OP SWAP SWAP SWAP SWAP POP POP POP PUSH2 JUMP JUMPDEST STOP JUMPDEST PUSH2 PUSH2 JUMP JUMPDEST STOP JUMPDEST PUSH2 PUSH2 JUMP JUMPDEST STOP JUMPDEST PUSH2 PUSH2 JUMP JUMPDEST STOP JUMPDEST CALLVALUE DUP ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST POP PUSH2 PUSH1 DUP CALLDATASIZE ARITHMETIC_OP PUSH1 DUP COMPARASION ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST DUP ARITHMETIC_OP SWAP DUP DUP CALLDATALOAD SWAP PUSH1 ARITHMETIC_OP SWAP SWAP SWAP SWAP POP POP POP PUSH2 JUMP JUMPDEST STOP JUMPDEST PUSH2 PUSH2 JUMP JUMPDEST STOP JUMPDEST CALLVALUE DUP ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST POP PUSH2 PUSH1 DUP CALLDATASIZE ARITHMETIC_OP PUSH1 DUP COMPARASION ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST DUP ARITHMETIC_OP SWAP DUP DUP CALLDATALOAD SWAP PUSH1 ARITHMETIC_OP SWAP SWAP SWAP SWAP POP POP POP PUSH2 JUMP JUMPDEST STOP JUMPDEST CALLVALUE DUP ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST POP PUSH2 PUSH1 DUP CALLDATASIZE ARITHMETIC_OP PUSH1 DUP COMPARASION ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST DUP ARITHMETIC_OP SWAP DUP DUP CALLDATALOAD PUSH LOGIC_OP SWAP PUSH1 ARITHMETIC_OP SWAP SWAP SWAP SWAP DUP CALLDATALOAD PUSH LOGIC_OP SWAP PUSH1 ARITHMETIC_OP SWAP SWAP SWAP SWAP DUP CALLDATALOAD SWAP PUSH1 ARITHMETIC_OP SWAP SWAP SWAP SWAP DUP CALLDATALOAD SWAP PUSH1 ARITHMETIC_OP SWAP SWAP SWAP SWAP DUP CALLDATALOAD SWAP PUSH1 ARITHMETIC_OP SWAP SWAP SWAP SWAP POP POP POP PUSH2 JUMP JUMPDEST STOP JUMPDEST PUSH2 PUSH2 JUMP JUMPDEST STOP JUMPDEST PUSH2 PUSH2 JUMP JUMPDEST STOP JUMPDEST CALLVALUE DUP ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST POP PUSH2 PUSH2 JUMP JUMPDEST PUSH1 MLOAD DUP DUP DUP MSTORE PUSH1 ARITHMETIC_OP SWAP POP POP PUSH1 MLOAD DUP SWAP ARITHMETIC_OP SWAP RETURN JUMPDEST CALLVALUE DUP ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST POP PUSH2 PUSH1 DUP CALLDATASIZE ARITHMETIC_OP PUSH1 DUP COMPARASION ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST DUP ARITHMETIC_OP SWAP DUP DUP CALLDATALOAD SWAP PUSH1 ARITHMETIC_OP SWAP SWAP SWAP SWAP POP POP POP PUSH2 JUMP JUMPDEST STOP JUMPDEST CALLVALUE DUP ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST POP PUSH2 PUSH1 DUP CALLDATASIZE ARITHMETIC_OP PUSH1 DUP COMPARASION ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST DUP ARITHMETIC_OP SWAP DUP DUP CALLDATALOAD SWAP PUSH1 ARITHMETIC_OP SWAP SWAP SWAP SWAP POP POP POP PUSH2 JUMP JUMPDEST STOP JUMPDEST PUSH2 PUSH2 JUMP JUMPDEST STOP JUMPDEST CALLVALUE DUP ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST POP PUSH2 PUSH1 DUP CALLDATASIZE ARITHMETIC_OP PUSH1 DUP COMPARASION ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST DUP ARITHMETIC_OP SWAP DUP DUP CALLDATALOAD SWAP PUSH1 ARITHMETIC_OP SWAP SWAP SWAP SWAP POP POP POP PUSH2 JUMP JUMPDEST STOP JUMPDEST CALLVALUE DUP ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST POP PUSH2 PUSH2 JUMP JUMPDEST PUSH1 MLOAD DUP DUP DUP MSTORE PUSH1 ARITHMETIC_OP SWAP POP POP PUSH1 MLOAD DUP SWAP ARITHMETIC_OP SWAP RETURN JUMPDEST PUSH2 PUSH2 JUMP JUMPDEST STOP JUMPDEST PUSH2 PUSH2 JUMP JUMPDEST STOP JUMPDEST PUSH2 PUSH2 JUMP JUMPDEST STOP JUMPDEST PUSH2 PUSH2 JUMP JUMPDEST STOP JUMPDEST CALLVALUE DUP ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST POP PUSH2 PUSH2 JUMP JUMPDEST PUSH1 MLOAD DUP DUP PUSH LOGIC_OP PUSH LOGIC_OP DUP MSTORE PUSH1 ARITHMETIC_OP SWAP POP POP PUSH1 MLOAD DUP SWAP ARITHMETIC_OP SWAP RETURN JUMPDEST CALLVALUE DUP ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST POP PUSH2 PUSH1 DUP CALLDATASIZE ARITHMETIC_OP PUSH1 DUP COMPARASION ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST DUP ARITHMETIC_OP SWAP DUP DUP CALLDATALOAD PUSH LOGIC_OP SWAP PUSH1 ARITHMETIC_OP SWAP SWAP SWAP SWAP DUP CALLDATALOAD SWAP PUSH1 ARITHMETIC_OP SWAP SWAP SWAP SWAP DUP CALLDATALOAD PUSH LOGIC_OP SWAP PUSH1 ARITHMETIC_OP SWAP SWAP SWAP SWAP DUP CALLDATALOAD SWAP PUSH1 ARITHMETIC_OP SWAP PUSH DUP COMPARASION ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST DUP ARITHMETIC_OP DUP PUSH1 DUP ARITHMETIC_OP COMPARASION ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST DUP CALLDATALOAD SWAP PUSH1 ARITHMETIC_OP SWAP DUP PUSH1 DUP ARITHMETIC_OP DUP ARITHMETIC_OP COMPARASION PUSH DUP COMPARASION LOGIC_OP ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST SWAP SWAP DUP DUP PUSH1 ARITHMETIC_OP PUSH1 DUP SWAP ARITHMETIC_OP ARITHMETIC_OP PUSH1 ARITHMETIC_OP PUSH1 MLOAD SWAP DUP ARITHMETIC_OP PUSH1 MSTORE DUP SWAP SWAP SWAP SWAP DUP DUP MSTORE PUSH1 ARITHMETIC_OP DUP DUP DUP DUP DUP CALLDATACOPY PUSH1 DUP DUP ARITHMETIC_OP MSTORE PUSH1 LOGIC_OP PUSH1 DUP ARITHMETIC_OP LOGIC_OP SWAP POP DUP DUP ARITHMETIC_OP SWAP POP POP POP POP POP POP POP SWAP SWAP SWAP SWAP SWAP POP POP POP PUSH2 JUMP JUMPDEST STOP JUMPDEST CALLVALUE DUP ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST POP PUSH2 PUSH1 DUP CALLDATASIZE ARITHMETIC_OP PUSH1 DUP COMPARASION ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST DUP ARITHMETIC_OP SWAP DUP DUP CALLDATALOAD SWAP PUSH1 ARITHMETIC_OP SWAP SWAP SWAP SWAP POP POP POP PUSH2 JUMP JUMPDEST STOP JUMPDEST CALLVALUE DUP ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST POP PUSH2 PUSH2 JUMP JUMPDEST PUSH1 MLOAD DUP DUP PUSH LOGIC_OP PUSH LOGIC_OP DUP MSTORE PUSH1 ARITHMETIC_OP SWAP POP POP PUSH1 MLOAD DUP SWAP ARITHMETIC_OP SWAP RETURN JUMPDEST PUSH2 PUSH2 JUMP JUMPDEST STOP JUMPDEST PUSH2 PUSH2 JUMP JUMPDEST STOP JUMPDEST PUSH2 PUSH2 JUMP JUMPDEST STOP JUMPDEST CALLVALUE DUP ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST POP PUSH2 PUSH1 DUP CALLDATASIZE ARITHMETIC_OP PUSH1 DUP COMPARASION ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST DUP ARITHMETIC_OP SWAP DUP DUP CALLDATALOAD SWAP PUSH1 ARITHMETIC_OP SWAP SWAP SWAP SWAP POP POP POP PUSH2 JUMP JUMPDEST STOP JUMPDEST CALLVALUE DUP ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST POP PUSH2 PUSH1 DUP CALLDATASIZE ARITHMETIC_OP PUSH1 DUP COMPARASION ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST DUP ARITHMETIC_OP SWAP DUP DUP CALLDATALOAD SWAP PUSH1 ARITHMETIC_OP SWAP SWAP SWAP SWAP POP POP POP PUSH2 JUMP JUMPDEST STOP JUMPDEST CALLVALUE DUP ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST POP PUSH2 PUSH1 DUP CALLDATASIZE ARITHMETIC_OP PUSH1 DUP COMPARASION ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST DUP ARITHMETIC_OP SWAP DUP DUP CALLDATALOAD SWAP PUSH1 ARITHMETIC_OP SWAP SWAP SWAP SWAP POP POP POP PUSH2 JUMP JUMPDEST STOP JUMPDEST CALLVALUE DUP ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST POP PUSH2 PUSH1 DUP CALLDATASIZE ARITHMETIC_OP PUSH1 DUP COMPARASION ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST DUP ARITHMETIC_OP SWAP DUP DUP CALLDATALOAD PUSH LOGIC_OP SWAP PUSH1 ARITHMETIC_OP SWAP SWAP SWAP SWAP DUP CALLDATALOAD SWAP PUSH1 ARITHMETIC_OP SWAP SWAP SWAP SWAP DUP CALLDATALOAD SWAP PUSH1 ARITHMETIC_OP SWAP PUSH DUP COMPARASION ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST DUP ARITHMETIC_OP DUP PUSH1 DUP ARITHMETIC_OP COMPARASION ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST DUP CALLDATALOAD SWAP PUSH1 ARITHMETIC_OP SWAP DUP PUSH1 DUP ARITHMETIC_OP DUP ARITHMETIC_OP COMPARASION PUSH DUP COMPARASION LOGIC_OP ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST SWAP SWAP DUP DUP PUSH1 ARITHMETIC_OP PUSH1 DUP SWAP ARITHMETIC_OP ARITHMETIC_OP PUSH1 ARITHMETIC_OP PUSH1 MLOAD SWAP DUP ARITHMETIC_OP PUSH1 MSTORE DUP SWAP SWAP SWAP SWAP DUP DUP MSTORE PUSH1 ARITHMETIC_OP DUP DUP DUP DUP DUP CALLDATACOPY PUSH1 DUP DUP ARITHMETIC_OP MSTORE PUSH1 LOGIC_OP PUSH1 DUP ARITHMETIC_OP LOGIC_OP SWAP POP DUP DUP ARITHMETIC_OP SWAP POP POP POP POP POP POP POP SWAP SWAP SWAP SWAP SWAP DUP CALLDATALOAD SWAP PUSH1 ARITHMETIC_OP SWAP PUSH DUP COMPARASION ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST DUP ARITHMETIC_OP DUP PUSH1 DUP ARITHMETIC_OP COMPARASION ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST DUP CALLDATALOAD SWAP PUSH1 ARITHMETIC_OP SWAP DUP PUSH1 DUP ARITHMETIC_OP DUP ARITHMETIC_OP COMPARASION PUSH DUP COMPARASION LOGIC_OP ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST SWAP SWAP DUP DUP PUSH1 ARITHMETIC_OP PUSH1 DUP SWAP ARITHMETIC_OP ARITHMETIC_OP PUSH1 ARITHMETIC_OP PUSH1 MLOAD SWAP DUP ARITHMETIC_OP PUSH1 MSTORE DUP SWAP SWAP SWAP SWAP DUP DUP MSTORE PUSH1 ARITHMETIC_OP DUP DUP DUP DUP DUP CALLDATACOPY PUSH1 DUP DUP ARITHMETIC_OP MSTORE PUSH1 LOGIC_OP PUSH1 DUP ARITHMETIC_OP LOGIC_OP SWAP POP DUP DUP ARITHMETIC_OP SWAP POP POP POP POP POP POP POP SWAP SWAP SWAP SWAP SWAP POP POP POP PUSH2 JUMP JUMPDEST PUSH1 MLOAD DUP DUP DUP MSTORE PUSH1 ARITHMETIC_OP SWAP POP POP PUSH1 MLOAD DUP SWAP ARITHMETIC_OP SWAP RETURN JUMPDEST PUSH2 PUSH2 JUMP JUMPDEST STOP JUMPDEST PUSH2 PUSH2 JUMP JUMPDEST STOP JUMPDEST PUSH2 PUSH2 JUMP JUMPDEST STOP JUMPDEST CALLVALUE DUP ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST POP PUSH2 PUSH1 DUP CALLDATASIZE ARITHMETIC_OP PUSH1 DUP COMPARASION ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST DUP ARITHMETIC_OP SWAP DUP DUP CALLDATALOAD SWAP PUSH1 ARITHMETIC_OP SWAP SWAP SWAP SWAP POP POP POP PUSH2 JUMP JUMPDEST STOP JUMPDEST CALLVALUE DUP ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST POP PUSH2 PUSH1 DUP CALLDATASIZE ARITHMETIC_OP PUSH1 DUP COMPARASION ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST DUP ARITHMETIC_OP SWAP DUP DUP CALLDATALOAD SWAP PUSH1 ARITHMETIC_OP SWAP SWAP SWAP SWAP POP POP POP PUSH2 JUMP JUMPDEST STOP JUMPDEST PUSH2 PUSH2 JUMP JUMPDEST STOP JUMPDEST CALLVALUE DUP ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST POP PUSH2 PUSH1 DUP CALLDATASIZE ARITHMETIC_OP PUSH1 DUP COMPARASION ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST DUP ARITHMETIC_OP SWAP DUP DUP CALLDATALOAD SWAP PUSH1 ARITHMETIC_OP SWAP SWAP SWAP SWAP POP POP POP PUSH2 JUMP JUMPDEST STOP JUMPDEST CALLVALUE DUP ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST POP PUSH2 PUSH1 DUP CALLDATASIZE ARITHMETIC_OP PUSH1 DUP COMPARASION ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST DUP ARITHMETIC_OP SWAP DUP DUP CALLDATALOAD SWAP PUSH1 ARITHMETIC_OP SWAP SWAP SWAP SWAP DUP CALLDATALOAD ISZERO ISZERO SWAP PUSH1 ARITHMETIC_OP SWAP SWAP SWAP SWAP POP POP POP PUSH2 JUMP JUMPDEST PUSH1 MLOAD DUP DUP DUP MSTORE PUSH1 ARITHMETIC_OP SWAP POP POP PUSH1 MLOAD DUP SWAP ARITHMETIC_OP SWAP RETURN JUMPDEST CALLVALUE DUP ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST POP PUSH2 PUSH1 DUP CALLDATASIZE ARITHMETIC_OP PUSH1 DUP COMPARASION ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST DUP ARITHMETIC_OP SWAP DUP DUP CALLDATALOAD SWAP PUSH1 ARITHMETIC_OP SWAP SWAP SWAP SWAP POP POP POP PUSH2 JUMP JUMPDEST STOP JUMPDEST PUSH2 PUSH2 JUMP JUMPDEST STOP JUMPDEST PUSH2 PUSH2 JUMP JUMPDEST STOP JUMPDEST CALLVALUE DUP ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST POP PUSH2 PUSH1 DUP CALLDATASIZE ARITHMETIC_OP PUSH1 DUP COMPARASION ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST DUP ARITHMETIC_OP SWAP DUP DUP CALLDATALOAD SWAP PUSH1 ARITHMETIC_OP SWAP SWAP SWAP SWAP POP POP POP PUSH2 JUMP JUMPDEST STOP JUMPDEST CALLVALUE DUP ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST POP PUSH2 PUSH1 DUP CALLDATASIZE ARITHMETIC_OP PUSH1 DUP COMPARASION ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST DUP ARITHMETIC_OP SWAP DUP DUP CALLDATALOAD SWAP PUSH1 ARITHMETIC_OP SWAP SWAP SWAP SWAP POP POP POP PUSH2 JUMP JUMPDEST STOP JUMPDEST PUSH2 PUSH2 JUMP JUMPDEST STOP JUMPDEST CALLVALUE DUP ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST POP PUSH2 PUSH1 DUP CALLDATASIZE ARITHMETIC_OP PUSH1 DUP COMPARASION ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST DUP ARITHMETIC_OP SWAP DUP DUP CALLDATALOAD SWAP PUSH1 ARITHMETIC_OP SWAP SWAP SWAP SWAP DUP CALLDATALOAD ISZERO ISZERO SWAP PUSH1 ARITHMETIC_OP SWAP SWAP SWAP SWAP POP POP POP PUSH2 JUMP JUMPDEST PUSH1 MLOAD DUP DUP DUP MSTORE PUSH1 ARITHMETIC_OP SWAP POP POP PUSH1 MLOAD DUP SWAP ARITHMETIC_OP SWAP RETURN JUMPDEST CALLVALUE DUP ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST POP PUSH2 PUSH1 DUP CALLDATASIZE ARITHMETIC_OP PUSH1 DUP COMPARASION ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST DUP ARITHMETIC_OP SWAP DUP DUP CALLDATALOAD SWAP PUSH1 ARITHMETIC_OP SWAP SWAP SWAP SWAP POP POP POP PUSH2 JUMP JUMPDEST STOP JUMPDEST CALLVALUE DUP ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST POP PUSH2 PUSH1 DUP CALLDATASIZE ARITHMETIC_OP PUSH1 DUP COMPARASION ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST DUP ARITHMETIC_OP SWAP DUP DUP CALLDATALOAD SWAP PUSH1 ARITHMETIC_OP SWAP SWAP SWAP SWAP DUP CALLDATALOAD PUSH LOGIC_OP SWAP PUSH1 ARITHMETIC_OP SWAP SWAP SWAP SWAP DUP CALLDATALOAD SWAP PUSH1 ARITHMETIC_OP SWAP SWAP SWAP SWAP DUP CALLDATALOAD SWAP PUSH1 ARITHMETIC_OP SWAP PUSH DUP COMPARASION ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST DUP ARITHMETIC_OP DUP PUSH1 DUP ARITHMETIC_OP COMPARASION ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST DUP CALLDATALOAD SWAP PUSH1 ARITHMETIC_OP SWAP DUP PUSH1 DUP ARITHMETIC_OP DUP ARITHMETIC_OP COMPARASION PUSH DUP COMPARASION LOGIC_OP ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST SWAP SWAP DUP DUP PUSH1 ARITHMETIC_OP PUSH1 DUP SWAP ARITHMETIC_OP ARITHMETIC_OP PUSH1 ARITHMETIC_OP PUSH1 MLOAD SWAP DUP ARITHMETIC_OP PUSH1 MSTORE DUP SWAP SWAP SWAP SWAP DUP DUP MSTORE PUSH1 ARITHMETIC_OP DUP DUP DUP DUP DUP CALLDATACOPY PUSH1 DUP DUP ARITHMETIC_OP MSTORE PUSH1 LOGIC_OP PUSH1 DUP ARITHMETIC_OP LOGIC_OP SWAP POP DUP DUP ARITHMETIC_OP SWAP POP POP POP POP POP POP POP SWAP SWAP SWAP SWAP SWAP POP POP POP PUSH2 JUMP JUMPDEST PUSH1 MLOAD DUP DUP ISZERO ISZERO ISZERO ISZERO DUP MSTORE PUSH1 ARITHMETIC_OP SWAP POP POP PUSH1 MLOAD DUP SWAP ARITHMETIC_OP SWAP RETURN JUMPDEST CALLVALUE DUP ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST POP PUSH2 PUSH1 DUP CALLDATASIZE ARITHMETIC_OP PUSH1 DUP COMPARASION ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST DUP ARITHMETIC_OP SWAP DUP DUP CALLDATALOAD PUSH LOGIC_OP SWAP PUSH1 ARITHMETIC_OP SWAP SWAP SWAP SWAP POP POP POP PUSH2 JUMP JUMPDEST STOP JUMPDEST CALLVALUE DUP ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST POP PUSH2 PUSH1 DUP CALLDATASIZE ARITHMETIC_OP PUSH1 DUP COMPARASION ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST DUP ARITHMETIC_OP SWAP DUP DUP CALLDATALOAD SWAP PUSH1 ARITHMETIC_OP SWAP SWAP SWAP SWAP POP POP POP PUSH2 JUMP JUMPDEST STOP JUMPDEST PUSH2 PUSH2 JUMP JUMPDEST STOP JUMPDEST CALLVALUE DUP ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST POP PUSH2 PUSH1 DUP CALLDATASIZE ARITHMETIC_OP PUSH1 DUP COMPARASION ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST DUP ARITHMETIC_OP SWAP DUP DUP CALLDATALOAD PUSH LOGIC_OP SWAP PUSH1 ARITHMETIC_OP SWAP SWAP SWAP SWAP DUP CALLDATALOAD PUSH LOGIC_OP SWAP PUSH1 ARITHMETIC_OP SWAP SWAP SWAP SWAP DUP CALLDATALOAD SWAP PUSH1 ARITHMETIC_OP SWAP SWAP SWAP SWAP DUP CALLDATALOAD SWAP PUSH1 ARITHMETIC_OP SWAP SWAP SWAP SWAP DUP CALLDATALOAD SWAP PUSH1 ARITHMETIC_OP SWAP SWAP SWAP SWAP POP POP POP PUSH2 JUMP JUMPDEST STOP JUMPDEST PUSH1 SLOAD DUP JUMP JUMPDEST PUSH1 DUP DUP SLOAD DUP COMPARASION ISZERO ISZERO PUSH2 JUMPI INVALID JUMPDEST SWAP PUSH1 MSTORE PUSH1 PUSH1 SHA3 SWAP PUSH1 ARITHMETIC_OP ARITHMETIC_OP PUSH1 SWAP POP SWAP POP DUP PUSH1 ARITHMETIC_OP PUSH1 SWAP SLOAD SWAP PUSH2 ARITHMETIC_OP SWAP ARITHMETIC_OP PUSH LOGIC_OP SWAP DUP PUSH1 ARITHMETIC_OP SLOAD SWAP DUP PUSH1 ARITHMETIC_OP DUP SLOAD PUSH1 DUP PUSH1 LOGIC_OP ISZERO PUSH2 ARITHMETIC_OP ARITHMETIC_OP LOGIC_OP PUSH1 SWAP ARITHMETIC_OP DUP PUSH1 ARITHMETIC_OP PUSH1 DUP SWAP ARITHMETIC_OP ARITHMETIC_OP PUSH1 ARITHMETIC_OP PUSH1 MLOAD SWAP DUP ARITHMETIC_OP PUSH1 MSTORE DUP SWAP SWAP SWAP DUP DUP MSTORE PUSH1 ARITHMETIC_OP DUP DUP SLOAD PUSH1 DUP PUSH1 LOGIC_OP ISZERO PUSH2 ARITHMETIC_OP ARITHMETIC_OP LOGIC_OP PUSH1 SWAP ARITHMETIC_OP DUP ISZERO PUSH2 JUMPI DUP PUSH1 COMPARASION PUSH2 JUMPI PUSH2 DUP DUP SLOAD ARITHMETIC_OP ARITHMETIC_OP DUP MSTORE SWAP PUSH1 ARITHMETIC_OP SWAP PUSH2 JUMP JUMPDEST DUP ARITHMETIC_OP SWAP SWAP PUSH1 MSTORE PUSH1 PUSH1 SHA3 SWAP JUMPDEST DUP SLOAD DUP MSTORE SWAP PUSH1 ARITHMETIC_OP SWAP PUSH1 ARITHMETIC_OP DUP DUP COMPARASION PUSH2 JUMPI DUP SWAP ARITHMETIC_OP PUSH1 LOGIC_OP DUP ARITHMETIC_OP SWAP JUMPDEST POP POP POP POP POP SWAP DUP PUSH1 ARITHMETIC_OP PUSH1 SWAP SLOAD SWAP PUSH2 ARITHMETIC_OP SWAP ARITHMETIC_OP PUSH1 LOGIC_OP SWAP DUP PUSH1 ARITHMETIC_OP SLOAD SWAP DUP PUSH1 ARITHMETIC_OP PUSH1 SWAP SLOAD SWAP PUSH2 ARITHMETIC_OP SWAP ARITHMETIC_OP PUSH1 LOGIC_OP SWAP DUP PUSH1 ARITHMETIC_OP PUSH1 SWAP SLOAD SWAP PUSH2 ARITHMETIC_OP SWAP ARITHMETIC_OP PUSH1 LOGIC_OP SWAP DUP PUSH1 ARITHMETIC_OP SLOAD SWAP DUP PUSH1 ARITHMETIC_OP SLOAD SWAP POP DUP JUMP JUMPDEST PUSH1 PUSH1 SWAP SLOAD SWAP PUSH2 ARITHMETIC_OP SWAP ARITHMETIC_OP PUSH1 LOGIC_OP ISZERO ISZERO ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST PUSH1 DUP COMPARASION ISZERO ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST CONSTANT2 PUSH LOGIC_OP PUSH2 PUSH1 SLOAD SWAP DUP ISZERO ARITHMETIC_OP SWAP PUSH1 MLOAD PUSH1 PUSH1 MLOAD DUP DUP ARITHMETIC_OP DUP DUP DUP DUP CALL SWAP POP POP POP POP ISZERO DUP ISZERO PUSH2 JUMPI RETURNDATASIZE PUSH1 DUP RETURNDATACOPY RETURNDATASIZE PUSH1 REVERT JUMPDEST POP PUSH1 PUSH1 PUSH1 PUSH2 ARITHMETIC_OP DUP SLOAD DUP PUSH1 ARITHMETIC_OP LOGIC_OP LOGIC_OP SWAP DUP ISZERO ISZERO ARITHMETIC_OP LOGIC_OP SWAP SSTORE POP POP JUMP JUMPDEST PUSH1 PUSH1 SWAP SLOAD SWAP PUSH2 ARITHMETIC_OP SWAP ARITHMETIC_OP PUSH1 LOGIC_OP ISZERO ISZERO ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST PUSH1 DUP COMPARASION ISZERO ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST CONSTANT2 PUSH LOGIC_OP PUSH2 PUSH1 SLOAD SWAP DUP ISZERO ARITHMETIC_OP SWAP PUSH1 MLOAD PUSH1 PUSH1 MLOAD DUP DUP ARITHMETIC_OP DUP DUP DUP DUP CALL SWAP POP POP POP POP ISZERO DUP ISZERO PUSH2 JUMPI RETURNDATASIZE PUSH1 DUP RETURNDATACOPY RETURNDATASIZE PUSH1 REVERT JUMPDEST POP PUSH1 PUSH1 PUSH1 PUSH2 ARITHMETIC_OP DUP SLOAD DUP PUSH1 ARITHMETIC_OP LOGIC_OP LOGIC_OP SWAP DUP ISZERO ISZERO ARITHMETIC_OP LOGIC_OP SWAP SSTORE POP POP JUMP JUMPDEST PUSH1 PUSH1 SWAP SLOAD SWAP PUSH2 ARITHMETIC_OP SWAP ARITHMETIC_OP PUSH LOGIC_OP PUSH LOGIC_OP PUSH2 CALLVALUE SWAP DUP ISZERO ARITHMETIC_OP SWAP PUSH1 MLOAD PUSH1 PUSH1 MLOAD DUP DUP ARITHMETIC_OP DUP DUP DUP DUP CALL SWAP POP POP POP POP ISZERO DUP ISZERO PUSH2 JUMPI RETURNDATASIZE PUSH1 DUP RETURNDATACOPY RETURNDATASIZE PUSH1 REVERT JUMPDEST POP JUMP JUMPDEST PUSH1 MLOAD PUSH1 ARITHMETIC_OP DUP DUP PUSH1 ARITHMETIC_OP DUP DUP ARITHMETIC_OP DUP MSTORE PUSH1 DUP MSTORE PUSH1 ARITHMETIC_OP DUP PUSH DUP MSTORE POP PUSH1 ARITHMETIC_OP SWAP POP POP PUSH1 MLOAD PUSH1 DUP DUP ARITHMETIC_OP ARITHMETIC_OP DUP MSTORE SWAP PUSH1 MSTORE DUP MLOAD SWAP PUSH1 ARITHMETIC_OP SHA3 DUP PUSH1 MLOAD PUSH1 ARITHMETIC_OP DUP DUP DUP MSTORE PUSH1 ARITHMETIC_OP SWAP POP POP PUSH1 MLOAD PUSH1 DUP DUP ARITHMETIC_OP ARITHMETIC_OP DUP MSTORE SWAP PUSH1 MSTORE DUP MLOAD SWAP PUSH1 ARITHMETIC_OP SHA3 EQ ISZERO PUSH2 JUMPI CONSTANT2 PUSH1 PUSH1 PUSH2 ARITHMETIC_OP DUP SLOAD DUP PUSH ARITHMETIC_OP LOGIC_OP LOGIC_OP SWAP DUP PUSH LOGIC_OP ARITHMETIC_OP LOGIC_OP SWAP SSTORE POP JUMPDEST POP JUMP JUMPDEST PUSH1 MLOAD PUSH1 ARITHMETIC_OP DUP DUP PUSH1 ARITHMETIC_OP DUP DUP ARITHMETIC_OP DUP MSTORE PUSH1 DUP MSTORE PUSH1 ARITHMETIC_OP DUP PUSH DUP MSTORE POP PUSH1 ARITHMETIC_OP SWAP POP POP PUSH1 MLOAD PUSH1 DUP DUP ARITHMETIC_OP ARITHMETIC_OP DUP MSTORE SWAP PUSH1 MSTORE DUP MLOAD SWAP PUSH1 ARITHMETIC_OP SHA3 DUP PUSH1 MLOAD PUSH1 ARITHMETIC_OP DUP DUP DUP MSTORE PUSH1 ARITHMETIC_OP SWAP POP POP PUSH1 MLOAD PUSH1 DUP DUP ARITHMETIC_OP ARITHMETIC_OP DUP MSTORE SWAP PUSH1 MSTORE DUP MLOAD SWAP PUSH1 ARITHMETIC_OP SHA3 EQ ISZERO PUSH2 JUMPI CONSTANT2 PUSH1 PUSH1 PUSH2 ARITHMETIC_OP DUP SLOAD DUP PUSH ARITHMETIC_OP LOGIC_OP LOGIC_OP SWAP DUP PUSH LOGIC_OP ARITHMETIC_OP LOGIC_OP SWAP SSTORE POP JUMPDEST POP JUMP JUMPDEST PUSH1 DUP SWAP SLOAD SWAP PUSH2 ARITHMETIC_OP SWAP ARITHMETIC_OP PUSH1 LOGIC_OP ISZERO ISZERO ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST PUSH1 PUSH1 SWAP SLOAD SWAP PUSH2 ARITHMETIC_OP SWAP ARITHMETIC_OP PUSH LOGIC_OP PUSH LOGIC_OP CONSTANT2 PUSH LOGIC_OP EQ ISZERO ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST PUSH1 PUSH1 SWAP SLOAD SWAP PUSH2 ARITHMETIC_OP SWAP ARITHMETIC_OP PUSH LOGIC_OP PUSH LOGIC_OP PUSH2 PUSH1 SLOAD SWAP DUP ISZERO ARITHMETIC_OP SWAP PUSH1 MLOAD PUSH1 PUSH1 MLOAD DUP DUP ARITHMETIC_OP DUP DUP DUP DUP CALL SWAP POP POP POP POP ISZERO DUP ISZERO PUSH2 JUMPI RETURNDATASIZE PUSH1 DUP RETURNDATACOPY RETURNDATASIZE PUSH1 REVERT JUMPDEST POP CALLVALUE PUSH1 DUP SWAP SSTORE POP JUMP JUMPDEST PUSH1 PUSH1 SWAP SLOAD SWAP PUSH2 ARITHMETIC_OP SWAP ARITHMETIC_OP PUSH1 LOGIC_OP DUP JUMP JUMPDEST PUSH1 MLOAD PUSH1 ARITHMETIC_OP DUP DUP PUSH1 ARITHMETIC_OP DUP DUP ARITHMETIC_OP DUP MSTORE PUSH1 DUP MSTORE PUSH1 ARITHMETIC_OP DUP PUSH DUP MSTORE POP PUSH1 ARITHMETIC_OP SWAP POP POP PUSH1 MLOAD PUSH1 DUP DUP ARITHMETIC_OP ARITHMETIC_OP DUP MSTORE SWAP PUSH1 MSTORE DUP MLOAD SWAP PUSH1 ARITHMETIC_OP SHA3 DUP PUSH1 MLOAD PUSH1 ARITHMETIC_OP DUP DUP DUP MSTORE PUSH1 ARITHMETIC_OP SWAP POP POP PUSH1 MLOAD PUSH1 DUP DUP ARITHMETIC_OP ARITHMETIC_OP DUP MSTORE SWAP PUSH1 MSTORE DUP MLOAD SWAP PUSH1 ARITHMETIC_OP SHA3 EQ ISZERO PUSH2 JUMPI CONSTANT2 PUSH1 PUSH1 PUSH2 ARITHMETIC_OP DUP SLOAD DUP PUSH ARITHMETIC_OP LOGIC_OP LOGIC_OP SWAP DUP PUSH LOGIC_OP ARITHMETIC_OP LOGIC_OP SWAP SSTORE POP JUMPDEST POP JUMP JUMPDEST PUSH1 PUSH1 SWAP SLOAD SWAP PUSH2 ARITHMETIC_OP SWAP ARITHMETIC_OP PUSH1 LOGIC_OP ISZERO ISZERO ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST PUSH1 DUP COMPARASION ISZERO ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST CONSTANT2 PUSH LOGIC_OP PUSH2 PUSH1 SLOAD SWAP DUP ISZERO ARITHMETIC_OP SWAP PUSH1 MLOAD PUSH1 PUSH1 MLOAD DUP DUP ARITHMETIC_OP DUP DUP DUP DUP CALL SWAP POP POP POP POP ISZERO DUP ISZERO PUSH2 JUMPI RETURNDATASIZE PUSH1 DUP RETURNDATACOPY RETURNDATASIZE PUSH1 REVERT JUMPDEST POP PUSH1 PUSH1 PUSH1 PUSH2 ARITHMETIC_OP DUP SLOAD DUP PUSH1 ARITHMETIC_OP LOGIC_OP LOGIC_OP SWAP DUP ISZERO ISZERO ARITHMETIC_OP LOGIC_OP SWAP SSTORE POP POP JUMP JUMPDEST PUSH1 PUSH1 SWAP SLOAD SWAP PUSH2 ARITHMETIC_OP SWAP ARITHMETIC_OP PUSH1 LOGIC_OP ISZERO ISZERO ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST PUSH1 DUP COMPARASION ISZERO ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST CONSTANT2 PUSH LOGIC_OP PUSH2 PUSH1 SLOAD SWAP DUP ISZERO ARITHMETIC_OP SWAP PUSH1 MLOAD PUSH1 PUSH1 MLOAD DUP DUP ARITHMETIC_OP DUP DUP DUP DUP CALL SWAP POP POP POP POP ISZERO DUP ISZERO PUSH2 JUMPI RETURNDATASIZE PUSH1 DUP RETURNDATACOPY RETURNDATASIZE PUSH1 REVERT JUMPDEST POP PUSH1 PUSH1 PUSH1 PUSH2 ARITHMETIC_OP DUP SLOAD DUP PUSH1 ARITHMETIC_OP LOGIC_OP LOGIC_OP SWAP DUP ISZERO ISZERO ARITHMETIC_OP LOGIC_OP SWAP SSTORE POP POP JUMP JUMPDEST PUSH1 PUSH1 SWAP SLOAD SWAP PUSH2 ARITHMETIC_OP SWAP ARITHMETIC_OP PUSH1 LOGIC_OP ISZERO ISZERO ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST PUSH1 DUP COMPARASION ISZERO ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST CONSTANT2 PUSH LOGIC_OP PUSH2 PUSH1 SLOAD SWAP DUP ISZERO ARITHMETIC_OP SWAP PUSH1 MLOAD PUSH1 PUSH1 MLOAD DUP DUP ARITHMETIC_OP DUP DUP DUP DUP CALL SWAP POP POP POP POP ISZERO DUP ISZERO PUSH2 JUMPI RETURNDATASIZE PUSH1 DUP RETURNDATACOPY RETURNDATASIZE PUSH1 REVERT JUMPDEST POP PUSH1 PUSH1 PUSH1 PUSH2 ARITHMETIC_OP DUP SLOAD DUP PUSH1 ARITHMETIC_OP LOGIC_OP LOGIC_OP SWAP DUP ISZERO ISZERO ARITHMETIC_OP LOGIC_OP SWAP SSTORE POP POP JUMP JUMPDEST PUSH1 PUSH1 SWAP SLOAD SWAP PUSH2 ARITHMETIC_OP SWAP ARITHMETIC_OP PUSH LOGIC_OP PUSH LOGIC_OP PUSH2 CALLVALUE SWAP DUP ISZERO ARITHMETIC_OP SWAP PUSH1 MLOAD PUSH1 PUSH1 MLOAD DUP DUP ARITHMETIC_OP DUP DUP DUP DUP CALL SWAP POP POP POP POP ISZERO DUP ISZERO PUSH2 JUMPI RETURNDATASIZE PUSH1 DUP RETURNDATACOPY RETURNDATASIZE PUSH1 REVERT JUMPDEST POP JUMP JUMPDEST PUSH1 PUSH1 SWAP SLOAD SWAP PUSH2 ARITHMETIC_OP SWAP ARITHMETIC_OP PUSH1 LOGIC_OP ISZERO ISZERO ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST PUSH1 PUSH1 SWAP SLOAD SWAP PUSH2 ARITHMETIC_OP SWAP ARITHMETIC_OP PUSH LOGIC_OP PUSH LOGIC_OP CONSTANT2 PUSH LOGIC_OP EQ ISZERO ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST PUSH1 PUSH1 SWAP SLOAD SWAP PUSH2 ARITHMETIC_OP SWAP ARITHMETIC_OP PUSH LOGIC_OP PUSH LOGIC_OP PUSH2 PUSH1 SLOAD SWAP DUP ISZERO ARITHME"]

In [ ]:
Get_vuln(opcode_TOD)

(1, 35)
1/1 [==============================] - 0s 27ms/step
[[0.1898466  0.63702965 0.07470956 0.68430847 0.9822651 ]] TOD


In [ ]:
opcode_timee = ["PUSH1 PUSH1 MSTORE PUSH1 CALLDATASIZE COMPARASION PUSH2 JUMPI PUSH1 CALLDATALOAD PUSH1 SHR DUP PUSH4 COMPARASION PUSH2 JUMPI DUP PUSH4 EQ PUSH2 JUMPI DUP PUSH4 EQ PUSH2 JUMPI DUP PUSH4 EQ PUSH2 JUMPI DUP PUSH4 EQ PUSH2 JUMPI DUP PUSH4 EQ PUSH2 JUMPI PUSH2 JUMP JUMPDEST DUP PUSH4 EQ PUSH2 JUMPI DUP PUSH4 EQ PUSH2 JUMPI DUP PUSH4 EQ PUSH2 JUMPI DUP PUSH4 EQ PUSH2 JUMPI JUMPDEST PUSH1 DUP REVERT JUMPDEST CALLVALUE DUP ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST POP PUSH2 PUSH1 DUP CALLDATASIZE ARITHMETIC_OP PUSH1 DUP COMPARASION ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST DUP ARITHMETIC_OP SWAP DUP DUP CALLDATALOAD SWAP PUSH1 ARITHMETIC_OP SWAP SWAP SWAP SWAP POP POP POP PUSH2 JUMP JUMPDEST STOP JUMPDEST CALLVALUE DUP ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST POP PUSH2 PUSH2 JUMP JUMPDEST STOP JUMPDEST CALLVALUE DUP ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST POP PUSH2 PUSH1 DUP CALLDATASIZE ARITHMETIC_OP PUSH1 DUP COMPARASION ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST DUP ARITHMETIC_OP SWAP DUP DUP CALLDATALOAD SWAP PUSH1 ARITHMETIC_OP SWAP SWAP SWAP SWAP POP POP POP PUSH2 JUMP JUMPDEST STOP JUMPDEST CALLVALUE DUP ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST POP PUSH2 PUSH2 JUMP JUMPDEST PUSH1 MLOAD DUP DUP PUSH LOGIC_OP PUSH LOGIC_OP DUP MSTORE PUSH1 ARITHMETIC_OP SWAP POP POP PUSH1 MLOAD DUP SWAP ARITHMETIC_OP SWAP RETURN JUMPDEST PUSH2 PUSH2 JUMP JUMPDEST STOP JUMPDEST CALLVALUE DUP ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST POP PUSH2 PUSH1 DUP CALLDATASIZE ARITHMETIC_OP PUSH1 DUP COMPARASION ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST DUP ARITHMETIC_OP SWAP DUP DUP CALLDATALOAD SWAP PUSH1 ARITHMETIC_OP SWAP SWAP SWAP SWAP POP POP POP PUSH2 JUMP JUMPDEST STOP JUMPDEST CALLVALUE DUP ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST POP PUSH2 PUSH2 JUMP JUMPDEST PUSH1 MLOAD DUP DUP PUSH LOGIC_OP PUSH LOGIC_OP DUP MSTORE PUSH1 ARITHMETIC_OP SWAP POP POP PUSH1 MLOAD DUP SWAP ARITHMETIC_OP SWAP RETURN JUMPDEST CALLVALUE DUP ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST POP PUSH2 PUSH1 DUP CALLDATASIZE ARITHMETIC_OP PUSH1 DUP COMPARASION ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST DUP ARITHMETIC_OP SWAP DUP DUP CALLDATALOAD SWAP PUSH1 ARITHMETIC_OP SWAP SWAP SWAP SWAP POP POP POP PUSH2 JUMP JUMPDEST STOP JUMPDEST CALLVALUE DUP ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST POP PUSH2 PUSH1 DUP CALLDATASIZE ARITHMETIC_OP PUSH1 DUP COMPARASION ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST DUP ARITHMETIC_OP SWAP DUP DUP CALLDATALOAD PUSH LOGIC_OP SWAP PUSH1 ARITHMETIC_OP SWAP SWAP SWAP SWAP POP POP POP PUSH2 JUMP JUMPDEST STOP JUMPDEST PUSH1 CONSTANT1 SWAP POP DUP PUSH3 DUP ARITHMETIC_OP EQ ISZERO PUSH2 JUMPI CONSTANT2 PUSH1 PUSH1 PUSH2 ARITHMETIC_OP DUP SLOAD DUP PUSH ARITHMETIC_OP LOGIC_OP LOGIC_OP SWAP DUP PUSH LOGIC_OP ARITHMETIC_OP LOGIC_OP SWAP SSTORE POP JUMPDEST POP POP JUMP JUMPDEST PUSH1 PUSH1 SWAP SLOAD SWAP PUSH2 ARITHMETIC_OP SWAP ARITHMETIC_OP PUSH LOGIC_OP PUSH LOGIC_OP CONSTANT2 PUSH LOGIC_OP EQ PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST PUSH1 PUSH1 SWAP SLOAD SWAP PUSH2 ARITHMETIC_OP SWAP ARITHMETIC_OP PUSH LOGIC_OP PUSH LOGIC_OP PUSH1 DUP SWAP SLOAD SWAP PUSH2 ARITHMETIC_OP SWAP ARITHMETIC_OP PUSH LOGIC_OP PUSH LOGIC_OP PUSH PUSH1 MLOAD PUSH1 MLOAD DUP SWAP ARITHMETIC_OP SWAP LOG PUSH1 PUSH1 SWAP SLOAD SWAP PUSH2 ARITHMETIC_OP SWAP ARITHMETIC_OP PUSH LOGIC_OP PUSH1 DUP PUSH2 ARITHMETIC_OP DUP SLOAD DUP PUSH ARITHMETIC_OP LOGIC_OP LOGIC_OP SWAP DUP PUSH LOGIC_OP ARITHMETIC_OP LOGIC_OP SWAP SSTORE POP PUSH1 PUSH1 PUSH1 PUSH2 ARITHMETIC_OP DUP SLOAD DUP PUSH ARITHMETIC_OP LOGIC_OP LOGIC_OP SWAP DUP PUSH LOGIC_OP ARITHMETIC_OP LOGIC_OP SWAP SSTORE POP JUMP JUMPDEST PUSH1 CONSTANT1 SWAP POP DUP PUSH3 DUP ARITHMETIC_OP EQ ISZERO PUSH2 JUMPI CONSTANT2 PUSH1 PUSH1 PUSH2 ARITHMETIC_OP DUP SLOAD DUP PUSH ARITHMETIC_OP LOGIC_OP LOGIC_OP SWAP DUP PUSH LOGIC_OP ARITHMETIC_OP LOGIC_OP SWAP SSTORE POP JUMPDEST POP POP JUMP JUMPDEST PUSH1 DUP SWAP SLOAD SWAP PUSH2 ARITHMETIC_OP SWAP ARITHMETIC_OP PUSH LOGIC_OP DUP JUMP JUMPDEST PUSH1 PUSH CALLVALUE EQ PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST DUP CONSTANT1 EQ ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST CONSTANT1 SWAP POP PUSH1 PUSH1 CONSTANT1 DUP PUSH2 JUMPI INVALID JUMPDEST ARITHMETIC_OP EQ ISZERO PUSH2 JUMPI CONSTANT2 PUSH LOGIC_OP PUSH2 CONSTANT2 PUSH LOGIC_OP BALANCE SWAP DUP ISZERO ARITHMETIC_OP SWAP PUSH1 MLOAD PUSH1 PUSH1 MLOAD DUP DUP ARITHMETIC_OP DUP DUP DUP DUP CALL SWAP POP POP POP POP ISZERO DUP ISZERO PUSH2 JUMPI RETURNDATASIZE PUSH1 DUP RETURNDATACOPY RETURNDATASIZE PUSH1 REVERT JUMPDEST POP JUMPDEST POP JUMP JUMPDEST PUSH1 CONSTANT1 SWAP POP DUP PUSH3 DUP ARITHMETIC_OP EQ ISZERO PUSH2 JUMPI CONSTANT2 PUSH1 PUSH1 PUSH2 ARITHMETIC_OP DUP SLOAD DUP PUSH ARITHMETIC_OP LOGIC_OP LOGIC_OP SWAP DUP PUSH LOGIC_OP ARITHMETIC_OP LOGIC_OP SWAP SSTORE POP JUMPDEST POP POP JUMP JUMPDEST PUSH1 PUSH1 SWAP SLOAD SWAP PUSH2 ARITHMETIC_OP SWAP ARITHMETIC_OP PUSH LOGIC_OP DUP JUMP JUMPDEST CONSTANT1 PUSH3 DUP ARITHMETIC_OP EQ ISZERO PUSH2 JUMPI CONSTANT2 PUSH1 PUSH1 PUSH2 ARITHMETIC_OP DUP SLOAD DUP PUSH ARITHMETIC_OP LOGIC_OP LOGIC_OP SWAP DUP PUSH LOGIC_OP ARITHMETIC_OP LOGIC_OP SWAP SSTORE POP JUMPDEST POP JUMP JUMPDEST PUSH1 DUP SWAP SLOAD SWAP PUSH2 ARITHMETIC_OP SWAP ARITHMETIC_OP PUSH LOGIC_OP PUSH LOGIC_OP CONSTANT2 PUSH LOGIC_OP EQ PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST DUP PUSH1 PUSH1 PUSH2 ARITHMETIC_OP DUP SLOAD DUP PUSH ARITHMETIC_OP LOGIC_OP LOGIC_OP SWAP DUP PUSH LOGIC_OP ARITHMETIC_OP LOGIC_OP SWAP SSTORE POP POP JUMP INVALID LOG PUSH SHA3 CONSTANT1 INVALID INVALID INVALID PUSH CONSTANT1 SWAP CONSTANT1 PUSH STOP ARITHMETIC_OP SIGNEXTEND STOP CONSTANT2 PUSH1 PUSH1 MSTORE PUSH1 CALLDATASIZE COMPARASION PUSH2 JUMPI PUSH1 CALLDATALOAD PUSH1 SHR DUP PUSH4 COMPARASION PUSH2 JUMPI DUP PUSH4 COMPARASION PUSH2 JUMPI DUP PUSH4 COMPARASION PUSH2 JUMPI DUP PUSH4 EQ PUSH2 JUMPI DUP PUSH4 EQ PUSH2 JUMPI DUP PUSH4 EQ PUSH2 JUMPI DUP PUSH4 EQ PUSH2 JUMPI DUP PUSH4 EQ PUSH2 JUMPI DUP PUSH4 EQ PUSH2 JUMPI PUSH2 JUMP JUMPDEST DUP PUSH4 EQ PUSH2 JUMPI DUP PUSH4 EQ PUSH2 JUMPI DUP PUSH4 EQ PUSH2 JUMPI DUP PUSH4 EQ PUSH2 JUMPI DUP PUSH4 EQ PUSH2 JUMPI PUSH2 JUMP JUMPDEST DUP PUSH4 COMPARASION PUSH2 JUMPI DUP PUSH4 EQ PUSH2 JUMPI DUP PUSH4 EQ PUSH2 JUMPI DUP PUSH4 EQ PUSH2 JUMPI DUP PUSH4 EQ PUSH2 JUMPI DUP PUSH4 EQ PUSH2 JUMPI DUP PUSH4 EQ PUSH2 JUMPI PUSH2 JUMP JUMPDEST DUP PUSH4 EQ PUSH2 JUMPI DUP PUSH4 EQ PUSH2 JUMPI DUP PUSH4 EQ PUSH2 JUMPI DUP PUSH4 EQ PUSH2 JUMPI DUP PUSH4 EQ PUSH2 JUMPI PUSH2 JUMP JUMPDEST DUP PUSH4 COMPARASION PUSH2 JUMPI DUP PUSH4 COMPARASION PUSH2 JUMPI DUP PUSH4 EQ PUSH2 JUMPI DUP PUSH4 EQ PUSH2 JUMPI DUP PUSH4 EQ PUSH2 JUMPI DUP PUSH4 EQ PUSH2 JUMPI DUP PUSH4 EQ PUSH2 JUMPI DUP PUSH4 EQ PUSH2 JUMPI PUSH2 JUMP JUMPDEST DUP PUSH4 EQ PUSH2 JUMPI DUP PUSH4 EQ PUSH2 JUMPI DUP PUSH4 EQ PUSH2 JUMPI DUP PUSH4 EQ PUSH2 JUMPI DUP PUSH4 EQ PUSH2 JUMPI PUSH2 JUMP JUMPDEST DUP PUSH4 COMPARASION PUSH2 JUMPI DUP PUSH4 EQ PUSH2 JUMPI DUP PUSH4 EQ PUSH2 JUMPI DUP PUSH4 EQ PUSH2 JUMPI DUP PUSH4 EQ PUSH2 JUMPI DUP PUSH4 EQ PUSH2 JUMPI DUP PUSH4 EQ PUSH2 JUMPI PUSH2 JUMP JUMPDEST DUP PUSH4 EQ PUSH2 JUMPI DUP PUSH4 EQ PUSH2 JUMPI DUP PUSH4 EQ PUSH2 JUMPI DUP PUSH4 EQ PUSH2 JUMPI DUP PUSH4 EQ PUSH2 JUMPI JUMPDEST PUSH1 DUP REVERT JUMPDEST PUSH2 PUSH2 JUMP JUMPDEST STOP JUMPDEST CALLVALUE DUP ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST POP PUSH2 PUSH2 JUMP JUMPDEST PUSH1 MLOAD DUP DUP PUSH1 ARITHMETIC_OP DUP DUP ARITHMETIC_OP DUP MSTORE DUP DUP DUP MLOAD DUP MSTORE PUSH1 ARITHMETIC_OP SWAP POP DUP MLOAD SWAP PUSH1 ARITHMETIC_OP SWAP DUP DUP DUP PUSH1 JUMPDEST DUP DUP COMPARASION ISZERO PUSH2 JUMPI DUP DUP ARITHMETIC_OP MLOAD DUP DUP ARITHMETIC_OP MSTORE PUSH1 DUP ARITHMETIC_OP SWAP POP PUSH2 JUMP JUMPDEST POP POP POP POP SWAP POP SWAP DUP ARITHMETIC_OP SWAP PUSH1 LOGIC_OP DUP ISZERO PUSH2 JUMPI DUP DUP ARITHMETIC_OP DUP MLOAD PUSH1 DUP PUSH1 ARITHMETIC_OP PUSH2 ARITHMETIC_OP ARITHMETIC_OP LOGIC_OP LOGIC_OP DUP MSTORE PUSH1 ARITHMETIC_OP SWAP POP JUMPDEST POP SWAP POP POP POP PUSH1 MLOAD DUP SWAP ARITHMETIC_OP SWAP RETURN JUMPDEST CALLVALUE DUP ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST POP PUSH2 PUSH1 DUP CALLDATASIZE ARITHMETIC_OP PUSH1 DUP COMPARASION ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST DUP ARITHMETIC_OP SWAP DUP DUP CALLDATALOAD PUSH LOGIC_OP SWAP PUSH1 ARITHMETIC_OP SWAP SWAP SWAP SWAP DUP CALLDATALOAD SWAP PUSH1 ARITHMETIC_OP SWAP SWAP SWAP SWAP POP POP POP PUSH2 JUMP JUMPDEST PUSH1 MLOAD DUP DUP ISZERO ISZERO ISZERO ISZERO DUP MSTORE PUSH1 ARITHMETIC_OP SWAP POP POP PUSH1 MLOAD DUP SWAP ARITHMETIC_OP SWAP RETURN JUMPDEST PUSH2 PUSH2 JUMP JUMPDEST STOP JUMPDEST CALLVALUE DUP ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST POP PUSH2 PUSH2 JUMP JUMPDEST PUSH1 MLOAD DUP DUP DUP MSTORE PUSH1 ARITHMETIC_OP SWAP POP POP PUSH1 MLOAD DUP SWAP ARITHMETIC_OP SWAP RETURN JUMPDEST CALLVALUE DUP ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST POP PUSH2 PUSH1 DUP CALLDATASIZE ARITHMETIC_OP PUSH1 DUP COMPARASION ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST DUP ARITHMETIC_OP SWAP DUP DUP CALLDATALOAD PUSH LOGIC_OP SWAP PUSH1 ARITHMETIC_OP SWAP SWAP SWAP SWAP DUP CALLDATALOAD PUSH LOGIC_OP SWAP PUSH1 ARITHMETIC_OP SWAP SWAP SWAP SWAP DUP CALLDATALOAD SWAP PUSH1 ARITHMETIC_OP SWAP SWAP SWAP SWAP POP POP POP PUSH2 JUMP JUMPDEST PUSH1 MLOAD DUP DUP ISZERO ISZERO ISZERO ISZERO DUP MSTORE PUSH1 ARITHMETIC_OP SWAP POP POP PUSH1 MLOAD DUP SWAP ARITHMETIC_OP SWAP RETURN JUMPDEST CALLVALUE DUP ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST POP PUSH2 PUSH1 DUP CALLDATASIZE ARITHMETIC_OP PUSH1 DUP COMPARASION ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST DUP ARITHMETIC_OP SWAP DUP DUP CALLDATALOAD SWAP PUSH1 ARITHMETIC_OP SWAP SWAP SWAP SWAP POP POP POP PUSH2 JUMP JUMPDEST STOP JUMPDEST CALLVALUE DUP ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST POP PUSH2 PUSH2 JUMP JUMPDEST PUSH1 MLOAD DUP DUP PUSH1 LOGIC_OP PUSH1 LOGIC_OP DUP MSTORE PUSH1 ARITHMETIC_OP SWAP POP POP PUSH1 MLOAD DUP SWAP ARITHMETIC_OP SWAP RETURN JUMPDEST CALLVALUE DUP ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST POP PUSH2 PUSH2 JUMP JUMPDEST PUSH1 MLOAD DUP DUP ISZERO ISZERO ISZERO ISZERO DUP MSTORE PUSH1 ARITHMETIC_OP SWAP POP POP PUSH1 MLOAD DUP SWAP ARITHMETIC_OP SWAP RETURN JUMPDEST CALLVALUE DUP ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST POP PUSH2 PUSH1 DUP CALLDATASIZE ARITHMETIC_OP PUSH1 DUP COMPARASION ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST DUP ARITHMETIC_OP SWAP DUP DUP CALLDATALOAD SWAP PUSH1 ARITHMETIC_OP SWAP SWAP SWAP SWAP POP POP POP PUSH2 JUMP JUMPDEST STOP JUMPDEST CALLVALUE DUP ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST POP PUSH2 PUSH1 DUP CALLDATASIZE ARITHMETIC_OP PUSH1 DUP COMPARASION ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST DUP ARITHMETIC_OP SWAP DUP DUP CALLDATALOAD SWAP PUSH1 ARITHMETIC_OP SWAP SWAP SWAP SWAP POP POP POP PUSH2 JUMP JUMPDEST STOP JUMPDEST PUSH2 PUSH2 JUMP JUMPDEST STOP JUMPDEST CALLVALUE DUP ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST POP PUSH2 PUSH2 JUMP JUMPDEST PUSH1 MLOAD DUP DUP ISZERO ISZERO ISZERO ISZERO DUP MSTORE PUSH1 ARITHMETIC_OP SWAP POP POP PUSH1 MLOAD DUP SWAP ARITHMETIC_OP SWAP RETURN JUMPDEST CALLVALUE DUP ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST POP PUSH2 PUSH2 JUMP JUMPDEST PUSH1 MLOAD DUP DUP ISZERO ISZERO ISZERO ISZERO DUP MSTORE PUSH1 ARITHMETIC_OP SWAP POP POP PUSH1 MLOAD DUP SWAP ARITHMETIC_OP SWAP RETURN JUMPDEST CALLVALUE DUP ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST POP PUSH2 PUSH1 DUP CALLDATASIZE ARITHMETIC_OP PUSH1 DUP COMPARASION ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST DUP ARITHMETIC_OP SWAP DUP DUP CALLDATALOAD SWAP PUSH1 ARITHMETIC_OP SWAP SWAP SWAP SWAP POP POP POP PUSH2 JUMP JUMPDEST STOP JUMPDEST CALLVALUE DUP ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST POP PUSH2 PUSH1 DUP CALLDATASIZE ARITHMETIC_OP PUSH1 DUP COMPARASION ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST DUP ARITHMETIC_OP SWAP DUP DUP CALLDATALOAD PUSH LOGIC_OP SWAP PUSH1 ARITHMETIC_OP SWAP SWAP SWAP SWAP DUP CALLDATALOAD SWAP PUSH1 ARITHMETIC_OP SWAP SWAP SWAP SWAP POP POP POP PUSH2 JUMP JUMPDEST PUSH1 MLOAD DUP DUP ISZERO ISZERO ISZERO ISZERO DUP MSTORE PUSH1 ARITHMETIC_OP SWAP POP POP PUSH1 MLOAD DUP SWAP ARITHMETIC_OP SWAP RETURN JUMPDEST CALLVALUE DUP ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST POP PUSH2 PUSH1 DUP CALLDATASIZE ARITHMETIC_OP PUSH1 DUP COMPARASION ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST DUP ARITHMETIC_OP SWAP DUP DUP CALLDATALOAD PUSH LOGIC_OP SWAP PUSH1 ARITHMETIC_OP SWAP SWAP SWAP SWAP POP POP POP PUSH2 JUMP JUMPDEST PUSH1 MLOAD DUP DUP DUP MSTORE PUSH1 ARITHMETIC_OP SWAP POP POP PUSH1 MLOAD DUP SWAP ARITHMETIC_OP SWAP RETURN JUMPDEST CALLVALUE DUP ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST POP PUSH2 PUSH2 JUMP JUMPDEST STOP JUMPDEST CALLVALUE DUP ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST POP PUSH2 PUSH2 JUMP JUMPDEST PUSH1 MLOAD DUP DUP ISZERO ISZERO ISZERO ISZERO DUP MSTORE PUSH1 ARITHMETIC_OP SWAP POP POP PUSH1 MLOAD DUP SWAP ARITHMETIC_OP SWAP RETURN JUMPDEST CALLVALUE DUP ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST POP PUSH2 PUSH1 DUP CALLDATASIZE ARITHMETIC_OP PUSH1 DUP COMPARASION ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST DUP ARITHMETIC_OP SWAP DUP DUP CALLDATALOAD SWAP PUSH1 ARITHMETIC_OP SWAP SWAP SWAP SWAP POP POP POP PUSH2 JUMP JUMPDEST STOP JUMPDEST CALLVALUE DUP ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST POP PUSH2 PUSH1 DUP CALLDATASIZE ARITHMETIC_OP PUSH1 DUP COMPARASION ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST DUP ARITHMETIC_OP SWAP DUP DUP CALLDATALOAD SWAP PUSH1 ARITHMETIC_OP SWAP SWAP SWAP SWAP POP POP POP PUSH2 JUMP JUMPDEST STOP JUMPDEST CALLVALUE DUP ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST POP PUSH2 PUSH2 JUMP JUMPDEST PUSH1 MLOAD DUP DUP ISZERO ISZERO ISZERO ISZERO DUP MSTORE PUSH1 ARITHMETIC_OP SWAP POP POP PUSH1 MLOAD DUP SWAP ARITHMETIC_OP SWAP RETURN JUMPDEST CALLVALUE DUP ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST POP PUSH2 PUSH1 DUP CALLDATASIZE ARITHMETIC_OP PUSH1 DUP COMPARASION ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST DUP ARITHMETIC_OP SWAP DUP DUP CALLDATALOAD SWAP PUSH1 ARITHMETIC_OP SWAP SWAP SWAP SWAP POP POP POP PUSH2 JUMP JUMPDEST STOP JUMPDEST CALLVALUE DUP ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST POP PUSH2 PUSH2 JUMP JUMPDEST PUSH1 MLOAD DUP DUP PUSH LOGIC_OP PUSH LOGIC_OP DUP MSTORE PUSH1 ARITHMETIC_OP SWAP POP POP PUSH1 MLOAD DUP SWAP ARITHMETIC_OP SWAP RETURN JUMPDEST CALLVALUE DUP ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST POP PUSH2 PUSH2 JUMP JUMPDEST PUSH1 MLOAD DUP DUP PUSH1 ARITHMETIC_OP DUP DUP ARITHMETIC_OP DUP MSTORE DUP DUP DUP MLOAD DUP MSTORE PUSH1 ARITHMETIC_OP SWAP POP DUP MLOAD SWAP PUSH1 ARITHMETIC_OP SWAP DUP DUP DUP PUSH1 JUMPDEST DUP DUP COMPARASION ISZERO PUSH2 JUMPI DUP DUP ARITHMETIC_OP MLOAD DUP DUP ARITHMETIC_OP MSTORE PUSH1 DUP ARITHMETIC_OP SWAP POP PUSH2 JUMP JUMPDEST POP POP POP POP SWAP POP SWAP DUP ARITHMETIC_OP SWAP PUSH1 LOGIC_OP DUP ISZERO PUSH2 JUMPI DUP DUP ARITHMETIC_OP DUP MLOAD PUSH1 DUP PUSH1 ARITHMETIC_OP PUSH2 ARITHMETIC_OP ARITHMETIC_OP LOGIC_OP LOGIC_OP DUP MSTORE PUSH1 ARITHMETIC_OP SWAP POP JUMPDEST POP SWAP POP POP POP PUSH1 MLOAD DUP SWAP ARITHMETIC_OP SWAP RETURN JUMPDEST CALLVALUE DUP ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST POP PUSH2 PUSH1 DUP CALLDATASIZE ARITHMETIC_OP PUSH1 DUP COMPARASION ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST DUP ARITHMETIC_OP SWAP DUP DUP CALLDATALOAD SWAP PUSH1 ARITHMETIC_OP SWAP SWAP SWAP SWAP POP POP POP PUSH2 JUMP JUMPDEST STOP JUMPDEST CALLVALUE DUP ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST POP PUSH2 PUSH1 DUP CALLDATASIZE ARITHMETIC_OP PUSH1 DUP COMPARASION ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST DUP ARITHMETIC_OP SWAP DUP DUP CALLDATALOAD SWAP PUSH1 ARITHMETIC_OP SWAP SWAP SWAP SWAP POP POP POP PUSH2 JUMP JUMPDEST STOP JUMPDEST CALLVALUE DUP ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST POP PUSH2 PUSH1 DUP CALLDATASIZE ARITHMETIC_OP PUSH1 DUP COMPARASION ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST DUP ARITHMETIC_OP SWAP DUP DUP CALLDATALOAD SWAP PUSH1 ARITHMETIC_OP SWAP SWAP SWAP SWAP POP POP POP PUSH2 JUMP JUMPDEST STOP JUMPDEST CALLVALUE DUP ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST POP PUSH2 PUSH1 DUP CALLDATASIZE ARITHMETIC_OP PUSH1 DUP COMPARASION ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST DUP ARITHMETIC_OP SWAP DUP DUP CALLDATALOAD SWAP PUSH1 ARITHMETIC_OP SWAP SWAP SWAP SWAP POP POP POP PUSH2 JUMP JUMPDEST STOP JUMPDEST CALLVALUE DUP ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST POP PUSH2 PUSH2 JUMP JUMPDEST PUSH1 MLOAD DUP DUP ISZERO ISZERO ISZERO ISZERO DUP MSTORE PUSH1 ARITHMETIC_OP SWAP POP POP PUSH1 MLOAD DUP SWAP ARITHMETIC_OP SWAP RETURN JUMPDEST CALLVALUE DUP ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST POP PUSH2 PUSH1 DUP CALLDATASIZE ARITHMETIC_OP PUSH1 DUP COMPARASION ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST DUP ARITHMETIC_OP SWAP DUP DUP CALLDATALOAD PUSH LOGIC_OP SWAP PUSH1 ARITHMETIC_OP SWAP SWAP SWAP SWAP DUP CALLDATALOAD SWAP PUSH1 ARITHMETIC_OP SWAP SWAP SWAP SWAP POP POP POP PUSH2 JUMP JUMPDEST PUSH1 MLOAD DUP DUP ISZERO ISZERO ISZERO ISZERO DUP MSTORE PUSH1 ARITHMETIC_OP SWAP POP POP PUSH1 MLOAD DUP SWAP ARITHMETIC_OP SWAP RETURN JUMPDEST PUSH2 PUSH2 JUMP JUMPDEST STOP JUMPDEST CALLVALUE DUP ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST POP PUSH2 PUSH2 JUMP JUMPDEST PUSH1 MLOAD DUP DUP ISZERO ISZERO ISZERO ISZERO DUP MSTORE PUSH1 ARITHMETIC_OP SWAP POP POP PUSH1 MLOAD DUP SWAP ARITHMETIC_OP SWAP RETURN JUMPDEST CALLVALUE DUP ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST POP PUSH2 PUSH1 DUP CALLDATASIZE ARITHMETIC_OP PUSH1 DUP COMPARASION ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST DUP ARITHMETIC_OP SWAP DUP DUP CALLDATALOAD SWAP PUSH1 ARITHMETIC_OP SWAP SWAP SWAP SWAP POP POP POP PUSH2 JUMP JUMPDEST STOP JUMPDEST CALLVALUE DUP ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST POP PUSH2 PUSH1 DUP CALLDATASIZE ARITHMETIC_OP PUSH1 DUP COMPARASION ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST DUP ARITHMETIC_OP SWAP DUP DUP CALLDATALOAD PUSH LOGIC_OP SWAP PUSH1 ARITHMETIC_OP SWAP SWAP SWAP SWAP DUP CALLDATALOAD SWAP PUSH1 ARITHMETIC_OP SWAP SWAP SWAP SWAP DUP CALLDATALOAD SWAP PUSH1 ARITHMETIC_OP SWAP PUSH DUP COMPARASION ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST DUP ARITHMETIC_OP DUP PUSH1 DUP ARITHMETIC_OP COMPARASION ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST DUP CALLDATALOAD SWAP PUSH1 ARITHMETIC_OP SWAP DUP PUSH1 DUP ARITHMETIC_OP DUP ARITHMETIC_OP COMPARASION PUSH DUP COMPARASION LOGIC_OP ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST SWAP SWAP DUP DUP PUSH1 ARITHMETIC_OP PUSH1 DUP SWAP ARITHMETIC_OP ARITHMETIC_OP PUSH1 ARITHMETIC_OP PUSH1 MLOAD SWAP DUP ARITHMETIC_OP PUSH1 MSTORE DUP SWAP SWAP SWAP SWAP DUP DUP MSTORE PUSH1 ARITHMETIC_OP DUP DUP DUP DUP DUP CALLDATACOPY PUSH1 DUP DUP ARITHMETIC_OP MSTORE PUSH1 LOGIC_OP PUSH1 DUP ARITHMETIC_OP LOGIC_OP SWAP POP DUP DUP ARITHMETIC_OP SWAP POP POP POP POP POP POP POP SWAP SWAP SWAP SWAP SWAP POP POP POP PUSH2 JUMP JUMPDEST PUSH1 MLOAD DUP DUP ISZERO ISZERO ISZERO ISZERO DUP MSTORE PUSH1 ARITHMETIC_OP SWAP POP POP PUSH1 MLOAD DUP SWAP ARITHMETIC_OP SWAP RETURN JUMPDEST CALLVALUE DUP ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST POP PUSH2 PUSH2 JUMP JUMPDEST PUSH1 MLOAD DUP DUP PUSH LOGIC_OP PUSH LOGIC_OP DUP MSTORE PUSH1 ARITHMETIC_OP SWAP POP POP PUSH1 MLOAD DUP SWAP ARITHMETIC_OP SWAP RETURN JUMPDEST PUSH2 PUSH2 JUMP JUMPDEST STOP JUMPDEST CALLVALUE DUP ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST POP PUSH2 PUSH1 DUP CALLDATASIZE ARITHMETIC_OP PUSH1 DUP COMPARASION ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST DUP ARITHMETIC_OP SWAP DUP DUP CALLDATALOAD PUSH LOGIC_OP SWAP PUSH1 ARITHMETIC_OP SWAP SWAP SWAP SWAP DUP CALLDATALOAD SWAP PUSH1 ARITHMETIC_OP SWAP SWAP SWAP SWAP POP POP POP PUSH2 JUMP JUMPDEST PUSH1 MLOAD DUP DUP ISZERO ISZERO ISZERO ISZERO DUP MSTORE PUSH1 ARITHMETIC_OP SWAP POP POP PUSH1 MLOAD DUP SWAP ARITHMETIC_OP SWAP RETURN JUMPDEST PUSH2 PUSH2 JUMP JUMPDEST STOP JUMPDEST CALLVALUE DUP ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST POP PUSH2 PUSH1 DUP CALLDATASIZE ARITHMETIC_OP PUSH1 DUP COMPARASION ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST DUP ARITHMETIC_OP SWAP DUP DUP CALLDATALOAD PUSH LOGIC_OP SWAP PUSH1 ARITHMETIC_OP SWAP SWAP SWAP SWAP DUP CALLDATALOAD SWAP PUSH1 ARITHMETIC_OP SWAP SWAP SWAP SWAP POP POP POP PUSH2 JUMP JUMPDEST PUSH1 MLOAD DUP DUP ISZERO ISZERO ISZERO ISZERO DUP MSTORE PUSH1 ARITHMETIC_OP SWAP POP POP PUSH1 MLOAD DUP SWAP ARITHMETIC_OP SWAP RETURN JUMPDEST CALLVALUE DUP ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST POP PUSH2 PUSH1 DUP CALLDATASIZE ARITHMETIC_OP PUSH1 DUP COMPARASION ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST DUP ARITHMETIC_OP SWAP DUP DUP CALLDATALOAD PUSH LOGIC_OP SWAP PUSH1 ARITHMETIC_OP SWAP SWAP SWAP SWAP DUP CALLDATALOAD PUSH LOGIC_OP SWAP PUSH1 ARITHMETIC_OP SWAP SWAP SWAP SWAP POP POP POP PUSH2 JUMP JUMPDEST PUSH1 MLOAD DUP DUP DUP MSTORE PUSH1 ARITHMETIC_OP SWAP POP POP PUSH1 MLOAD DUP SWAP ARITHMETIC_OP SWAP RETURN JUMPDEST CALLVALUE DUP ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST POP PUSH2 PUSH1 DUP CALLDATASIZE ARITHMETIC_OP PUSH1 DUP COMPARASION ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST DUP ARITHMETIC_OP SWAP DUP DUP CALLDATALOAD SWAP PUSH1 ARITHMETIC_OP SWAP SWAP SWAP SWAP POP POP POP PUSH2 JUMP JUMPDEST STOP JUMPDEST CALLVALUE DUP ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST POP PUSH2 PUSH1 DUP CALLDATASIZE ARITHMETIC_OP PUSH1 DUP COMPARASION ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST DUP ARITHMETIC_OP SWAP DUP DUP CALLDATALOAD SWAP PUSH1 ARITHMETIC_OP SWAP SWAP SWAP SWAP POP POP POP PUSH2 JUMP JUMPDEST STOP JUMPDEST CALLVALUE DUP ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST POP PUSH2 PUSH1 DUP CALLDATASIZE ARITHMETIC_OP PUSH1 DUP COMPARASION ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST DUP ARITHMETIC_OP SWAP DUP DUP CALLDATALOAD PUSH LOGIC_OP SWAP PUSH1 ARITHMETIC_OP SWAP SWAP SWAP SWAP POP POP POP PUSH2 JUMP JUMPDEST STOP JUMPDEST PUSH1 PUSH CALLVALUE EQ PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST DUP CONSTANT1 EQ ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST CONSTANT1 SWAP POP PUSH1 PUSH1 CONSTANT1 DUP PUSH2 JUMPI INVALID JUMPDEST ARITHMETIC_OP EQ ISZERO PUSH2 JUMPI CONSTANT2 PUSH LOGIC_OP PUSH2 CONSTANT2 PUSH LOGIC_OP BALANCE SWAP DUP ISZERO ARITHMETIC_OP SWAP PUSH1 MLOAD PUSH1 PUSH1 MLOAD DUP DUP ARITHMETIC_OP DUP DUP DUP DUP CALL SWAP POP POP POP POP ISZERO DUP ISZERO PUSH2 JUMPI RETURNDATASIZE PUSH1 DUP RETURNDATACOPY RETURNDATASIZE PUSH1 REVERT JUMPDEST POP JUMPDEST POP JUMP JUMPDEST PUSH1 DUP SLOAD PUSH1 DUP PUSH1 LOGIC_OP ISZERO PUSH2 ARITHMETIC_OP ARITHMETIC_OP LOGIC_OP PUSH1 SWAP ARITHMETIC_OP DUP PUSH1 ARITHMETIC_OP PUSH1 DUP SWAP ARITHMETIC_OP ARITHMETIC_OP PUSH1 ARITHMETIC_OP PUSH1 MLOAD SWAP DUP ARITHMETIC_OP PUSH1 MSTORE DUP SWAP SWAP SWAP DUP DUP MSTORE PUSH1 ARITHMETIC_OP DUP DUP SLOAD PUSH1 DUP PUSH1 LOGIC_OP ISZERO PUSH2 ARITHMETIC_OP ARITHMETIC_OP LOGIC_OP PUSH1 SWAP ARITHMETIC_OP DUP ISZERO PUSH2 JUMPI DUP PUSH1 COMPARASION PUSH2 JUMPI PUSH2 DUP DUP SLOAD ARITHMETIC_OP ARITHMETIC_OP DUP MSTORE SWAP PUSH1 ARITHMETIC_OP SWAP PUSH2 JUMP JUMPDEST DUP ARITHMETIC_OP SWAP SWAP PUSH1 MSTORE PUSH1 PUSH1 SHA3 SWAP JUMPDEST DUP SLOAD DUP MSTORE SWAP PUSH1 ARITHMETIC_OP SWAP PUSH1 ARITHMETIC_OP DUP DUP COMPARASION PUSH2 JUMPI DUP SWAP ARITHMETIC_OP PUSH1 LOGIC_OP DUP ARITHMETIC_OP SWAP JUMPDEST POP POP POP POP POP DUP JUMP JUMPDEST PUSH1 DUP PUSH1 PUSH1 CONSTANT2 PUSH LOGIC_OP PUSH LOGIC_OP DUP MSTORE PUSH1 ARITHMETIC_OP SWAP DUP MSTORE PUSH1 ARITHMETIC_OP PUSH1 SHA3 PUSH1 DUP PUSH LOGIC_OP PUSH LOGIC_OP DUP MSTORE PUSH1 ARITHMETIC_OP SWAP DUP MSTORE PUSH1 ARITHMETIC_OP PUSH1 SHA3 DUP SWAP SSTORE POP DUP PUSH LOGIC_OP CONSTANT2 PUSH LOGIC_OP PUSH DUP PUSH1 MLOAD DUP DUP DUP MSTORE PUSH1 ARITHMETIC_OP SWAP POP POP PUSH1 MLOAD DUP SWAP ARITHMETIC_OP SWAP LOG PUSH1 SWAP POP SWAP SWAP POP POP JUMP JUMPDEST PUSH1 PUSH CALLVALUE EQ PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST DUP CONSTANT1 EQ ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST CONSTANT1 SWAP POP PUSH1 PUSH1 CONSTANT1 DUP PUSH2 JUMPI INVALID JUMPDEST ARITHMETIC_OP EQ ISZERO PUSH2 JUMPI CONSTANT2 PUSH LOGIC_OP PUSH2 CONSTANT2 PUSH LOGIC_OP BALANCE SWAP DUP ISZERO ARITHMETIC_OP SWAP PUSH1 MLOAD PUSH1 PUSH1 MLOAD DUP DUP ARITHMETIC_OP DUP DUP DUP DUP CALL SWAP POP POP POP POP ISZERO DUP ISZERO PUSH2 JUMPI RETURNDATASIZE PUSH1 DUP RETURNDATACOPY RETURNDATASIZE PUSH1 REVERT JUMPDEST POP JUMPDEST POP JUMP JUMPDEST PUSH1 PUSH2 PUSH1 PUSH1 DUP PUSH LOGIC_OP PUSH LOGIC_OP DUP MSTORE PUSH1 ARITHMETIC_OP SWAP DUP MSTORE PUSH1 ARITHMETIC_OP PUSH1 SHA3 SLOAD PUSH1 SLOAD PUSH2 SWAP SWAP SWAP PUSH4 LOGIC_OP JUMP JUMPDEST SWAP POP SWAP JUMP JUMPDEST PUSH1 PUSH2 DUP PUSH1 PUSH1 DUP PUSH LOGIC_OP PUSH LOGIC_OP DUP MSTORE PUSH1 ARITHMETIC_OP SWAP DUP MSTORE PUSH1 ARITHMETIC_OP PUSH1 SHA3 SLOAD PUSH2 SWAP SWAP SWAP PUSH4 LOGIC_OP JUMP JUMPDEST PUSH1 PUSH1 DUP PUSH LOGIC_OP PUSH LOGIC_OP DUP MSTORE PUSH1 ARITHMETIC_OP SWAP DUP MSTORE PUSH1 ARITHMETIC_OP PUSH1 SHA3 DUP SWAP SSTORE POP PUSH2 DUP PUSH1 PUSH1 DUP PUSH LOGIC_OP PUSH LOGIC_OP DUP MSTORE PUSH1 ARITHMETIC_OP SWAP DUP MSTORE PUSH1 ARITHMETIC_OP PUSH1 SHA3 PUSH1 CONSTANT2 PUSH LOGIC_OP PUSH LOGIC_OP DUP MSTORE PUSH1 ARITHMETIC_OP SWAP DUP MSTORE PUSH1 ARITHMETIC_OP PUSH1 SHA3 SLOAD PUSH2 SWAP SWAP SWAP PUSH4 LOGIC_OP JUMP JUMPDEST PUSH1 PUSH1 DUP PUSH LOGIC_OP PUSH LOGIC_OP DUP MSTORE PUSH1 ARITHMETIC_OP SWAP DUP MSTORE PUSH1 ARITHMETIC_OP PUSH1 SHA3 PUSH1 CONSTANT2 PUSH LOGIC_OP PUSH LOGIC_OP DUP MSTORE PUSH1 ARITHMETIC_OP SWAP DUP MSTORE PUSH1 ARITHMETIC_OP PUSH1 SHA3 DUP SWAP SSTORE POP PUSH2 DUP PUSH1 PUSH1 DUP PUSH LOGIC_OP PUSH LOGIC_OP DUP MSTORE PUSH1 ARITHMETIC_OP SWAP DUP MSTORE PUSH1 ARITHMETIC_OP PUSH1 SHA3 SLOAD PUSH2 SWAP SWAP SWAP PUSH4 LOGIC_OP JUMP JUMPDEST PUSH1 PUSH1 DUP PUSH LOGIC_OP PUSH LOGIC_OP DUP MSTORE PUSH1 ARITHMETIC_OP SWAP DUP MSTORE PUSH1 ARITHMETIC_OP PUSH1 SHA3 DUP SWAP SSTORE POP DUP PUSH LOGIC_OP DUP PUSH LOGIC_OP PUSH DUP PUSH1 MLOAD DUP DUP DUP MSTORE PUSH1 ARITHMETIC_OP SWAP POP POP PUSH1 MLOAD DUP SWAP ARITHMETIC_OP SWAP LOG PUSH1 SWAP POP SWAP SWAP POP POP POP JUMP JUMPDEST PUSH1 CONSTANT1 SWAP POP DUP PUSH3 DUP ARITHMETIC_OP EQ ISZERO PUSH2 JUMPI CONSTANT2 PUSH1 PUSH1 PUSH2 ARITHMETIC_OP DUP SLOAD DUP PUSH ARITHMETIC_OP LOGIC_OP LOGIC_OP SWAP DUP PUSH LOGIC_OP ARITHMETIC_OP LOGIC_OP SWAP SSTORE POP JUMPDEST POP POP JUMP JUMPDEST PUSH1 PUSH1 SWAP SLOAD SWAP PUSH2 ARITHMETIC_OP SWAP ARITHMETIC_OP PUSH1 LOGIC_OP DUP JUMP JUMPDEST PUSH1 PUSH4 CONSTANT1 COMPARASION ISZERO SWAP POP SWAP JUMP JUMPDEST CONSTANT1 PUSH3 DUP ARITHMETIC_OP EQ ISZERO PUSH2 JUMPI CONSTANT2 PUSH1 PUSH1 PUSH2 ARITHMETIC_OP DUP SLOAD DUP PUSH ARITHMETIC_OP LOGIC_OP LOGIC_OP SWAP DUP PUSH LOGIC_OP ARITHMETIC_OP LOGIC_OP SWAP SSTORE POP JUMPDEST POP JUMP JUMPDEST CONSTANT1 PUSH3 DUP ARITHMETIC_OP EQ ISZERO PUSH2 JUMPI CONSTANT2 PUSH1 PUSH1 PUSH2 ARITHMETIC_OP DUP SLOAD DUP PUSH ARITHMETIC_OP LOGIC_OP LOGIC_OP SWAP DUP PUSH LOGIC_OP ARITHMETIC_OP LOGIC_OP SWAP SSTORE POP JUMPDEST POP JUMP JUMPDEST PUSH1 PUSH CALLVALUE EQ PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST DUP CONSTANT1 EQ ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST CONSTANT1 SWAP POP PUSH1 PUSH1 CONSTANT1 DUP PUSH2 JUMPI INVALID JUMPDEST ARITHMETIC_OP EQ ISZERO PUSH2 JUMPI CONSTANT2 PUSH LOGIC_OP PUSH2 CONSTANT2 PUSH LOGIC_OP BALANCE SWAP DUP ISZERO ARITHMETIC_OP SWAP PUSH1 MLOAD PUSH1 PUSH1 MLOAD DUP DUP ARITHMETIC_OP DUP DUP DUP DUP CALL SWAP POP POP POP POP ISZERO DUP ISZERO PUSH2 JUMPI RETURNDATASIZE PUSH1 DUP RETURNDATACOPY RETURNDATASIZE PUSH1 REVERT JUMPDEST POP JUMPDEST POP JUMP JUMPDEST PUSH1 PUSH4 CONSTANT1 COMPARASION ISZERO SWAP POP SWAP JUMP JUMPDEST PUSH1 PUSH4 CONSTANT1 COMPARASION ISZERO SWAP POP SWAP JUMP JUMPDEST PUSH1 CONSTANT1 SWAP POP DUP PUSH3 DUP ARITHMETIC_OP EQ ISZERO PUSH2 JUMPI CONSTANT2 PUSH1 PUSH1 PUSH2 ARITHMETIC_OP DUP SLOAD DUP PUSH ARITHMETIC_OP LOGIC_OP LOGIC_OP SWAP DUP PUSH LOGIC_OP ARITHMETIC_OP LOGIC_OP SWAP SSTORE POP JUMPDEST POP POP JUMP JUMPDEST PUSH1 DUP PUSH1 PUSH1 CONSTANT2 PUSH LOGIC_OP PUSH LOGIC_OP DUP MSTORE PUSH1 ARITHMETIC_OP SWAP DUP MSTORE PUSH1 ARITHMETIC_OP PUSH1 SHA3 PUSH1 DUP PUSH LOGIC_OP PUSH LOGIC_OP DUP MSTORE PUSH1 ARITHMETIC_OP SWAP DUP MSTORE PUSH1 ARITHMETIC_OP PUSH1 SHA3 SLOAD SWAP POP DUP DUP COMPARASION ISZERO PUSH2 JUMPI PUSH1 PUSH1 PUSH1 CONSTANT2 PUSH LOGIC_OP PUSH LOGIC_OP DUP MSTORE PUSH1 ARITHMETIC_OP SWAP DUP MSTORE PUSH1 ARITHMETIC_OP PUSH1 SHA3 PUSH1 DUP PUSH LOGIC_OP PUSH LOGIC_OP DUP MSTORE PUSH1 ARITHMETIC_OP SWAP DUP MSTORE PUSH1 ARITHMETIC_OP PUSH1 SHA3 DUP SWAP SSTORE POP PUSH2 JUMP JUMPDEST PUSH2 DUP DUP PUSH2 SWAP SWAP SWAP PUSH4 LOGIC_OP JUMP JUMPDEST PUSH1 PUSH1 CONSTANT2 PUSH LOGIC_OP PUSH LOGIC_OP DUP MSTORE PUSH1 ARITHMETIC_OP SWAP DUP MSTORE PUSH1 ARITHMETIC_OP PUSH1 SHA3 PUSH1 DUP PUSH LOGIC_OP PUSH LOGIC_OP DUP MSTORE PUSH1 ARITHMETIC_OP SWAP DUP MSTORE PUSH1 ARITHMETIC_OP PUSH1 SHA3 DUP SWAP SSTORE POP JUMPDEST DUP PUSH LOGIC_OP CONSTANT2 PUSH LOGIC_OP PUSH PUSH1 PUSH1 CONSTANT2 PUSH LOGIC_OP PUSH LOGIC_OP DUP MSTORE PUSH1 ARITHMETIC_OP SWAP DUP MSTORE PUSH1 ARITHMETIC_OP PUSH1 SHA3 PUSH1 DUP PUSH LOGIC_OP PUSH LOGIC_OP DUP MSTORE PUSH1 ARITHMETIC_OP SWAP DUP MSTORE PUSH1 ARITHMETIC_OP PUSH1 SHA3 SLOAD PUSH1 MLOAD DUP DUP DUP MSTORE PUSH1 ARITHMETIC_OP SWAP POP POP PUSH1 MLOAD DUP SWAP ARITHMETIC_OP SWAP LOG PUSH1 SWAP POP POP SWAP SWAP POP POP JUMP JUMPDEST PUSH1 PUSH1 PUSH1 DUP PUSH LOGIC_OP PUSH LOGIC_OP DUP MSTORE PUSH1 ARITHMETIC_OP SWAP DUP MSTORE PUSH1 ARITHMETIC_OP PUSH1 SHA3 SLOAD SWAP POP SWAP SWAP POP JUMP JUMPDEST PUSH1 PUSH1 SWAP SLOAD SWAP PUSH2 ARITHMETIC_OP SWAP ARITHMETIC_OP PUSH LOGIC_OP PUSH LOGIC_OP CONSTANT2 PUSH LOGIC_OP EQ PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST PUSH1 PUSH1 SWAP SLOAD SWAP PUSH2 ARITHMETIC_OP SWAP ARITHMETIC_OP PUSH LOGIC_OP PUSH LOGIC_OP PUSH1 PUSH1 SWAP SLOAD SWAP PUSH2 ARITHMETIC_OP SWAP ARITHMETIC_OP PUSH LOGIC_OP PUSH LOGIC_OP PUSH PUSH1 MLOAD PUSH1 MLOAD DUP SWAP ARITHMETIC_OP SWAP LOG PUSH1 PUSH1 SWAP SLOAD SWAP PUSH2 ARITHMETIC_OP SWAP ARITHMETIC_OP PUSH LOGIC_OP PUSH1 PUSH1 PUSH2 ARITHMETIC_OP DUP SLOAD DUP PUSH ARITHMETIC_OP LOGIC_OP LOGIC_OP SWAP DUP PUSH LOGIC_OP ARITHMETIC_OP LOGIC_OP SWAP SSTORE POP PUSH1 PUSH1 PUSH1 PUSH2 ARITHMETIC_OP DUP SLOAD DUP PUSH ARITHMETIC_OP LOGIC_OP LOGIC_OP SWAP DUP PUSH LOGIC_OP ARITHMETIC_OP LOGIC_OP SWAP SSTORE POP JUMP JUMPDEST PUSH1 PUSH4 CONSTANT1 COMPARASION ISZERO SWAP POP SWAP JUMP JUMPDEST PUSH1 CONSTANT1 SWAP POP DUP PUSH3 DUP ARITHMETIC_OP EQ ISZERO PUSH2 JUMPI CONSTANT2 PUSH1 PUSH1 PUSH2 ARITHMETIC_OP DUP SLOAD DUP PUSH ARITHMETIC_OP LOGIC_OP LOGIC_OP SWAP DUP PUSH LOGIC_OP ARITHMETIC_OP LOGIC_OP SWAP SSTORE POP JUMPDEST POP POP JUMP JUMPDEST PUSH1 CONSTANT1 SWAP POP DUP PUSH3 DUP ARITHMETIC_OP EQ ISZERO PUSH2 JUMPI CONSTANT2 PUSH1 PUSH1 PUSH2 ARITHMETIC_OP DUP SLOAD DUP PUSH ARITHMETIC_OP LOGIC_OP LOGIC_OP SWAP DUP PUSH LOGIC_OP ARITHMETIC_OP LOGIC_OP SWAP SSTORE POP JUMPDEST POP POP JUMP JUMPDEST PUSH1 PUSH4 CONSTANT1 COMPARASION ISZERO SWAP POP SWAP JUMP JUMPDEST PUSH1 CONSTANT1 SWAP POP DUP PUSH3 DUP ARITHMETIC_OP EQ ISZERO PUSH2 JUMPI CONSTANT2 PUSH1 PUSH1 PUSH2 ARITHMETIC_OP DUP SLOAD DUP PUSH ARITHMETIC_OP LOGIC_OP LOGIC_OP SWAP DUP PUSH LOGIC_OP ARITHMETIC_OP LOGIC_OP SWAP SSTORE POP JUMPDEST POP POP JUMP JUMPDEST PUSH1 PUSH1 SWAP SLOAD SWAP PUSH2 ARITHMETIC_OP SWAP ARITHMETIC_OP PUSH LOGIC_OP DUP JUMP JUMPDEST PUSH1 DUP SLOAD PUSH1 DUP PUSH1 LOGIC_OP ISZERO PUSH2 ARITHMETIC_OP ARITHMETIC_OP LOGIC_OP PUSH1 SWAP ARITHMETIC_OP DUP PUSH1 ARITHMETIC_OP PUSH1 DUP SWAP ARITHMETIC_OP ARITHMETIC_OP PUSH1 ARITHMETIC_OP PUSH1 MLOAD SWAP DUP ARITHMETIC_OP PUSH1 MSTORE DUP SWAP SWAP SWAP DUP DUP MSTORE PUSH1 ARITHMETIC_OP DUP DUP SLOAD PUSH1 DUP PUSH1 LOGIC_OP ISZERO PUSH2 ARITHMETIC_OP ARITHMETIC_OP LOGIC_OP PUSH1 SWAP ARITHMETIC_OP DUP ISZERO PUSH2 JUMPI DUP PUSH1 COMPARASION PUSH2 JUMPI PUSH2 DUP DUP SLOAD ARITHMETIC_OP ARITHMETIC_OP DUP MSTORE SWAP PUSH1 ARITHMETIC_OP SWAP PUSH2 JUMP JUMPDEST DUP ARITHMETIC_OP SWAP SWAP PUSH1 MSTORE PUSH1 PUSH1 SHA3 SWAP JUMPDEST DUP SLOAD DUP MSTORE SWAP PUSH1 ARITHMETIC_OP SWAP PUSH1 ARITHMETIC_OP DUP DUP COMPARASION PUSH2 JUMPI DUP SWAP ARITHMETIC_OP PUSH1 LOGIC_OP DUP ARITHMETIC_OP SWAP JUMPDEST POP POP POP POP POP DUP JUMP JUMPDEST CONSTANT1 PUSH3 DUP ARITHMETIC_OP EQ ISZERO PUSH2 JUMPI CONSTANT2 PUSH1 PUSH1 PUSH2 ARITHMETIC_OP DUP SLOAD DUP PUSH ARITHMETIC_OP LOGIC_OP LOGIC_OP SWAP DUP PUSH LOGIC_OP ARITHMETIC_OP LOGIC_OP SWAP SSTORE POP JUMPDEST POP JUMP JUMPDEST PUSH1 CONSTANT1 SWAP POP DUP PUSH3 DUP ARITHMETIC_OP EQ ISZERO PUSH2 JUMPI CONSTANT2 PUSH1 PUSH1 PUSH2 ARITHMETIC_OP DUP SLOAD DUP PUSH ARITHMETIC_OP LOGIC_OP LOGIC_OP SWAP DUP PUSH LOGIC_OP ARITHMETIC_OP LOGIC_OP SWAP SSTORE POP JUMPDEST POP POP JUMP JUMPDEST CONSTANT1 PUSH3 DUP ARITHMETIC_OP EQ ISZERO PUSH2 JUMPI CONSTANT2 PUSH1 PUSH1 PUSH2 ARITHMETIC_OP DUP SLOAD DUP PUSH ARITHMETIC_OP LOGIC_OP LOGIC_OP SWAP DUP PUSH LOGIC_OP ARITHMETIC_OP LOGIC_OP SWAP SSTORE POP JUMPDEST POP JUMP JUMPDEST PUSH1 CON"]

In [ ]:
Get_vuln(opcode_timee)

(1, 39)
1/1 [==============================] - 0s 36ms/step
[[0.53928834 0.15792888 0.68605095 0.724651   0.6304774 ]] time_manipulation


In [ ]:
opcode_reentrancy = ["PUSH1 PUSH1 MSTORE PUSH1 CALLDATASIZE COMPARASION PUSH2 JUMPI PUSH1 CALLDATALOAD PUSH SWAP ARITHMETIC_OP DUP PUSH4 EQ PUSH2 JUMPI DUP PUSH4 EQ PUSH2 JUMPI DUP PUSH4 EQ PUSH2 JUMPI DUP PUSH4 EQ PUSH2 JUMPI DUP PUSH4 EQ PUSH2 JUMPI DUP PUSH4 EQ PUSH2 JUMPI DUP PUSH4 EQ PUSH2 JUMPI DUP PUSH4 EQ PUSH2 JUMPI DUP PUSH4 EQ PUSH2 JUMPI DUP PUSH4 EQ PUSH2 JUMPI DUP PUSH4 EQ PUSH2 JUMPI DUP PUSH4 EQ PUSH2 JUMPI DUP PUSH4 EQ PUSH2 JUMPI DUP PUSH4 EQ PUSH2 JUMPI DUP PUSH4 EQ PUSH2 JUMPI DUP PUSH4 EQ PUSH2 JUMPI DUP PUSH4 EQ PUSH2 JUMPI DUP PUSH4 EQ PUSH2 JUMPI DUP PUSH4 EQ PUSH2 JUMPI DUP PUSH4 EQ PUSH2 JUMPI DUP PUSH4 EQ PUSH2 JUMPI DUP PUSH4 EQ PUSH2 JUMPI JUMPDEST PUSH1 DUP REVERT JUMPDEST CALLVALUE DUP ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST POP PUSH2 PUSH1 DUP CALLDATASIZE ARITHMETIC_OP PUSH1 DUP COMPARASION ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST DUP ARITHMETIC_OP SWAP DUP DUP CALLDATALOAD PUSH LOGIC_OP SWAP PUSH1 ARITHMETIC_OP SWAP SWAP SWAP SWAP DUP CALLDATALOAD SWAP PUSH1 ARITHMETIC_OP SWAP SWAP SWAP SWAP POP POP POP PUSH2 JUMP JUMPDEST PUSH1 MLOAD DUP DUP ISZERO ISZERO ISZERO ISZERO DUP MSTORE PUSH1 ARITHMETIC_OP SWAP POP POP PUSH1 MLOAD DUP SWAP ARITHMETIC_OP SWAP RETURN JUMPDEST CALLVALUE DUP ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST POP PUSH2 PUSH2 JUMP JUMPDEST STOP JUMPDEST CALLVALUE DUP ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST POP PUSH2 PUSH2 JUMP JUMPDEST PUSH1 MLOAD DUP DUP DUP MSTORE PUSH1 ARITHMETIC_OP SWAP POP POP PUSH1 MLOAD DUP SWAP ARITHMETIC_OP SWAP RETURN JUMPDEST CALLVALUE DUP ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST POP PUSH2 PUSH2 JUMP JUMPDEST STOP JUMPDEST CALLVALUE DUP ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST POP PUSH2 PUSH1 DUP CALLDATASIZE ARITHMETIC_OP PUSH1 DUP COMPARASION ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST DUP ARITHMETIC_OP SWAP DUP DUP CALLDATALOAD PUSH LOGIC_OP SWAP PUSH1 ARITHMETIC_OP SWAP SWAP SWAP SWAP DUP CALLDATALOAD PUSH LOGIC_OP SWAP PUSH1 ARITHMETIC_OP SWAP SWAP SWAP SWAP DUP CALLDATALOAD SWAP PUSH1 ARITHMETIC_OP SWAP SWAP SWAP SWAP POP POP POP PUSH2 JUMP JUMPDEST PUSH1 MLOAD DUP DUP ISZERO ISZERO ISZERO ISZERO DUP MSTORE PUSH1 ARITHMETIC_OP SWAP POP POP PUSH1 MLOAD DUP SWAP ARITHMETIC_OP SWAP RETURN JUMPDEST CALLVALUE DUP ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST POP PUSH2 PUSH1 DUP CALLDATASIZE ARITHMETIC_OP PUSH1 DUP COMPARASION ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST DUP ARITHMETIC_OP SWAP DUP DUP CALLDATALOAD SWAP PUSH1 ARITHMETIC_OP SWAP SWAP SWAP SWAP POP POP POP PUSH2 JUMP JUMPDEST STOP JUMPDEST CALLVALUE DUP ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST POP PUSH2 PUSH1 DUP CALLDATASIZE ARITHMETIC_OP PUSH1 DUP COMPARASION ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST DUP ARITHMETIC_OP SWAP DUP DUP CALLDATALOAD SWAP PUSH1 ARITHMETIC_OP SWAP SWAP SWAP SWAP POP POP POP PUSH2 JUMP JUMPDEST STOP JUMPDEST CALLVALUE DUP ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST POP PUSH2 PUSH2 JUMP JUMPDEST STOP JUMPDEST CALLVALUE DUP ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST POP PUSH2 PUSH2 JUMP JUMPDEST STOP JUMPDEST CALLVALUE DUP ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST POP PUSH2 PUSH2 JUMP JUMPDEST STOP JUMPDEST CALLVALUE DUP ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST POP PUSH2 PUSH1 DUP CALLDATASIZE ARITHMETIC_OP PUSH1 DUP COMPARASION ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST DUP ARITHMETIC_OP SWAP DUP DUP CALLDATALOAD PUSH LOGIC_OP SWAP PUSH1 ARITHMETIC_OP SWAP SWAP SWAP SWAP POP POP POP PUSH2 JUMP JUMPDEST PUSH1 MLOAD DUP DUP DUP MSTORE PUSH1 ARITHMETIC_OP SWAP POP POP PUSH1 MLOAD DUP SWAP ARITHMETIC_OP SWAP RETURN JUMPDEST CALLVALUE DUP ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST POP PUSH2 PUSH2 JUMP JUMPDEST STOP JUMPDEST CALLVALUE DUP ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST POP PUSH2 PUSH2 JUMP JUMPDEST STOP JUMPDEST CALLVALUE DUP ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST POP PUSH2 PUSH2 JUMP JUMPDEST STOP JUMPDEST CALLVALUE DUP ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST POP PUSH2 PUSH2 JUMP JUMPDEST STOP JUMPDEST CALLVALUE DUP ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST POP PUSH2 PUSH2 JUMP JUMPDEST STOP JUMPDEST CALLVALUE DUP ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST POP PUSH2 PUSH1 DUP CALLDATASIZE ARITHMETIC_OP PUSH1 DUP COMPARASION ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST DUP ARITHMETIC_OP SWAP DUP DUP CALLDATALOAD SWAP PUSH1 ARITHMETIC_OP SWAP SWAP SWAP SWAP POP POP POP PUSH2 JUMP JUMPDEST STOP JUMPDEST CALLVALUE DUP ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST POP PUSH2 PUSH1 DUP CALLDATASIZE ARITHMETIC_OP PUSH1 DUP COMPARASION ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST DUP ARITHMETIC_OP SWAP DUP DUP CALLDATALOAD PUSH LOGIC_OP SWAP PUSH1 ARITHMETIC_OP SWAP SWAP SWAP SWAP DUP CALLDATALOAD SWAP PUSH1 ARITHMETIC_OP SWAP SWAP SWAP SWAP POP POP POP PUSH2 JUMP JUMPDEST PUSH1 MLOAD DUP DUP ISZERO ISZERO ISZERO ISZERO DUP MSTORE PUSH1 ARITHMETIC_OP SWAP POP POP PUSH1 MLOAD DUP SWAP ARITHMETIC_OP SWAP RETURN JUMPDEST CALLVALUE DUP ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST POP PUSH2 PUSH2 JUMP JUMPDEST STOP JUMPDEST CALLVALUE DUP ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST POP PUSH2 PUSH1 DUP CALLDATASIZE ARITHMETIC_OP PUSH1 DUP COMPARASION ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST DUP ARITHMETIC_OP SWAP DUP DUP CALLDATALOAD PUSH LOGIC_OP SWAP PUSH1 ARITHMETIC_OP SWAP SWAP SWAP SWAP DUP CALLDATALOAD PUSH LOGIC_OP SWAP PUSH1 ARITHMETIC_OP SWAP SWAP SWAP SWAP POP POP POP PUSH2 JUMP JUMPDEST PUSH1 MLOAD DUP DUP DUP MSTORE PUSH1 ARITHMETIC_OP SWAP POP POP PUSH1 MLOAD DUP SWAP ARITHMETIC_OP SWAP RETURN JUMPDEST CALLVALUE DUP ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST POP PUSH2 PUSH2 JUMP JUMPDEST STOP JUMPDEST CALLVALUE DUP ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST POP PUSH2 PUSH2 JUMP JUMPDEST STOP JUMPDEST PUSH1 PUSH2 CONSTANT2 DUP DUP PUSH2 JUMP JUMPDEST PUSH1 SWAP POP SWAP SWAP POP POP JUMP JUMPDEST PUSH1 PUSH1 SLOAD COMPARASION ISZERO ISZERO ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST CONSTANT2 PUSH LOGIC_OP PUSH2 PUSH SWAP DUP ISZERO ARITHMETIC_OP SWAP PUSH1 MLOAD PUSH1 PUSH1 MLOAD DUP DUP ARITHMETIC_OP DUP DUP DUP DUP CALL SWAP POP POP POP POP ISZERO ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST PUSH1 PUSH1 PUSH1 DUP DUP SLOAD ARITHMETIC_OP SWAP POP POP DUP SWAP SSTORE POP JUMP JUMPDEST PUSH1 PUSH1 SLOAD SWAP POP SWAP JUMP JUMPDEST PUSH1 PUSH1 SLOAD COMPARASION ISZERO ISZERO ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST CONSTANT2 PUSH LOGIC_OP PUSH2 PUSH SWAP DUP ISZERO ARITHMETIC_OP SWAP PUSH1 MLOAD PUSH1 PUSH1 MLOAD DUP DUP ARITHMETIC_OP DUP DUP DUP DUP CALL SWAP POP POP POP POP ISZERO ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST PUSH1 PUSH1 PUSH1 DUP DUP SLOAD ARITHMETIC_OP SWAP POP POP DUP SWAP SSTORE POP JUMP JUMPDEST PUSH1 PUSH2 DUP DUP DUP PUSH2 JUMP JUMPDEST PUSH2 DUP CONSTANT2 PUSH2 DUP PUSH1 PUSH1 DUP PUSH LOGIC_OP PUSH LOGIC_OP DUP MSTORE PUSH1 ARITHMETIC_OP SWAP DUP MSTORE PUSH1 ARITHMETIC_OP PUSH1 SHA3 PUSH1 CONSTANT2 PUSH LOGIC_OP PUSH LOGIC_OP DUP MSTORE PUSH1 ARITHMETIC_OP SWAP DUP MSTORE PUSH1 ARITHMETIC_OP PUSH1 SHA3 SLOAD PUSH2 SWAP SWAP SWAP PUSH4 LOGIC_OP JUMP JUMPDEST PUSH2 JUMP JUMPDEST PUSH1 SWAP POP SWAP SWAP POP POP POP JUMP JUMPDEST DUP PUSH1 PUSH1 CONSTANT2 PUSH LOGIC_OP PUSH LOGIC_OP DUP MSTORE PUSH1 ARITHMETIC_OP SWAP DUP MSTORE PUSH1 ARITHMETIC_OP PUSH1 SHA3 SLOAD COMPARASION ISZERO ISZERO ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST CONSTANT2 PUSH LOGIC_OP PUSH2 DUP SWAP DUP ISZERO ARITHMETIC_OP SWAP PUSH1 MLOAD PUSH1 PUSH1 MLOAD DUP DUP ARITHMETIC_OP DUP DUP DUP DUP CALL SWAP POP POP POP POP ISZERO ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST DUP PUSH1 PUSH1 CONSTANT2 PUSH LOGIC_OP PUSH LOGIC_OP DUP MSTORE PUSH1 ARITHMETIC_OP SWAP DUP MSTORE PUSH1 ARITHMETIC_OP PUSH1 SHA3 PUSH1 DUP DUP SLOAD ARITHMETIC_OP SWAP POP POP DUP SWAP SSTORE POP POP JUMP JUMPDEST PUSH2 CONSTANT2 DUP PUSH2 JUMP JUMPDEST POP JUMP JUMPDEST CONSTANT2 PUSH LOGIC_OP PUSH2 PUSH1 PUSH1 CONSTANT2 PUSH LOGIC_OP PUSH LOGIC_OP DUP MSTORE PUSH1 ARITHMETIC_OP SWAP DUP MSTORE PUSH1 ARITHMETIC_OP PUSH1 SHA3 SLOAD SWAP DUP ISZERO ARITHMETIC_OP SWAP PUSH1 MLOAD PUSH1 PUSH1 MLOAD DUP DUP ARITHMETIC_OP DUP DUP DUP DUP CALL SWAP POP POP POP POP ISZERO ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST PUSH1 PUSH1 PUSH1 CONSTANT2 PUSH LOGIC_OP PUSH LOGIC_OP DUP MSTORE PUSH1 ARITHMETIC_OP SWAP DUP MSTORE PUSH1 ARITHMETIC_OP PUSH1 SHA3 DUP SWAP SSTORE POP JUMP JUMPDEST PUSH1 PUSH1 SWAP SLOAD SWAP PUSH2 ARITHMETIC_OP SWAP ARITHMETIC_OP PUSH LOGIC_OP PUSH LOGIC_OP PUSH2 PUSH1 SLOAD SWAP DUP ISZERO ARITHMETIC_OP SWAP PUSH1 MLOAD PUSH1 PUSH1 MLOAD DUP DUP ARITHMETIC_OP DUP DUP DUP DUP CALL SWAP POP POP POP POP ISZERO ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST CONSTANT2 PUSH1 PUSH1 PUSH2 ARITHMETIC_OP DUP SLOAD DUP PUSH ARITHMETIC_OP LOGIC_OP LOGIC_OP SWAP DUP PUSH LOGIC_OP ARITHMETIC_OP LOGIC_OP SWAP SSTORE POP CONSTANT2 PUSH LOGIC_OP BALANCE PUSH1 DUP SWAP SSTORE POP JUMP JUMPDEST CONSTANT2 PUSH LOGIC_OP PUSH2 PUSH1 PUSH1 CONSTANT2 PUSH LOGIC_OP PUSH LOGIC_OP DUP MSTORE PUSH1 ARITHMETIC_OP SWAP DUP MSTORE PUSH1 ARITHMETIC_OP PUSH1 SHA3 SLOAD SWAP DUP ISZERO ARITHMETIC_OP SWAP PUSH1 MLOAD PUSH1 PUSH1 MLOAD DUP DUP ARITHMETIC_OP DUP DUP DUP DUP CALL SWAP POP POP POP POP ISZERO PUSH2 JUMPI PUSH1 PUSH1 PUSH1 CONSTANT2 PUSH LOGIC_OP PUSH LOGIC_OP DUP MSTORE PUSH1 ARITHMETIC_OP SWAP DUP MSTORE PUSH1 ARITHMETIC_OP PUSH1 SHA3 DUP SWAP SSTORE POP JUMPDEST JUMP JUMPDEST PUSH1 PUSH1 PUSH1 DUP PUSH LOGIC_OP PUSH LOGIC_OP DUP MSTORE PUSH1 ARITHMETIC_OP SWAP DUP MSTORE PUSH1 ARITHMETIC_OP PUSH1 SHA3 SLOAD SWAP POP SWAP SWAP POP JUMP JUMPDEST PUSH1 PUSH1 PUSH1 CONSTANT2 PUSH LOGIC_OP PUSH LOGIC_OP DUP MSTORE PUSH1 ARITHMETIC_OP SWAP DUP MSTORE PUSH1 ARITHMETIC_OP PUSH1 SHA3 SLOAD COMPARASION ISZERO ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST PUSH1 PUSH1 PUSH1 CONSTANT2 PUSH LOGIC_OP PUSH LOGIC_OP DUP MSTORE PUSH1 ARITHMETIC_OP SWAP DUP MSTORE PUSH1 ARITHMETIC_OP PUSH1 SHA3 SLOAD SWAP POP CONSTANT2 PUSH LOGIC_OP PUSH2 DUP SWAP DUP ISZERO ARITHMETIC_OP SWAP PUSH1 MLOAD PUSH1 PUSH1 MLOAD DUP DUP ARITHMETIC_OP DUP DUP DUP DUP CALL SWAP POP POP POP POP ISZERO DUP ISZERO PUSH2 JUMPI RETURNDATASIZE PUSH1 DUP RETURNDATACOPY RETURNDATASIZE PUSH1 REVERT JUMPDEST POP PUSH1 PUSH1 PUSH1 CONSTANT2 PUSH LOGIC_OP PUSH LOGIC_OP DUP MSTORE PUSH1 ARITHMETIC_OP SWAP DUP MSTORE PUSH1 ARITHMETIC_OP PUSH1 SHA3 DUP SWAP SSTORE POP POP JUMP JUMPDEST PUSH1 PUSH1 PUSH1 CONSTANT2 PUSH LOGIC_OP PUSH LOGIC_OP DUP MSTORE PUSH1 ARITHMETIC_OP SWAP DUP MSTORE PUSH1 ARITHMETIC_OP PUSH1 SHA3 SLOAD COMPARASION ISZERO ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST PUSH1 PUSH1 PUSH1 CONSTANT2 PUSH LOGIC_OP PUSH LOGIC_OP DUP MSTORE PUSH1 ARITHMETIC_OP SWAP DUP MSTORE PUSH1 ARITHMETIC_OP PUSH1 SHA3 SLOAD SWAP POP CONSTANT2 PUSH LOGIC_OP PUSH2 DUP SWAP DUP ISZERO ARITHMETIC_OP SWAP PUSH1 MLOAD PUSH1 PUSH1 MLOAD DUP DUP ARITHMETIC_OP DUP DUP DUP DUP CALL SWAP POP POP POP POP ISZERO DUP ISZERO PUSH2 JUMPI RETURNDATASIZE PUSH1 DUP RETURNDATACOPY RETURNDATASIZE PUSH1 REVERT JUMPDEST POP PUSH1 PUSH1 PUSH1 CONSTANT2 PUSH LOGIC_OP PUSH LOGIC_OP DUP MSTORE PUSH1 ARITHMETIC_OP SWAP DUP MSTORE PUSH1 ARITHMETIC_OP PUSH1 SHA3 DUP SWAP SSTORE POP POP JUMP JUMPDEST PUSH1 PUSH1 PUSH1 CONSTANT2 PUSH LOGIC_OP PUSH LOGIC_OP DUP MSTORE PUSH1 ARITHMETIC_OP SWAP DUP MSTORE PUSH1 ARITHMETIC_OP PUSH1 SHA3 SLOAD COMPARASION ISZERO ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST PUSH1 PUSH1 PUSH1 CONSTANT2 PUSH LOGIC_OP PUSH LOGIC_OP DUP MSTORE PUSH1 ARITHMETIC_OP SWAP DUP MSTORE PUSH1 ARITHMETIC_OP PUSH1 SHA3 SLOAD SWAP POP CONSTANT2 PUSH LOGIC_OP PUSH2 DUP SWAP DUP ISZERO ARITHMETIC_OP SWAP PUSH1 MLOAD PUSH1 PUSH1 MLOAD DUP DUP ARITHMETIC_OP DUP DUP DUP DUP CALL SWAP POP POP POP POP ISZERO DUP ISZERO PUSH2 JUMPI RETURNDATASIZE PUSH1 DUP RETURNDATACOPY RETURNDATASIZE PUSH1 REVERT JUMPDEST POP PUSH1 PUSH1 PUSH1 CONSTANT2 PUSH LOGIC_OP PUSH LOGIC_OP DUP MSTORE PUSH1 ARITHMETIC_OP SWAP DUP MSTORE PUSH1 ARITHMETIC_OP PUSH1 SHA3 DUP SWAP SSTORE POP POP JUMP JUMPDEST PUSH1 PUSH1 PUSH1 CONSTANT2 PUSH LOGIC_OP PUSH LOGIC_OP DUP MSTORE PUSH1 ARITHMETIC_OP SWAP DUP MSTORE PUSH1 ARITHMETIC_OP PUSH1 SHA3 SLOAD COMPARASION ISZERO ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST PUSH1 PUSH1 PUSH1 CONSTANT2 PUSH LOGIC_OP PUSH LOGIC_OP DUP MSTORE PUSH1 ARITHMETIC_OP SWAP DUP MSTORE PUSH1 ARITHMETIC_OP PUSH1 SHA3 SLOAD SWAP POP CONSTANT2 PUSH LOGIC_OP PUSH2 DUP SWAP DUP ISZERO ARITHMETIC_OP SWAP PUSH1 MLOAD PUSH1 PUSH1 MLOAD DUP DUP ARITHMETIC_OP DUP DUP DUP DUP CALL SWAP POP POP POP POP ISZERO DUP ISZERO PUSH2 JUMPI RETURNDATASIZE PUSH1 DUP RETURNDATACOPY RETURNDATASIZE PUSH1 REVERT JUMPDEST POP PUSH1 PUSH1 PUSH1 CONSTANT2 PUSH LOGIC_OP PUSH LOGIC_OP DUP MSTORE PUSH1 ARITHMETIC_OP SWAP DUP MSTORE PUSH1 ARITHMETIC_OP PUSH1 SHA3 DUP SWAP SSTORE POP POP JUMP JUMPDEST PUSH1 CONSTANT2 PUSH LOGIC_OP PUSH1 PUSH1 CONSTANT2 PUSH LOGIC_OP PUSH LOGIC_OP DUP MSTORE PUSH1 ARITHMETIC_OP SWAP DUP MSTORE PUSH1 ARITHMETIC_OP PUSH1 SHA3 SLOAD PUSH1 MLOAD DUP PUSH1 ARITHMETIC_OP SWAP POP PUSH1 PUSH1 MLOAD DUP DUP ARITHMETIC_OP DUP DUP DUP GAS CALL SWAP POP POP POP RETURNDATASIZE DUP PUSH1 DUP EQ PUSH2 JUMPI PUSH1 MLOAD SWAP POP PUSH1 LOGIC_OP PUSH1 RETURNDATASIZE ARITHMETIC_OP LOGIC_OP DUP ARITHMETIC_OP PUSH1 MSTORE RETURNDATASIZE DUP MSTORE RETURNDATASIZE PUSH1 PUSH1 DUP ARITHMETIC_OP RETURNDATACOPY PUSH2 JUMP JUMPDEST PUSH1 SWAP POP JUMPDEST POP POP SWAP POP DUP ISZERO PUSH2 JUMPI PUSH1 PUSH1 PUSH1 CONSTANT2 PUSH LOGIC_OP PUSH LOGIC_OP DUP MSTORE PUSH1 ARITHMETIC_OP SWAP DUP MSTORE PUSH1 ARITHMETIC_OP PUSH1 SHA3 DUP SWAP SSTORE POP JUMPDEST POP JUMP JUMPDEST DUP PUSH1 DUP CONSTANT2 PUSH LOGIC_OP PUSH LOGIC_OP DUP MSTORE PUSH1 ARITHMETIC_OP SWAP DUP MSTORE PUSH1 ARITHMETIC_OP PUSH1 SHA3 SLOAD COMPARASION ISZERO ISZERO ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST PUSH1 CONSTANT2 PUSH LOGIC_OP DUP PUSH1 MLOAD DUP PUSH1 ARITHMETIC_OP SWAP POP PUSH1 PUSH1 MLOAD DUP DUP ARITHMETIC_OP DUP DUP DUP GAS CALL SWAP POP POP POP RETURNDATASIZE DUP PUSH1 DUP EQ PUSH2 JUMPI PUSH1 MLOAD SWAP POP PUSH1 LOGIC_OP PUSH1 RETURNDATASIZE ARITHMETIC_OP LOGIC_OP DUP ARITHMETIC_OP PUSH1 MSTORE RETURNDATASIZE DUP MSTORE RETURNDATASIZE PUSH1 PUSH1 DUP ARITHMETIC_OP RETURNDATACOPY PUSH2 JUMP JUMPDEST PUSH1 SWAP POP JUMPDEST POP POP SWAP POP DUP ISZERO ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST DUP PUSH1 DUP CONSTANT2 PUSH LOGIC_OP PUSH LOGIC_OP DUP MSTORE PUSH1 ARITHMETIC_OP SWAP DUP MSTORE PUSH1 ARITHMETIC_OP PUSH1 SHA3 PUSH1 DUP DUP SLOAD ARITHMETIC_OP SWAP POP POP DUP SWAP SSTORE POP POP POP JUMP JUMPDEST PUSH1 PUSH2 CONSTANT2 DUP DUP PUSH2 JUMP JUMPDEST PUSH1 SWAP POP SWAP SWAP POP POP JUMP JUMPDEST PUSH1 PUSH1 PUSH1 SWAP SLOAD SWAP PUSH2 ARITHMETIC_OP SWAP ARITHMETIC_OP PUSH LOGIC_OP PUSH LOGIC_OP PUSH1 SLOAD PUSH1 MLOAD DUP PUSH1 ARITHMETIC_OP SWAP POP PUSH1 PUSH1 MLOAD DUP DUP ARITHMETIC_OP DUP DUP DUP GAS CALL SWAP POP POP POP RETURNDATASIZE DUP PUSH1 DUP EQ PUSH2 JUMPI PUSH1 MLOAD SWAP POP PUSH1 LOGIC_OP PUSH1 RETURNDATASIZE ARITHMETIC_OP LOGIC_OP DUP ARITHMETIC_OP PUSH1 MSTORE RETURNDATASIZE DUP MSTORE RETURNDATASIZE PUSH1 PUSH1 DUP ARITHMETIC_OP RETURNDATACOPY PUSH2 JUMP JUMPDEST PUSH1 SWAP POP JUMPDEST POP POP SWAP POP DUP ISZERO ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST CONSTANT2 PUSH1 PUSH1 PUSH2 ARITHMETIC_OP DUP SLOAD DUP PUSH ARITHMETIC_OP LOGIC_OP LOGIC_OP SWAP DUP PUSH LOGIC_OP ARITHMETIC_OP LOGIC_OP SWAP SSTORE POP CONSTANT2 PUSH LOGIC_OP BALANCE PUSH1 DUP SWAP SSTORE POP POP JUMP JUMPDEST PUSH1 PUSH1 PUSH1 DUP PUSH LOGIC_OP PUSH LOGIC_OP DUP MSTORE PUSH1 ARITHMETIC_OP SWAP DUP MSTORE PUSH1 ARITHMETIC_OP PUSH1 SHA3 PUSH1 DUP PUSH LOGIC_OP PUSH LOGIC_OP DUP MSTORE PUSH1 ARITHMETIC_OP SWAP DUP MSTORE PUSH1 ARITHMETIC_OP PUSH1 SHA3 SLOAD SWAP POP SWAP SWAP POP POP JUMP JUMPDEST PUSH1 PUSH1 SWAP SLOAD SWAP PUSH2 ARITHMETIC_OP SWAP ARITHMETIC_OP PUSH LOGIC_OP PUSH LOGIC_OP PUSH2 PUSH1 SLOAD SWAP DUP ISZERO ARITHMETIC_OP SWAP PUSH1 MLOAD PUSH1 PUSH1 MLOAD DUP DUP ARITHMETIC_OP DUP DUP DUP DUP CALL SWAP POP POP POP POP ISZERO ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST CONSTANT2 PUSH1 PUSH1 PUSH2 ARITHMETIC_OP DUP SLOAD DUP PUSH ARITHMETIC_OP LOGIC_OP LOGIC_OP SWAP DUP PUSH LOGIC_OP ARITHMETIC_OP LOGIC_OP SWAP SSTORE POP CONSTANT2 PUSH LOGIC_OP BALANCE PUSH1 DUP SWAP SSTORE POP JUMP JUMPDEST PUSH1 PUSH1 SLOAD COMPARASION ISZERO ISZERO ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST CONSTANT2 PUSH LOGIC_OP PUSH2 PUSH SWAP DUP ISZERO ARITHMETIC_OP SWAP PUSH1 MLOAD PUSH1 PUSH1 MLOAD DUP DUP ARITHMETIC_OP DUP DUP DUP DUP CALL SWAP POP POP POP POP ISZERO ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST PUSH1 PUSH1 PUSH1 DUP DUP SLOAD ARITHMETIC_OP SWAP POP POP DUP SWAP SSTORE POP JUMP JUMPDEST PUSH1 PUSH LOGIC_OP DUP PUSH LOGIC_OP EQ ISZERO ISZERO ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST PUSH1 PUSH LOGIC_OP DUP PUSH LOGIC_OP EQ ISZERO ISZERO ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST DUP PUSH1 PUSH1 DUP PUSH LOGIC_OP PUSH LOGIC_OP DUP MSTORE PUSH1 ARITHMETIC_OP SWAP DUP MSTORE PUSH1 ARITHMETIC_OP PUSH1 SHA3 PUSH1 DUP PUSH LOGIC_OP PUSH LOGIC_OP DUP MSTORE PUSH1 ARITHMETIC_OP SWAP DUP MSTORE PUSH1 ARITHMETIC_OP PUSH1 SHA3 DUP SWAP SSTORE POP DUP PUSH LOGIC_OP DUP PUSH LOGIC_OP PUSH DUP PUSH1 MLOAD DUP DUP DUP MSTORE PUSH1 ARITHMETIC_OP SWAP POP POP PUSH1 MLOAD DUP SWAP ARITHMETIC_OP SWAP LOG POP POP POP JUMP JUMPDEST PUSH1 PUSH LOGIC_OP DUP PUSH LOGIC_OP EQ ISZERO ISZERO ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST PUSH2 DUP PUSH1 PUSH1 DUP PUSH LOGIC_OP PUSH LOGIC_OP DUP MSTORE PUSH1 ARITHMETIC_OP SWAP DUP MSTORE PUSH1 ARITHMETIC_OP PUSH1 SHA3 SLOAD PUSH2 SWAP SWAP SWAP PUSH4 LOGIC_OP JUMP JUMPDEST PUSH1 PUSH1 DUP PUSH LOGIC_OP PUSH LOGIC_OP DUP MSTORE PUSH1 ARITHMETIC_OP SWAP DUP MSTORE PUSH1 ARITHMETIC_OP PUSH1 SHA3 DUP SWAP SSTORE POP PUSH2 DUP PUSH1 PUSH1 DUP PUSH LOGIC_OP PUSH LOGIC_OP DUP MSTORE PUSH1 ARITHMETIC_OP SWAP DUP MSTORE PUSH1 ARITHMETIC_OP PUSH1 SHA3 SLOAD PUSH2 SWAP SWAP SWAP PUSH4 LOGIC_OP JUMP JUMPDEST PUSH1 PUSH1 DUP PUSH LOGIC_OP PUSH LOGIC_OP DUP MSTORE PUSH1 ARITHMETIC_OP SWAP DUP MSTORE PUSH1 ARITHMETIC_OP PUSH1 SHA3 DUP SWAP SSTORE POP DUP PUSH LOGIC_OP DUP PUSH LOGIC_OP PUSH DUP PUSH1 MLOAD DUP DUP DUP MSTORE PUSH1 ARITHMETIC_OP SWAP POP POP PUSH1 MLOAD DUP SWAP ARITHMETIC_OP SWAP LOG POP POP POP JUMP JUMPDEST PUSH1 DUP DUP COMPARASION ISZERO ISZERO ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST PUSH1 DUP DUP ARITHMETIC_OP SWAP POP DUP SWAP POP POP SWAP SWAP POP POP JUMP JUMPDEST PUSH1 PUSH LOGIC_OP DUP PUSH LOGIC_OP EQ ISZERO ISZERO ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST PUSH2 DUP PUSH1 SLOAD PUSH2 SWAP SWAP SWAP PUSH4 LOGIC_OP JUMP JUMPDEST PUSH1 DUP SWAP SSTORE POP PUSH2 DUP PUSH1 PUSH1 DUP PUSH LOGIC_OP PUSH LOGIC_OP DUP MSTORE PUSH1 ARITHMETIC_OP SWAP DUP MSTORE PUSH1 ARITHMETIC_OP PUSH1 SHA3 SLOAD PUSH2 SWAP SWAP SWAP PUSH4 LOGIC_OP JUMP JUMPDEST PUSH1 PUSH1 DUP PUSH LOGIC_OP PUSH LOGIC_OP DUP MSTORE PUSH1 ARITHMETIC_OP SWAP DUP MSTORE PUSH1 ARITHMETIC_OP PUSH1 SHA3 DUP SWAP SSTORE POP PUSH1 PUSH LOGIC_OP DUP PUSH LOGIC_OP PUSH DUP PUSH1 MLOAD DUP DUP DUP MSTORE PUSH1 ARITHMETIC_OP SWAP POP POP PUSH1 MLOAD DUP SWAP ARITHMETIC_OP SWAP LOG POP POP JUMP JUMPDEST PUSH1 DUP DUP DUP ARITHMETIC_OP SWAP POP DUP DUP COMPARASION ISZERO ISZERO ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST DUP SWAP POP POP SWAP SWAP POP POP JUMP INVALID LOG PUSH SHA3 INVALID JUMPDEST LOGIC_OP LOGIC_OP SLOAD MSIZE COMPARASION INVALID JUMPI ARITHMETIC_OP INVALID RETURNDATASIZE INVALID CALLVALUE INVALID INVALID LOGIC_OP INVALID RETURNDATASIZE PUSH INVALID INVALID STOP INVALID PUSH CONSTANT2 EQ PUSH1 PUSH1 MSTORE PUSH1 DUP REVERT INVALID LOG PUSH SHA3 PUSH INVALID PUSH CALLDATALOAD PUSH SWAP ARITHMETIC_OP DUP PUSH4 EQ PUSH2 JUMPI DUP PUSH4 EQ PUSH2 JUMPI DUP PUSH4 EQ PUSH2 JUMPI DUP PUSH4 EQ PUSH2 JUMPI DUP PUSH4 EQ PUSH2 JUMPI DUP PUSH4 EQ PUSH2 JUMPI DUP PUSH4 EQ PUSH2 JUMPI DUP PUSH4 EQ PUSH2 JUMPI DUP PUSH4 EQ PUSH2 JUMPI DUP PUSH4 EQ PUSH2 JUMPI DUP PUSH4 EQ PUSH2 JUMPI DUP PUSH4 EQ PUSH2 JUMPI DUP PUSH4 EQ PUSH2 JUMPI DUP PUSH4 EQ PUSH2 JUMPI DUP PUSH4 EQ PUSH2 JUMPI DUP PUSH4 EQ PUSH2 JUMPI DUP PUSH4 EQ PUSH2 JUMPI DUP PUSH4 EQ PUSH2 JUMPI DUP PUSH4 EQ PUSH2 JUMPI DUP PUSH4 EQ PUSH2 JUMPI DUP PUSH4 EQ PUSH2 JUMPI DUP PUSH4 EQ PUSH2 JUMPI DUP PUSH4 EQ PUSH2 JUMPI DUP PUSH4 EQ PUSH2 JUMPI DUP PUSH4 EQ PUSH2 JUMPI DUP PUSH4 EQ PUSH2 JUMPI DUP PUSH4 EQ PUSH2 JUMPI DUP PUSH4 EQ PUSH2 JUMPI DUP PUSH4 EQ PUSH2 JUMPI DUP PUSH4 EQ PUSH2 JUMPI DUP PUSH4 EQ PUSH2 JUMPI DUP PUSH4 EQ PUSH2 JUMPI DUP PUSH4 EQ PUSH2 JUMPI DUP PUSH4 EQ PUSH2 JUMPI JUMPDEST PUSH1 DUP REVERT JUMPDEST CALLVALUE DUP ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST POP PUSH2 PUSH2 JUMP JUMPDEST PUSH1 MLOAD DUP DUP PUSH1 ARITHMETIC_OP DUP DUP ARITHMETIC_OP DUP MSTORE DUP DUP DUP MLOAD DUP MSTORE PUSH1 ARITHMETIC_OP SWAP POP DUP MLOAD SWAP PUSH1 ARITHMETIC_OP SWAP DUP DUP DUP PUSH1 JUMPDEST DUP DUP COMPARASION ISZERO PUSH2 JUMPI DUP DUP ARITHMETIC_OP MLOAD DUP DUP ARITHMETIC_OP MSTORE PUSH1 DUP ARITHMETIC_OP SWAP POP PUSH2 JUMP JUMPDEST POP POP POP POP SWAP POP SWAP DUP ARITHMETIC_OP SWAP PUSH1 LOGIC_OP DUP ISZERO PUSH2 JUMPI DUP DUP ARITHMETIC_OP DUP MLOAD PUSH1 DUP PUSH1 ARITHMETIC_OP PUSH2 ARITHMETIC_OP ARITHMETIC_OP LOGIC_OP LOGIC_OP DUP MSTORE PUSH1 ARITHMETIC_OP SWAP POP JUMPDEST POP SWAP POP POP POP PUSH1 MLOAD DUP SWAP ARITHMETIC_OP SWAP RETURN JUMPDEST CALLVALUE DUP ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST POP PUSH2 PUSH1 DUP CALLDATASIZE ARITHMETIC_OP PUSH1 DUP COMPARASION ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST DUP ARITHMETIC_OP SWAP DUP DUP CALLDATALOAD PUSH LOGIC_OP SWAP PUSH1 ARITHMETIC_OP SWAP SWAP SWAP SWAP DUP CALLDATALOAD SWAP PUSH1 ARITHMETIC_OP SWAP SWAP SWAP SWAP POP POP POP PUSH2 JUMP JUMPDEST PUSH1 MLOAD DUP DUP ISZERO ISZERO ISZERO ISZERO DUP MSTORE PUSH1 ARITHMETIC_OP SWAP POP POP PUSH1 MLOAD DUP SWAP ARITHMETIC_OP SWAP RETURN JUMPDEST CALLVALUE DUP ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST POP PUSH2 PUSH2 JUMP JUMPDEST STOP JUMPDEST CALLVALUE DUP ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST POP PUSH2 PUSH2 JUMP JUMPDEST PUSH1 MLOAD DUP DUP DUP MSTORE PUSH1 ARITHMETIC_OP SWAP POP POP PUSH1 MLOAD DUP SWAP ARITHMETIC_OP SWAP RETURN JUMPDEST CALLVALUE DUP ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST POP PUSH2 PUSH2 JUMP JUMPDEST STOP JUMPDEST CALLVALUE DUP ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST POP PUSH2 PUSH1 DUP CALLDATASIZE ARITHMETIC_OP PUSH1 DUP COMPARASION ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST DUP ARITHMETIC_OP SWAP DUP DUP CALLDATALOAD PUSH LOGIC_OP SWAP PUSH1 ARITHMETIC_OP SWAP SWAP SWAP SWAP DUP CALLDATALOAD PUSH LOGIC_OP SWAP PUSH1 ARITHMETIC_OP SWAP SWAP SWAP SWAP DUP CALLDATALOAD SWAP PUSH1 ARITHMETIC_OP SWAP SWAP SWAP SWAP POP POP POP PUSH2 JUMP JUMPDEST PUSH1 MLOAD DUP DUP ISZERO ISZERO ISZERO ISZERO DUP MSTORE PUSH1 ARITHMETIC_OP SWAP POP POP PUSH1 MLOAD DUP SWAP ARITHMETIC_OP SWAP RETURN JUMPDEST CALLVALUE DUP ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST POP PUSH2 PUSH1 DUP CALLDATASIZE ARITHMETIC_OP PUSH1 DUP COMPARASION ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST DUP ARITHMETIC_OP SWAP DUP DUP CALLDATALOAD SWAP PUSH1 ARITHMETIC_OP SWAP SWAP SWAP SWAP POP POP POP PUSH2 JUMP JUMPDEST STOP JUMPDEST CALLVALUE DUP ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST POP PUSH2 PUSH2 JUMP JUMPDEST PUSH1 MLOAD DUP DUP PUSH1 LOGIC_OP PUSH1 LOGIC_OP DUP MSTORE PUSH1 ARITHMETIC_OP SWAP POP POP PUSH1 MLOAD DUP SWAP ARITHMETIC_OP SWAP RETURN JUMPDEST CALLVALUE DUP ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST POP PUSH2 PUSH2 JUMP JUMPDEST PUSH1 MLOAD DUP DUP DUP MSTORE PUSH1 ARITHMETIC_OP SWAP POP POP PUSH1 MLOAD DUP SWAP ARITHMETIC_OP SWAP RETURN JUMPDEST CALLVALUE DUP ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST POP PUSH2 PUSH2 JUMP JUMPDEST PUSH1 MLOAD DUP DUP PUSH1 LOGIC_OP PUSH1 LOGIC_OP DUP MSTORE PUSH1 ARITHMETIC_OP SWAP POP POP PUSH1 MLOAD DUP SWAP ARITHMETIC_OP SWAP RETURN JUMPDEST CALLVALUE DUP ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST POP PUSH2 PUSH1 DUP CALLDATASIZE ARITHMETIC_OP PUSH1 DUP COMPARASION ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST DUP ARITHMETIC_OP SWAP DUP DUP CALLDATALOAD SWAP PUSH1 ARITHMETIC_OP SWAP SWAP SWAP SWAP POP POP POP PUSH2 JUMP JUMPDEST STOP JUMPDEST CALLVALUE DUP ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST POP PUSH2 PUSH2 JUMP JUMPDEST STOP JUMPDEST CALLVALUE DUP ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST POP PUSH2 PUSH2 JUMP JUMPDEST STOP JUMPDEST CALLVALUE DUP ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST POP PUSH2 PUSH2 JUMP JUMPDEST STOP JUMPDEST CALLVALUE DUP ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST POP PUSH2 PUSH2 JUMP JUMPDEST STOP JUMPDEST CALLVALUE DUP ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST POP PUSH2 PUSH1 DUP CALLDATASIZE ARITHMETIC_OP PUSH1 DUP COMPARASION ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST DUP ARITHMETIC_OP SWAP DUP DUP CALLDATALOAD PUSH LOGIC_OP SWAP PUSH1 ARITHMETIC_OP SWAP SWAP SWAP SWAP POP POP POP PUSH2 JUMP JUMPDEST PUSH1 MLOAD DUP DUP DUP MSTORE PUSH1 ARITHMETIC_OP SWAP POP POP PUSH1 MLOAD DUP SWAP ARITHMETIC_OP SWAP RETURN JUMPDEST CALLVALUE DUP ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST POP PUSH2 PUSH2 JUMP JUMPDEST STOP JUMPDEST CALLVALUE DUP ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST POP PUSH2 PUSH2 JUMP JUMPDEST STOP JUMPDEST CALLVALUE DUP ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST POP PUSH2 PUSH2 JUMP JUMPDEST STOP JUMPDEST CALLVALUE DUP ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST POP PUSH2 PUSH2 JUMP JUMPDEST STOP JUMPDEST CALLVALUE DUP ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST POP PUSH2 PUSH2 JUMP JUMPDEST STOP JUMPDEST CALLVALUE DUP ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST POP PUSH2 PUSH2 JUMP JUMPDEST PUSH1 MLOAD DUP DUP PUSH1 ARITHMETIC_OP DUP DUP ARITHMETIC_OP DUP MSTORE DUP DUP DUP MLOAD DUP MSTORE PUSH1 ARITHMETIC_OP SWAP POP DUP MLOAD SWAP PUSH1 ARITHMETIC_OP SWAP DUP DUP DUP PUSH1 JUMPDEST DUP DUP COMPARASION ISZERO PUSH2 JUMPI DUP DUP ARITHMETIC_OP MLOAD DUP DUP ARITHMETIC_OP MSTORE PUSH1 DUP ARITHMETIC_OP SWAP POP PUSH2 JUMP JUMPDEST POP POP POP POP SWAP POP SWAP DUP ARITHMETIC_OP SWAP PUSH1 LOGIC_OP DUP ISZERO PUSH2 JUMPI DUP DUP ARITHMETIC_OP DUP MLOAD PUSH1 DUP PUSH1 ARITHMETIC_OP PUSH2 ARITHMETIC_OP ARITHMETIC_OP LOGIC_OP LOGIC_OP DUP MSTORE PUSH1 ARITHMETIC_OP SWAP POP JUMPDEST POP SWAP POP POP POP PUSH1 MLOAD DUP SWAP ARITHMETIC_OP SWAP RETURN JUMPDEST CALLVALUE DUP ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST POP PUSH2 PUSH2 JUMP JUMPDEST STOP JUMPDEST CALLVALUE DUP ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST POP PUSH2 PUSH1 DUP CALLDATASIZE ARITHMETIC_OP PUSH1 DUP COMPARASION ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST DUP ARITHMETIC_OP SWAP DUP DUP CALLDATALOAD SWAP PUSH1 ARITHMETIC_OP SWAP SWAP SWAP SWAP POP POP POP PUSH2 JUMP JUMPDEST STOP JUMPDEST CALLVALUE DUP ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST POP PUSH2 PUSH2 JUMP JUMPDEST STOP JUMPDEST CALLVALUE DUP ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST POP PUSH2 PUSH2 JUMP JUMPDEST STOP JUMPDEST CALLVALUE DUP ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST POP PUSH2 PUSH1 DUP CALLDATASIZE ARITHMETIC_OP PUSH1 DUP COMPARASION ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST DUP ARITHMETIC_OP SWAP DUP DUP CALLDATALOAD SWAP PUSH1 ARITHMETIC_OP SWAP SWAP SWAP SWAP POP POP POP PUSH2 JUMP JUMPDEST STOP JUMPDEST CALLVALUE DUP ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST POP PUSH2 PUSH1 DUP CALLDATASIZE ARITHMETIC_OP PUSH1 DUP COMPARASION ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST DUP ARITHMETIC_OP SWAP DUP DUP CALLDATALOAD PUSH LOGIC_OP SWAP PUSH1 ARITHMETIC_OP SWAP SWAP SWAP SWAP DUP CALLDATALOAD SWAP PUSH1 ARITHMETIC_OP SWAP SWAP SWAP SWAP POP POP POP PUSH2 JUMP JUMPDEST PUSH1 MLOAD DUP DUP ISZERO ISZERO ISZERO ISZERO DUP MSTORE PUSH1 ARITHMETIC_OP SWAP POP POP PUSH1 MLOAD DUP SWAP ARITHMETIC_OP SWAP RETURN JUMPDEST CALLVALUE DUP ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST POP PUSH2 PUSH2 JUMP JUMPDEST STOP JUMPDEST CALLVALUE DUP ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST POP PUSH2 PUSH2 JUMP JUMPDEST STOP JUMPDEST CALLVALUE DUP ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST POP PUSH2 PUSH2 JUMP JUMPDEST STOP JUMPDEST CALLVALUE DUP ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST POP PUSH2 PUSH1 DUP CALLDATASIZE ARITHMETIC_OP PUSH1 DUP COMPARASION ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST DUP ARITHMETIC_OP SWAP DUP DUP CALLDATALOAD PUSH LOGIC_OP SWAP PUSH1 ARITHMETIC_OP SWAP SWAP SWAP SWAP DUP CALLDATALOAD PUSH LOGIC_OP SWAP PUSH1 ARITHMETIC_OP SWAP SWAP SWAP SWAP POP POP POP PUSH2 JUMP JUMPDEST PUSH1 MLOAD DUP DUP DUP MSTORE PUSH1 ARITHMETIC_OP SWAP POP POP PUSH1 MLOAD DUP SWAP ARITHMETIC_OP SWAP RETURN JUMPDEST CALLVALUE DUP ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST POP PUSH2 PUSH2 JUMP JUMPDEST STOP JUMPDEST CALLVALUE DUP ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST POP PUSH2 PUSH2 JUMP JUMPDEST STOP JUMPDEST PUSH1 PUSH1 DUP SLOAD PUSH1 DUP PUSH1 LOGIC_OP ISZERO PUSH2 ARITHMETIC_OP ARITHMETIC_OP LOGIC_OP PUSH1 SWAP ARITHMETIC_OP DUP PUSH1 ARITHMETIC_OP PUSH1 DUP SWAP ARITHMETIC_OP ARITHMETIC_OP PUSH1 ARITHMETIC_OP PUSH1 MLOAD SWAP DUP ARITHMETIC_OP PUSH1 MSTORE DUP SWAP SWAP SWAP DUP DUP MSTORE PUSH1 ARITHMETIC_OP DUP DUP SLOAD PUSH1 DUP PUSH1 LOGIC_OP ISZERO PUSH2 ARITHMETIC_OP ARITHMETIC_OP LOGIC_OP PUSH1 SWAP ARITHMETIC_OP DUP ISZERO PUSH2 JUMPI DUP PUSH1 COMPARASION PUSH2 JUMPI PUSH2 DUP DUP SLOAD ARITHMETIC_OP ARITHMETIC_OP DUP MSTORE SWAP PUSH1 ARITHMETIC_OP SWAP PUSH2 JUMP JUMPDEST DUP ARITHMETIC_OP SWAP SWAP PUSH1 MSTORE PUSH1 PUSH1 SHA3 SWAP JUMPDEST DUP SLOAD DUP MSTORE SWAP PUSH1 ARITHMETIC_OP SWAP PUSH1 ARITHMETIC_OP DUP DUP COMPARASION PUSH2 JUMPI DUP SWAP ARITHMETIC_OP PUSH1 LOGIC_OP DUP ARITHMETIC_OP SWAP JUMPDEST POP POP POP POP POP SWAP POP SWAP JUMP JUMPDEST PUSH1 PUSH2 CONSTANT2 DUP DUP PUSH2 JUMP JUMPDEST PUSH1 SWAP POP SWAP SWAP POP POP JUMP JUMPDEST PUSH1 PUSH1 SLOAD COMPARASION ISZERO ISZERO ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST CONSTANT2 PUSH LOGIC_OP PUSH2 PUSH SWAP DUP ISZERO ARITHMETIC_OP SWAP PUSH1 MLOAD PUSH1 PUSH1 MLOAD DUP DUP ARITHMETIC_OP DUP DUP DUP DUP CALL SWAP POP POP POP POP ISZERO ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST PUSH1 PUSH1 PUSH1 DUP DUP SLOAD ARITHMETIC_OP SWAP POP POP DUP SWAP SSTORE POP JUMP JUMPDEST PUSH1 PUSH1 SLOAD SWAP POP SWAP JUMP JUMPDEST PUSH1 PUSH1 SLOAD COMPARASION ISZERO ISZERO ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST CONSTANT2 PUSH LOGIC_OP PUSH2 PUSH SWAP DUP ISZERO ARITHMETIC_OP SWAP PUSH1 MLOAD PUSH1 PUSH1 MLOAD DUP DUP ARITHMETIC_OP DUP DUP DUP DUP CALL SWAP POP POP POP POP ISZERO ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST PUSH1 PUSH1 PUSH1 DUP DUP SLOAD ARITHMETIC_OP SWAP POP POP DUP SWAP SSTORE POP JUMP JUMPDEST PUSH1 PUSH2 DUP DUP DUP PUSH2 JUMP JUMPDEST PUSH2 DUP CONSTANT2 PUSH2 DUP PUSH1 PUSH1 DUP PUSH LOGIC_OP PUSH LOGIC_OP DUP MSTORE PUSH1 ARITHMETIC_OP SWAP DUP MSTORE PUSH1 ARITHMETIC_OP PUSH1 SHA3 PUSH1 CONSTANT2 PUSH LOGIC_OP PUSH LOGIC_OP DUP MSTORE PUSH1 ARITHMETIC_OP SWAP DUP MSTORE PUSH1 ARITHMETIC_OP PUSH1 SHA3 SLOAD PUSH2 SWAP SWAP SWAP PUSH4 LOGIC_OP JUMP JUMPDEST PUSH2 JUMP JUMPDEST PUSH1 SWAP POP SWAP SWAP POP POP POP JUMP JUMPDEST DUP PUSH1 PUSH1 CONSTANT2 PUSH LOGIC_OP PUSH LOGIC_OP DUP MSTORE PUSH1 ARITHMETIC_OP SWAP DUP MSTORE PUSH1 ARITHMETIC_OP PUSH1 SHA3 SLOAD COMPARASION ISZERO ISZERO ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST CONSTANT2 PUSH LOGIC_OP PUSH2 DUP SWAP DUP ISZERO ARITHMETIC_OP SWAP PUSH1 MLOAD PUSH1 PUSH1 MLOAD DUP DUP ARITHMETIC_OP DUP DUP DUP DUP CALL SWAP POP POP POP POP ISZERO ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST DUP PUSH1 PUSH1 CONSTANT2 PUSH LOGIC_OP PUSH LOGIC_OP DUP MSTORE PUSH1 ARITHMETIC_OP SWAP DUP MSTORE PUSH1 ARITHMETIC_OP PUSH1 SHA3 PUSH1 DUP DUP SLOAD ARITHMETIC_OP SWAP POP POP DUP SWAP SSTORE POP POP JUMP JUMPDEST PUSH1 DUP JUMP JUMPDEST PUSH1 PUSH1 LOGIC_OP PUSH1 ARITHMETIC_OP PUSH ARITHMETIC_OP DUP JUMP JUMPDEST PUSH1 PUSH1 PUSH1 SWAP SLOAD SWAP PUSH2 ARITHMETIC_OP SWAP ARITHMETIC_OP PUSH1 LOGIC_OP SWAP POP SWAP JUMP JUMPDEST PUSH2 CONSTANT2 DUP PUSH2 JUMP JUMPDEST POP JUMP JUMPDEST CONSTANT2 PUSH LOGIC_OP PUSH2 PUSH1 PUSH1 CONSTANT2 PUSH LOGIC_OP PUSH LOGIC_OP DUP MSTORE PUSH1 ARITHMETIC_OP SWAP DUP MSTORE PUSH1 ARITHMETIC_OP PUSH1 SHA3 SLOAD SWAP DUP ISZERO ARITHMETIC_OP SWAP PUSH1 MLOAD PUSH1 PUSH1 MLOAD DUP DUP ARITHMETIC_OP DUP DUP DUP DUP CALL SWAP POP POP POP POP ISZERO ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST PUSH1 PUSH1 PUSH1 CONSTANT2 PUSH LOGIC_OP PUSH LOGIC_OP DUP MSTORE PUSH1 ARITHMETIC_OP SWAP DUP MSTORE PUSH1 ARITHMETIC_OP PUSH1 SHA3 DUP SWAP SSTORE POP JUMP JUMPDEST PUSH1 PUSH1 SWAP SLOAD SWAP PUSH2 ARITHMETIC_OP SWAP ARITHMETIC_OP PUSH LOGIC_OP PUSH LOGIC_OP PUSH2 PUSH1 SLOAD SWAP DUP ISZERO ARITHMETIC_OP SWAP PUSH1 MLOAD PUSH1 PUSH1 MLOAD DUP DUP ARITHMETIC_OP DUP DUP DUP DUP CALL SWAP POP POP POP POP ISZERO ISZERO PUSH2 JUMPI PUSH1 DUP REVERT JUMPDEST CONSTANT2 PUSH1 PUSH1 PUSH2 ARITHMETIC_OP DUP SLOAD DUP PUSH ARITHMETIC_OP LOGIC_OP LOGIC_OP SWAP DUP PUSH LOGIC_OP ARITHMETIC_OP LOGIC_OP SWAP SSTORE POP CONS"]

In [ ]:
Get_vuln(opcode_reentrancy)

(1, 36)
1/1 [==============================] - 0s 32ms/step
[[0.86006624 0.8829907  0.287754   0.40049687 0.1262516 ]] clean
